# Install Requirements

In [13]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached openai-1.65.2-py3-none-any.whl.metadata (27 kB)
  Using cached langchain_core-0.3.40-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_google_genai-2.0.11-py3-none-any.whl.metadata (3.6 kB)
  Using cached langchain_openai-0.3.7-py3-none-any.whl.metadata (2.3 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached langchain_community-0.3.18-py3-none-any.whl.metadata (2.4 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tavily-search (from versions: none)
ERROR: No matching distribution found for tavily-search


# Dependencies

In [2]:
import pandas as pd
import numpy as np
import os, requests, json, csv

In [1]:
from pydantic import BaseModel, Field
from openai import OpenAI

# Initialize Ontology Translator

In [3]:
class OntologyTranslator:
    def __init__(self, json_file):
        # Load JSON data from the file
        with open(json_file, "r", encoding="utf-8") as file:
            data = json.load(file)
        
        # Extract dictionaries
        self.ensembl_to_gene = data.get("ensembl_to_gene", {})
        self.cl_to_cell = data.get("cl_to_cell", {})
        self.uberon_to_tissue = data.get("uberon_to_tissue", {})

        # Create reverse mappings
        self.gene_to_ensembl = {v: k for k, v in self.ensembl_to_gene.items()}
        self.cell_to_cl = {v: k for k, v in self.cl_to_cell.items()}
        self.tissue_to_uberon = {v: k for k, v in self.uberon_to_tissue.items()}

    def tl_ensembl(self, identifier, rev=False):
        """Translate Ensembl ID ↔ Gene Symbol"""
        if rev:
            return self.gene_to_ensembl.get(identifier, None)
        return self.ensembl_to_gene.get(identifier, None)

    def tl_cl(self, identifier, rev=False):
        """Translate Cell Ontology (CL) ID ↔ Cell Type"""
        if rev:
            return self.cell_to_cl.get(identifier, None)
        return self.cl_to_cell.get(identifier, None)

    def tl_uberon(self, identifier, rev=False):
        """Translate UBERON ID ↔ Tissue Type"""
        if rev:
            return self.tissue_to_uberon.get(identifier, None)
        return self.uberon_to_tissue.get(identifier, None)

In [4]:
tl = OntologyTranslator("dictionary.json")

# Load CSV

In [5]:
def csv_read(file_path):
    with open(file_path, 'r') as file:
        sample = file.read(1024) 
        file.seek(0)
        detected_delimiter = csv.Sniffer().sniff(sample).delimiter    
    # Read the file with the detected delimiter
    df = pd.read_csv(file_path, sep=detected_delimiter)
    return df

In [6]:
file_path = "10-top-cluster.csv"
df = csv_read(file_path)

In [7]:
df

,myAUC,avg_diff,power,avg_log2FC,pct.1,pct.2,Cluster,Gene
0,0.939,0.918256,0.878,1.351082,1.000,0.996,0,KRT14
1,0.855,0.713125,0.710,1.219866,1.000,0.848,0,IFI27
2,0.783,0.621616,0.566,1.154583,0.969,0.787,0,COL17A1
3,0.766,0.519959,0.532,1.144090,0.931,0.630,0,HSPA1A
4,0.755,0.507771,0.510,1.248768,0.875,0.509,0,LGALS3BP
...,...,...,...,...,...,...,...,...
260,0.765,0.103585,0.530,0.156168,1.000,0.880,26,COL1A1
261,0.750,0.118977,0.500,0.190017,1.000,0.802,26,COL3A1
262,0.724,0.189142,0.448,0.306940,1.000,0.828,26,SPARC
263,0.703,0.282657,0.406,0.431395,1.000,0.998,26,MT-CYB


# Preprocess Data

## Group Genes by Clusters

In [8]:
def extract_clusters(df):
    # Ensure the column names are correct
    if "Cluster" not in df.columns or "Gene" not in df.columns:
        raise ValueError("The file must contain 'Cluster' and 'Gene' columns")

    # Group the data by clusters and pad genes to ensure columns have equal lengths
    grouped = df.groupby("Cluster")["Gene"].apply(list)

    # Create a DataFrame where each column is a cluster and rows contain the genes
    max_length = max(grouped.apply(len))  # Determine the maximum number of genes in a cluster
    return pd.DataFrame({cluster: genes + [None] * (max_length - len(genes)) for cluster, genes in grouped.items()})

In [9]:
clustered_df = extract_clusters(df)

clustered_df

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,KRT14,TXNIP,LY6D,S100A8,TIMP1,ITM2B,TINAGL1,KRT1,KRT16,KRT14,...,IGHG4,FADS2,IGKC,SPRR1B,IGHG4,FASN,LOR,S100A8,COL3A1,COL1A1
1,IFI27,ATP1B3,KRT14,S100A7,APOD,PERP,TM4SF1,GPX2,SBSN,LY6D,...,IGKC,THRSP,IGHG4,S100A7,IGKC,APOC1,FLG2,S100A7,COL1A1,COL3A1
2,COL17A1,MAF,KRT16,SPRR1B,SAA1,GPNMB,IGFBP6,LTF,LY6D,S100A2,...,MYL9,FADS1,IGHG3,SPRR2D,MALAT1,MGST1,ARG1,CSTA,COL1A2,SPARC
3,HSPA1A,EIF4A2,LGALS7,KRTDAP,IGFBP5,TXNIP,MMP28,CCL27,LGALS7,LGALS7,...,IGHG3,PECR,IGHG1,MUCL1,C1QA,SAA1,LCE1B,KRTDAP,SPARC,MT-CYB
4,LGALS3BP,MMP13,IFI27,SBSN,C3,GJB6,BIRC5,FOSB,KRTDAP,IFI27,...,IGHG1,HSD11B1,JCHAIN,S100A7A,HLA-DRA,AWAT2,KPRP,SPRR1B,POSTN,MT-ND2
5,EFNA1,CAPN2,S100A2,CRABP2,MMP3,ATP1B3,CA2,EPHB6,DMKN,KRT5,...,JCHAIN,HSD3B1,IGHA1,SPRR1A,MT2A,ACSBG1,WFDC12,GJB6,DCN,None
6,RBM42,SULF2,HIST1H1C,SLPI,COMP,CLDN1,CENPW,ATF3,GJB2,TAGLN2,...,IGFBP6,FABP7,IGLC2,IL36RN,MYL9,CYP4F8,LCE1A,MUCL1,COL6A3,None
7,HIST1H1C,CEBPG,LSR,IVL,CXCL12,TNFSF10,SERPINA3,TNFRSF19,HIST1H1C,FXYD3,...,SULF1,ACSM6,IGLC1,LCE3E,IGHG1,TLCD4,LCE2C,SLPI,COL5A2,None
8,SCPEP1,PALLD,FAM83H,PI3,CCL19,DSG3,RRM2,AC103591.3,CRABP2,HSPB1,...,IGHA1,ACOT1,IGHG2,SLC6A14,ATP6V0C,CLMP,LCE2B,S100A7A,PRRX1,None
9,GAMT,PLAU,TMEM79,MUCL1,MMP1,MAF,CCNB2,HLA-DQB2,SLPI,COX6B1,...,GGT5,PKLR,MZB1,SPP1,IGHG3,GLDC,LCE1E,SPP1,COL11A1,None


## Translate Genes into Ensembl ID

In [10]:
def translate_genes(clustered_df):
    # Translate gene names to Ensembl IDs in the clustered DataFrame
    return clustered_df.apply(lambda col: col.map(lambda gene: tl.tl_ensembl(gene, rev=True) if pd.notna(gene) else None))

In [11]:
ensembl_df = translate_genes(clustered_df)
ensembl_df

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,ENSG00000186847,ENSG00000265972,ENSG00000167656,ENSG00000143546,ENSG00000102265,ENSG00000136156,ENSG00000142910,ENSG00000167768,ENSG00000186832,ENSG00000186847,...,ENSG00000211892,ENSG00000134824,ENSG00000211592,ENSG00000169469,ENSG00000211892,ENSG00000169710,None,ENSG00000143546,ENSG00000168542,ENSG00000108821
1,ENSG00000165949,ENSG00000069849,ENSG00000186847,ENSG00000143556,ENSG00000189058,ENSG00000112378,ENSG00000169908,ENSG00000176153,ENSG00000189001,ENSG00000167656,...,ENSG00000211592,ENSG00000151365,ENSG00000211892,ENSG00000143556,ENSG00000211592,ENSG00000130208,ENSG00000143520,ENSG00000143556,ENSG00000108821,ENSG00000168542
2,ENSG00000065618,ENSG00000178573,ENSG00000186832,ENSG00000169469,ENSG00000173432,ENSG00000136235,ENSG00000167779,ENSG00000012223,ENSG00000167656,ENSG00000196754,...,ENSG00000101335,ENSG00000149485,ENSG00000211897,ENSG00000163216,ENSG00000251562,ENSG00000008394,ENSG00000118520,ENSG00000121552,ENSG00000164692,ENSG00000113140
3,ENSG00000204389,ENSG00000156976,ENSG00000205076,ENSG00000188508,ENSG00000115461,ENSG00000265972,ENSG00000271447,ENSG00000213927,ENSG00000205076,ENSG00000205076,...,ENSG00000211897,ENSG00000115425,ENSG00000211896,ENSG00000172551,ENSG00000173372,ENSG00000173432,ENSG00000196734,ENSG00000188508,ENSG00000113140,ENSG00000198727
4,ENSG00000108679,ENSG00000137745,ENSG00000165949,ENSG00000189001,None,ENSG00000121742,ENSG00000089685,ENSG00000125740,ENSG00000188508,ENSG00000165949,...,ENSG00000211896,ENSG00000117594,ENSG00000132465,ENSG00000184330,ENSG00000204287,ENSG00000147160,ENSG00000203786,ENSG00000169469,ENSG00000133110,ENSG00000198763
5,ENSG00000169242,ENSG00000162909,ENSG00000196754,ENSG00000143320,ENSG00000149968,ENSG00000069849,ENSG00000104267,ENSG00000106123,ENSG00000161249,ENSG00000186081,...,ENSG00000132465,ENSG00000203857,ENSG00000211895,ENSG00000169474,ENSG00000125148,ENSG00000103740,ENSG00000168703,ENSG00000121742,ENSG00000011465,None
6,ENSG00000126254,ENSG00000196562,None,ENSG00000124107,ENSG00000105664,ENSG00000163347,ENSG00000203760,ENSG00000162772,ENSG00000165474,ENSG00000158710,...,ENSG00000167779,ENSG00000164434,ENSG00000211677,ENSG00000136695,ENSG00000101335,ENSG00000186526,ENSG00000186844,ENSG00000172551,ENSG00000163359,None
7,None,ENSG00000153879,ENSG00000105699,ENSG00000163207,ENSG00000107562,ENSG00000121858,ENSG00000196136,ENSG00000127863,None,ENSG00000089356,...,ENSG00000137573,ENSG00000173124,ENSG00000211675,ENSG00000185966,ENSG00000211896,ENSG00000152078,ENSG00000187180,ENSG00000124107,ENSG00000204262,None
8,ENSG00000121064,ENSG00000129116,ENSG00000180921,ENSG00000124102,ENSG00000172724,ENSG00000134757,ENSG00000171848,None,ENSG00000143320,ENSG00000106211,...,ENSG00000211895,ENSG00000184227,ENSG00000211893,ENSG00000268104,ENSG00000185883,ENSG00000166250,ENSG00000159455,ENSG00000184330,ENSG00000116132,None
9,ENSG00000130005,ENSG00000122861,ENSG00000163472,ENSG00000172551,ENSG00000196611,ENSG00000178573,ENSG00000157456,ENSG00000232629,ENSG00000124107,ENSG00000126267,...,ENSG00000099998,ENSG00000143627,ENSG00000170476,ENSG00000118785,ENSG00000211897,ENSG00000178445,ENSG00000186226,ENSG00000118785,ENSG00000060718,None


# Statistical Analysis with CellxGene's Gene Expression Data

## Function Initialization

Function to fetch gene expression data

In [12]:
def fetch_expression_data(cluster, ensembl_ids):
    """
    Fetches gene expression data from the Cellxgene API for a given cluster and list of Ensembl IDs.
    
    :param cluster: The cluster name.
    :param ensembl_ids: List of Ensembl IDs for the cluster.
    :return: JSON response from the API or None if an error occurred.
    """

    payload = {
        "filter": {
            "dataset_ids": [],
            "development_stage_ontology_term_ids": [],
            "disease_ontology_term_ids": [],
            "gene_ontology_term_ids": ensembl_ids,
            "organism_ontology_term_id": "NCBITaxon:9606",
            "self_reported_ethnicity_ontology_term_ids": [],
            "sex_ontology_term_ids": [],
            "publication_citations": [],
        },
        "is_rollup": True
    }

    # URL for the POST request
    API_URL = "https://api.cellxgene.cziscience.com/wmg/v2/query"

    try:
        response = requests.post(API_URL, json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.json()['expression_summary']
    except requests.RequestException as e:
        print(f"Error fetching data for Cluster {cluster}: {e}")
        return None

Function to convert the data to a DataFrame

In [13]:
def expression_data_to_df(data):
    flattened_data = []
    for gene_id, anatomical_structures in data.items():
        for anatomical_id, cell_types in anatomical_structures.items():
            for cell_type_id, aggregated_data in cell_types.items():
                metrics = aggregated_data['aggregated']
                flattened_data.append({
                    'gene': gene_id,
                    'tissue': anatomical_id,
                    'cell': cell_type_id,
                    'expression': metrics.get('me', None),
                    'cell count': metrics.get('n', None),
                    'cell percentage': metrics.get('pc', None),
                    'tissue composition': metrics.get('tpc', None)
                })
    return pd.DataFrame(flattened_data)

Function to filter the data

In [14]:
def filter_expression_data(response_df, TARGET_UBERON_ID):
    tissue_df = response_df[response_df['tissue'] == TARGET_UBERON_ID].drop(columns=['tissue'])
    return tissue_df[~tissue_df['cell'].isin(['tissue_stats', 'CL:0000000'])]

Function to translate the ontology terms

In [15]:
def translate_ontology(df):
    df['cell'] = df['cell'].apply(lambda x: tl.tl_cl(x))
    df['gene'] = df['gene'].apply(lambda x: tl.tl_ensembl(x)).fillna(df['gene'])

    return df

In [16]:
def transform_results(results):
    # Count the number of entries for each cell
    cell_counts = results['cell'].value_counts().reset_index()
    cell_counts.columns = ['cell', 'count']

    # Merge the counts back to the original DataFrame
    merged_df = results.merge(cell_counts, on='cell')

    # Sort the DataFrame based on the count of entries in descending order
    return merged_df.groupby(['cell', 'gene']).first().sort_values(by=['count', 'cell', 'expression', 'cell percentage'], ascending=[False, True, False, False]).reset_index().drop(columns=['count'])

In [17]:
def calculate_cell_score(results):
    cell_df = results.copy()

    # Group by 'cell' and calculate the total cell count for each group
    sum_cell_count = cell_df.groupby('cell')['cell count'].sum().reset_index()
    sum_cell_count.columns = ['cell', 'total cell count']
    cell_df = cell_df.merge(sum_cell_count, on='cell')

    cell_df = cell_df[cell_df['total cell count'] >= 100]

    # Calculate scores
    cell_df['score'] = (1) * ((cell_df['expression'] - cell_df['expression'].min()) / (cell_df['expression'].max() - cell_df['expression'].min())) + (1.5) * (cell_df['cell percentage']) + (2.5) * ((cell_df['cell count'] - cell_df['cell count'].min()) / (cell_df['cell count'].max() - cell_df['cell count'].min()))

    # Group by 'cell' and calculate the standard deviation of 'score' for each group
    std_scores = cell_df.groupby('cell')['score'].std().reset_index()
    std_scores.columns = ['cell', 'std']
    cell_df = cell_df.merge(std_scores, on='cell')

    # Group by 'cell' and calculate the mean of 'score' for each group
    mean_scores = cell_df.groupby('cell')['score'].mean().reset_index().fillna(0)
    mean_scores.columns = ['cell', 'mean']
    cell_df = cell_df.merge(mean_scores, on='cell')

    # Calculate the coefficient of variation (CV) for each cell
    cell_df['CV'] = cell_df['std'] / cell_df['mean']

    # Group by 'cell' and calculate the kurtosis of 'score' for each group
    kurtosis_scores = cell_df.groupby('cell')['score'].apply(pd.Series.kurt).reset_index()
    kurtosis_scores.columns = ['cell', 'kurtosis']
    cell_df = cell_df.merge(kurtosis_scores, on='cell')

    # Normalize the scores
    cell_df['std norm'] = (cell_df['std'] - cell_df['std'].min()) / (cell_df['std'].max() - cell_df['std'].min())
    cell_df['mean norm'] = (cell_df['mean'] - cell_df['mean'].min()) / (cell_df['mean'].max() - cell_df['mean'].min())
    cell_df['CV norm'] = (cell_df['CV'] - cell_df['CV'].min()) / (cell_df['CV'].max() - cell_df['CV'].min())
    cell_df['kurtosis norm'] = ((cell_df['kurtosis'] - cell_df['kurtosis'].min()) / (cell_df['kurtosis'].max() - cell_df['kurtosis'].min()))

    # Count the number of entries for each cell
    cell_counts = cell_df['cell'].value_counts().reset_index()
    cell_counts.columns = ['cell', 'count']
    cell_df = cell_df.merge(cell_counts, on='cell')

    cell_df['cell score'] = (0.1 * (1 - cell_df['std norm']).fillna(0) + 0.7 * cell_df['mean norm'] + 0.1 * (1 - cell_df['CV norm']) + 0.1 * (1 - cell_df['kurtosis norm']).fillna(0)) * cell_df['count']
    return cell_df.sort_values(by=['count', 'cell score', 'cell', 'gene'], ascending=[False, False, False, True]).reset_index(drop=True)

In [18]:
def run_data_processing(cluster, ensembl_ids, target_uberon_id):
    try:
        # Step 1: Fetch expression data
        data = fetch_expression_data(cluster, ensembl_ids)

        # Step 2: Convert data to DataFrame
        response_df = expression_data_to_df(data)

        # Step 3: Filter data based on target UBERON ID
        filtered_df = filter_expression_data(response_df, target_uberon_id)

        # Step 4: Translate ontology terms
        translated_df = translate_ontology(filtered_df)

        # Step 5: Transform the results
        # transformed_df = transform_results(translated_df)

        # Step 6: Rank the cells based on the calculated score
        ranked_df = calculate_cell_score(translated_df)

        return ranked_df

    except Exception as e:
        print(f"An error occurred during the pipeline execution: {e}")
        return pd.DataFrame()

In [19]:
def main_data_analysis(ensembl_df, target_uberon_id):
    results = {}

    # Iterate over each cluster (column) in the DataFrame
    for cluster in ensembl_df.columns:
        # Extract Ensembl IDs for the current cluster, dropping any NaN values
        ensembl_ids = ensembl_df[cluster].dropna().tolist()
        
        if not ensembl_ids:
            continue
        
        # Run the expression pipeline for the current cluster
        result_df = run_data_processing(cluster, ensembl_ids, target_uberon_id)
        
        # Store the result in the dictionary
        results[cluster] = result_df.reset_index(drop=True)

    return results

Function to Display the Results

In [20]:
from IPython.display import Markdown, display
def display_dataframe_as_markdown(df):
    markdown_table = df.to_markdown(index=False)
    display(Markdown(markdown_table))

## Run the Section

In [21]:
# Specify the UBERON ID to filter
TARGET_UBERON_ID = "UBERON:0002097"

In [22]:
# Run the main pipeline
results = main_data_analysis(ensembl_df, TARGET_UBERON_ID)

In [23]:
display_dataframe_as_markdown(results[0])

| gene     | cell                                                    |   expression |   cell count |   cell percentage |   tissue composition |   total cell count |       score |       std |      mean |       CV |   kurtosis |   std norm |   mean norm |   CV norm |   kurtosis norm |   count |   cell score |
|:---------|:--------------------------------------------------------|-------------:|-------------:|------------------:|---------------------:|-------------------:|------------:|----------:|----------:|---------:|-----------:|-----------:|------------:|----------:|----------------:|--------:|-------------:|
| COL17A1  | epithelial cell                                         |      2.24252 |        53112 |       0.295526    |          0.0653639   |             348909 | 1.91273     | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| EFNA1    | epithelial cell                                         |      1.83681 |        49963 |       0.277986    |          0.0614885   |             348909 | 1.71185     | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| GAMT     | epithelial cell                                         |      1.62434 |         4490 |       0.0253536   |          0.00552575  |             348909 | 0.200841    | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| HSPA1A   | epithelial cell                                         |      2.55937 |        91631 |       0.509398    |          0.112768    |             348909 | 3.22615     | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| IFI27    | epithelial cell                                         |      2.64729 |        35789 |       0.199125    |          0.0440448   |             348909 | 1.45635     | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| KRT14    | epithelial cell                                         |      4.50854 |       105283 |       0.588226    |          0.12957     |             348909 | 4.14805     | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| LGALS3BP | epithelial cell                                         |      1.62171 |         1670 |       0.00929549  |          0.00205524  |             348909 | 0.109144    | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| RBM42    | epithelial cell                                         |      1.62746 |         4885 |       0.0274353   |          0.00601187  |             348909 | 0.214111    | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| SCPEP1   | epithelial cell                                         |      1.75099 |         2086 |       0.0115973   |          0.0025672   |             348909 | 0.154275    | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| COL17A1  | epidermal cell                                          |      2.24453 |        52858 |       0.389074    |          0.0650513   |             294917 | 2.04752     | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| EFNA1    | epidermal cell                                          |      1.83415 |        37420 |       0.275439    |          0.046052    |             294917 | 1.40953     | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| GAMT     | epidermal cell                                          |      1.62195 |         4435 |       0.033142    |          0.00545807  |             294917 | 0.210628    | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| HSPA1A   | epidermal cell                                          |      2.57748 |        81735 |       0.60163     |          0.10059     |             294917 | 3.13396     | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| IFI27    | epidermal cell                                          |      2.14654 |         9126 |       0.0671741   |          0.0112312   |             294917 | 0.502115    | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| KRT14    | epidermal cell                                          |      4.53267 |       103908 |       0.764839    |          0.127877    |             294917 | 4.38626     | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| LGALS3BP | epidermal cell                                          |      1.60195 |         1509 |       0.0111247   |          0.0018571   |             294917 | 0.103203    | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| RBM42    | epidermal cell                                          |      1.62528 |         2100 |       0.0156681   |          0.00258443  |             294917 | 0.12979     | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| SCPEP1   | epidermal cell                                          |      1.76011 |         1826 |       0.0134407   |          0.00224722  |             294917 | 0.153111    | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| COL17A1  | squamous epithelial cell                                |      2.24266 |        52717 |       0.387278    |          0.0648778   |             294999 | 2.04102     | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| EFNA1    | squamous epithelial cell                                |      1.83331 |        37570 |       0.27598     |          0.0462366   |             294999 | 1.41369     | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| GAMT     | squamous epithelial cell                                |      1.62028 |         4422 |       0.0331614   |          0.00544207  |             294999 | 0.209937    | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| HSPA1A   | squamous epithelial cell                                |      2.577   |        81720 |       0.600295    |          0.100571    |             294999 | 3.13148     | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| IFI27    | squamous epithelial cell                                |      2.1366  |         9401 |       0.0690575   |          0.0115696   |             294999 | 0.509025    | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| KRT14    | squamous epithelial cell                                |      4.53433 |       103735 |       0.766217    |          0.127665    |             294999 | 4.38462     | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| LGALS3BP | squamous epithelial cell                                |      1.601   |         1513 |       0.0111324   |          0.00186202  |             294999 | 0.103075    | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| RBM42    | squamous epithelial cell                                |      1.62399 |         2104 |       0.0156655   |          0.00258935  |             294999 | 0.129565    | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| SCPEP1   | squamous epithelial cell                                |      1.75054 |         1817 |       0.0133483   |          0.00223615  |             294999 | 0.150404    | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| COL17A1  | stratified squamous epithelial cell                     |      2.2427  |        52713 |       0.389353    |          0.0648728   |             294413 | 2.04404     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| EFNA1    | stratified squamous epithelial cell                     |      1.83356 |        37353 |       0.2759      |          0.0459696   |             294413 | 1.40848     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| GAMT     | stratified squamous epithelial cell                     |      1.62028 |         4422 |       0.0331614   |          0.00544207  |             294413 | 0.209937    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| HSPA1A   | stratified squamous epithelial cell                     |      2.57731 |        81690 |       0.603386    |          0.100534    |             294413 | 3.13549     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| IFI27    | stratified squamous epithelial cell                     |      2.14633 |         9121 |       0.0673703   |          0.011225    |             294413 | 0.502237    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| KRT14    | stratified squamous epithelial cell                     |      4.53433 |       103735 |       0.766217    |          0.127665    |             294413 | 4.38462     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| LGALS3BP | stratified squamous epithelial cell                     |      1.60042 |         1506 |       0.0111412   |          0.0018534   |             294413 | 0.102781    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| RBM42    | stratified squamous epithelial cell                     |      1.62156 |         2079 |       0.015566    |          0.00255858  |             294413 | 0.128225    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| SCPEP1   | stratified squamous epithelial cell                     |      1.75095 |         1794 |       0.013251    |          0.00220784  |             294413 | 0.149813    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| COL17A1  | stratified epithelial cell                              |      2.2427  |        52713 |       0.389353    |          0.0648728   |             294413 | 2.04404     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| EFNA1    | stratified epithelial cell                              |      1.83356 |        37353 |       0.2759      |          0.0459696   |             294413 | 1.40848     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| GAMT     | stratified epithelial cell                              |      1.62028 |         4422 |       0.0331614   |          0.00544207  |             294413 | 0.209937    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| HSPA1A   | stratified epithelial cell                              |      2.57731 |        81690 |       0.603386    |          0.100534    |             294413 | 3.13549     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| IFI27    | stratified epithelial cell                              |      2.14633 |         9121 |       0.0673703   |          0.011225    |             294413 | 0.502237    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| KRT14    | stratified epithelial cell                              |      4.53433 |       103735 |       0.766217    |          0.127665    |             294413 | 4.38462     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| LGALS3BP | stratified epithelial cell                              |      1.60042 |         1506 |       0.0111412   |          0.0018534   |             294413 | 0.102781    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| RBM42    | stratified epithelial cell                              |      1.62156 |         2079 |       0.015566    |          0.00255858  |             294413 | 0.128225    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| SCPEP1   | stratified epithelial cell                              |      1.75095 |         1794 |       0.013251    |          0.00220784  |             294413 | 0.149813    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| COL17A1  | keratinocyte                                            |      2.2427  |        52713 |       0.389353    |          0.0648728   |             294413 | 2.04404     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| EFNA1    | keratinocyte                                            |      1.83356 |        37353 |       0.2759      |          0.0459696   |             294413 | 1.40848     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| GAMT     | keratinocyte                                            |      1.62028 |         4422 |       0.0331614   |          0.00544207  |             294413 | 0.209937    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| HSPA1A   | keratinocyte                                            |      2.57731 |        81690 |       0.603386    |          0.100534    |             294413 | 3.13549     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| IFI27    | keratinocyte                                            |      2.14633 |         9121 |       0.0673703   |          0.011225    |             294413 | 0.502237    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| KRT14    | keratinocyte                                            |      4.53433 |       103735 |       0.766217    |          0.127665    |             294413 | 4.38462     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| LGALS3BP | keratinocyte                                            |      1.60042 |         1506 |       0.0111412   |          0.0018534   |             294413 | 0.102781    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| RBM42    | keratinocyte                                            |      1.62156 |         2079 |       0.015566    |          0.00255858  |             294413 | 0.128225    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| SCPEP1   | keratinocyte                                            |      1.75095 |         1794 |       0.013251    |          0.00220784  |             294413 | 0.149813    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| COL17A1  | basal cell of epidermis                                 |      2.2197  |        24534 |       0.785465    |          0.0301935   |              85041 | 1.96342     | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| EFNA1    | basal cell of epidermis                                 |      1.71246 |         7133 |       0.228366    |          0.00877844  |              85041 | 0.589794    | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| GAMT     | basal cell of epidermis                                 |      1.60329 |         2228 |       0.0713302   |          0.00274195  |              85041 | 0.210913    | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| HSPA1A   | basal cell of epidermis                                 |      2.46056 |        20246 |       0.648183    |          0.0249163   |              85041 | 1.71493     | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| IFI27    | basal cell of epidermis                                 |      2.19921 |         2083 |       0.066688    |          0.00256351  |              85041 | 0.347101    | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| KRT14    | basal cell of epidermis                                 |      5.4584  |        27283 |       0.873475    |          0.0335766   |              85041 | 2.95742     | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| LGALS3BP | basal cell of epidermis                                 |      1.59463 |          913 |       0.02923     |          0.00112361  |              85041 | 0.114408    | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| RBM42    | basal cell of epidermis                                 |      1.59668 |          317 |       0.0101489   |          0.000390126 |              85041 | 0.0721381   | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| SCPEP1   | basal cell of epidermis                                 |      1.71152 |          304 |       0.00973267  |          0.000374127 |              85041 | 0.0994544   | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| COL17A1  | stem cell of epidermis                                  |      2.21458 |        25193 |       0.77725     |          0.0310045   |              87642 | 1.96549     | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| EFNA1    | stem cell of epidermis                                  |      1.71224 |         7251 |       0.223707    |          0.00892366  |              87642 | 0.585554    | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| GAMT     | stem cell of epidermis                                  |      1.60667 |         2296 |       0.0708358   |          0.00282564  |              87642 | 0.212618    | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| HSPA1A   | stem cell of epidermis                                  |      2.451   |        20598 |       0.635486    |          0.0253495   |              87642 | 1.70189     | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| IFI27    | stem cell of epidermis                                  |      2.17103 |         2230 |       0.0687996   |          0.00274442  |              87642 | 0.346827    | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| KRT14    | stem cell of epidermis                                  |      5.46095 |        28455 |       0.877889    |          0.035019    |              87642 | 2.99249     | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| LGALS3BP | stem cell of epidermis                                  |      1.59595 |          937 |       0.0289082   |          0.00115315  |              87642 | 0.11482     | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| RBM42    | stem cell of epidermis                                  |      1.59951 |          358 |       0.011045    |          0.000440583 |              87642 | 0.0751511   | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| SCPEP1   | stem cell of epidermis                                  |      1.70432 |          324 |       0.00999599  |          0.00039874  |              87642 | 0.0985531   | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| COL17A1  | stem cell                                               |      2.21421 |        25216 |       0.694426    |          0.0310328   |              88427 | 1.84171     | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| EFNA1    | stem cell                                               |      1.7116  |         7758 |       0.211973    |          0.00954761  |              88427 | 0.579836    | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| GAMT     | stem cell                                               |      1.6117  |         2309 |       0.0705167   |          0.00284164  |              88427 | 0.213685    | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| HSPA1A   | stem cell                                               |      2.45247 |        20738 |       0.565885    |          0.0255218   |              88427 | 1.60117     | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| IFI27    | stem cell                                               |      2.16988 |         2235 |       0.0615499   |          0.00275057  |              88427 | 0.335788    | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| KRT14    | stem cell                                               |      5.45935 |        28467 |       0.783956    |          0.0350338   |              88427 | 2.85149     | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| LGALS3BP | stem cell                                               |      1.59592 |          961 |       0.0264651   |          0.00118268  |              88427 | 0.111717    | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| RBM42    | stem cell                                               |      1.65534 |          395 |       0.0107926   |          0.000486119 |              88427 | 0.0893862   | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| SCPEP1   | stem cell                                               |      1.76078 |          348 |       0.00949704  |          0.000428277 |              88427 | 0.112263    | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| COL17A1  | epithelial fate stem cell                               |      2.21932 |        24557 |       0.698953    |          0.0302218   |              85652 | 1.83411     | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| EFNA1    | epithelial fate stem cell                               |      1.71104 |         7635 |       0.217311    |          0.00939624  |              85652 | 0.584783    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| GAMT     | epithelial fate stem cell                               |      1.60329 |         2228 |       0.0713302   |          0.00274195  |              85652 | 0.210913    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| HSPA1A   | epithelial fate stem cell                               |      2.45952 |        20276 |       0.577105    |          0.0249533   |              85652 | 1.60877     | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| IFI27    | epithelial fate stem cell                               |      2.19791 |         2088 |       0.0594296   |          0.00256966  |              85652 | 0.336011    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| KRT14    | epithelial fate stem cell                               |      5.45673 |        27295 |       0.776883    |          0.0335914   |              85652 | 2.8124      | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| LGALS3BP | epithelial fate stem cell                               |      1.59463 |          937 |       0.0266693   |          0.00115315  |              85652 | 0.111137    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| RBM42    | epithelial fate stem cell                               |      1.59714 |          331 |       0.00942107  |          0.000407355 |              85652 | 0.0714925   | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| SCPEP1   | epithelial fate stem cell                               |      1.7119  |          305 |       0.00868105  |          0.000375357 |              85652 | 0.0979937   | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| COL17A1  | basal cell                                              |      2.21932 |        24557 |       0.698953    |          0.0302218   |              85652 | 1.83411     | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| EFNA1    | basal cell                                              |      1.71104 |         7635 |       0.217311    |          0.00939624  |              85652 | 0.584783    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| GAMT     | basal cell                                              |      1.60329 |         2228 |       0.0713302   |          0.00274195  |              85652 | 0.210913    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| HSPA1A   | basal cell                                              |      2.45952 |        20276 |       0.577105    |          0.0249533   |              85652 | 1.60877     | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| IFI27    | basal cell                                              |      2.19791 |         2088 |       0.0594296   |          0.00256966  |              85652 | 0.336011    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| KRT14    | basal cell                                              |      5.45673 |        27295 |       0.776883    |          0.0335914   |              85652 | 2.8124      | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| LGALS3BP | basal cell                                              |      1.59463 |          937 |       0.0266693   |          0.00115315  |              85652 | 0.111137    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| RBM42    | basal cell                                              |      1.59714 |          331 |       0.00942107  |          0.000407355 |              85652 | 0.0714925   | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| SCPEP1   | basal cell                                              |      1.7119  |          305 |       0.00868105  |          0.000375357 |              85652 | 0.0979937   | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| COL17A1  | progenitor cell                                         |      2.21943 |        24548 |       0.611362    |          0.0302107   |              89930 | 1.70253     | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| EFNA1    | progenitor cell                                         |      1.71297 |         7366 |       0.18365     |          0.00906519  |              89930 | 0.528379    | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| GAMT     | progenitor cell                                         |      1.60395 |         2337 |       0.0563961   |          0.0028761   |              89930 | 0.191264    | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| HSPA1A   | progenitor cell                                         |      2.61188 |        23720 |       0.571745    |          0.0291917   |              89930 | 1.71999     | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| IFI27    | progenitor cell                                         |      2.19649 |         2091 |       0.0519774   |          0.00257335  |              89930 | 0.324555    | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| KRT14    | progenitor cell                                         |      5.45703 |        27293 |       0.680471    |          0.0335889   |              89930 | 2.66781     | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| LGALS3BP | progenitor cell                                         |      1.59831 |          923 |       0.0228703   |          0.00113592  |              89930 | 0.106011    | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| RBM42    | progenitor cell                                         |      1.59964 |          590 |       0.0142809   |          0.000726101 |              89930 | 0.0855454   | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| SCPEP1   | progenitor cell                                         |      1.68384 |         1062 |       0.0257417   |          0.00130698  |              89930 | 0.134659    | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| COL17A1  | precursor cell                                          |      2.21395 |        25230 |       0.557816    |          0.0310501   |              96282 | 1.63706     | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| EFNA1    | precursor cell                                          |      1.71185 |         8006 |       0.156242    |          0.00985282  |              96282 | 0.50219     | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| GAMT     | precursor cell                                          |      1.61113 |         2659 |       0.054631    |          0.00327238  |              96282 | 0.198029    | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| HSPA1A   | precursor cell                                          |      2.63742 |        26038 |       0.49484     |          0.0320444   |              96282 | 1.66596     | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| IFI27    | precursor cell                                          |      2.16309 |         2260 |       0.0442495   |          0.00278134  |              96282 | 0.308759    | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| KRT14    | precursor cell                                          |      5.45803 |        28477 |       0.630217    |          0.0350461   |              96282 | 2.62079     | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| LGALS3BP | precursor cell                                          |      1.59784 |         1076 |       0.0210144   |          0.00132421  |              96282 | 0.106745    | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| RBM42    | precursor cell                                          |      1.62371 |         1323 |       0.0252259   |          0.00162819  |              96282 | 0.125289    | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| SCPEP1   | precursor cell                                          |      1.68718 |         1213 |       0.0231542   |          0.00149281  |              96282 | 0.135184    | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| COL17A1  | prickle cell                                            |      1.80445 |         2147 |       0.0815698   |          0.00264227  |              61334 | 0.273835    | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| EFNA1    | prickle cell                                            |      1.75523 |        10520 |       0.399681    |          0.0129468   |              61334 | 0.937716    | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| GAMT     | prickle cell                                            |      1.51306 |          131 |       0.00497701  |          0.000161219 |              61334 | 0.0393929   | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| HSPA1A   | prickle cell                                            |      2.42382 |        19968 |       0.758634    |          0.0245742   |              61334 | 1.86496     | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| IFI27    | prickle cell                                            |      2.21559 |         5086 |       0.19323     |          0.00625924  |              61334 | 0.612252    | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| KRT14    | prickle cell                                            |      3.27004 |        23107 |       0.877892    |          0.0284373   |              61334 | 2.32656     | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| LGALS3BP | prickle cell                                            |      1.53987 |          163 |       0.00619277  |          0.000200601 |              61334 | 0.0485727   | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| RBM42    | prickle cell                                            |      1.50598 |          153 |       0.00581285  |          0.000188294 |              61334 | 0.0394271   | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| SCPEP1   | prickle cell                                            |      1.52625 |           59 |       0.00224156  |          7.26101e-05 |              61334 | 0.036824    | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| COL17A1  | skin fibroblast                                         |      1.8836  |          575 |       0.00508503  |          0.000707641 |              77696 | 0.14125     | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| EFNA1    | skin fibroblast                                         |      1.79486 |         1375 |       0.0121599   |          0.00169218  |              77696 | 0.149027    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| GAMT     | skin fibroblast                                         |      1.73991 |         2775 |       0.0245408   |          0.00341514  |              77696 | 0.187327    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| HSPA1A   | skin fibroblast                                         |      2.78309 |         9483 |       0.0838632   |          0.0116705   |              77696 | 0.692213    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| IFI27    | skin fibroblast                                         |      2.37941 |         7002 |       0.0619224   |          0.00861722  |              77696 | 0.501086    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| KRT14    | skin fibroblast                                         |      2.22303 |        11550 |       0.102143    |          0.0142144   |              77696 | 0.630944    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| LGALS3BP | skin fibroblast                                         |      1.89644 |        20796 |       0.18391     |          0.0255932   |              77696 | 0.892807    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| RBM42    | skin fibroblast                                         |      1.75585 |         6708 |       0.0593224   |          0.0082554   |              77696 | 0.336812    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| SCPEP1   | skin fibroblast                                         |      1.81023 |        17432 |       0.15416     |          0.0214532   |              77696 | 0.747096    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| COL17A1  | columnar/cuboidal epithelial cell                       |      1.86484 |          267 |       0.234211    |          0.000328592 |               3149 | 0.473009    | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| EFNA1    | columnar/cuboidal epithelial cell                       |      1.80474 |          366 |       0.320771    |          0.000450429 |               3149 | 0.590417    | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| GAMT     | columnar/cuboidal epithelial cell                       |      1.72735 |            2 |       0.00175439  |          2.46136e-06 |               3149 | 0.0842105   | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| HSPA1A   | columnar/cuboidal epithelial cell                       |      2.40445 |          891 |       0.780894    |          0.00109654  |               3149 | 1.44059     | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| IFI27    | columnar/cuboidal epithelial cell                       |      1.78471 |            1 |       0.000877193 |          1.23068e-06 |               3149 | 0.096981    | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| KRT14    | columnar/cuboidal epithelial cell                       |      4.63062 |         1139 |       0.999123    |          0.00140174  |               3149 | 2.32145     | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| LGALS3BP | columnar/cuboidal epithelial cell                       |      1.66691 |           10 |       0.00877193  |          1.23068e-05 |               3149 | 0.0800587   | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| RBM42    | columnar/cuboidal epithelial cell                       |      1.57944 |           16 |       0.0140351   |          1.96909e-05 |               3149 | 0.0665777   | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| SCPEP1   | columnar/cuboidal epithelial cell                       |      1.83831 |          457 |       0.400877    |          0.000562421 |               3149 | 0.720993    | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| COL17A1  | neuron associated cell (sensu Vertebrata)               |      1.86484 |          267 |       0.234211    |          0.000328592 |               3147 | 0.473009    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| EFNA1    | neuron associated cell (sensu Vertebrata)               |      1.80201 |          365 |       0.320175    |          0.000449198 |               3147 | 0.588827    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| GAMT     | neuron associated cell (sensu Vertebrata)               |      1.72735 |            2 |       0.00175439  |          2.46136e-06 |               3147 | 0.0842105   | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| HSPA1A   | neuron associated cell (sensu Vertebrata)               |      2.40281 |          890 |       0.780702    |          0.00109531  |               3147 | 1.43988     | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| IFI27    | neuron associated cell (sensu Vertebrata)               |      1.78471 |            1 |       0.000877193 |          1.23068e-06 |               3147 | 0.096981    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| KRT14    | neuron associated cell (sensu Vertebrata)               |      4.63062 |         1139 |       0.999123    |          0.00140174  |               3147 | 2.32145     | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| LGALS3BP | neuron associated cell (sensu Vertebrata)               |      1.66691 |           10 |       0.00877193  |          1.23068e-05 |               3147 | 0.0800587   | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| RBM42    | neuron associated cell (sensu Vertebrata)               |      1.57944 |           16 |       0.0140351   |          1.96909e-05 |               3147 | 0.0665777   | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| SCPEP1   | neuron associated cell (sensu Vertebrata)               |      1.83831 |          457 |       0.400877    |          0.000562421 |               3147 | 0.720993    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| COL17A1  | Merkel cell                                             |      1.86484 |          267 |       0.234211    |          0.000328592 |               3147 | 0.473009    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| EFNA1    | Merkel cell                                             |      1.80201 |          365 |       0.320175    |          0.000449198 |               3147 | 0.588827    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| GAMT     | Merkel cell                                             |      1.72735 |            2 |       0.00175439  |          2.46136e-06 |               3147 | 0.0842105   | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| HSPA1A   | Merkel cell                                             |      2.40281 |          890 |       0.780702    |          0.00109531  |               3147 | 1.43988     | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| IFI27    | Merkel cell                                             |      1.78471 |            1 |       0.000877193 |          1.23068e-06 |               3147 | 0.096981    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| KRT14    | Merkel cell                                             |      4.63062 |         1139 |       0.999123    |          0.00140174  |               3147 | 2.32145     | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| LGALS3BP | Merkel cell                                             |      1.66691 |           10 |       0.00877193  |          1.23068e-05 |               3147 | 0.0800587   | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| RBM42    | Merkel cell                                             |      1.57944 |           16 |       0.0140351   |          1.96909e-05 |               3147 | 0.0665777   | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| SCPEP1   | Merkel cell                                             |      1.83831 |          457 |       0.400877    |          0.000562421 |               3147 | 0.720993    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| COL17A1  | connective tissue cell                                  |      1.80107 |         1627 |       0.00638012  |          0.00200232  |             149479 | 0.147871    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| EFNA1    | connective tissue cell                                  |      1.76844 |         6200 |       0.0237699   |          0.00763022  |             149479 | 0.274517    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| GAMT     | connective tissue cell                                  |      1.65474 |         7145 |       0.0273752   |          0.00879321  |             149479 | 0.274395    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| HSPA1A   | connective tissue cell                                  |      2.63005 |        55814 |       0.213821    |          0.0686892   |             149479 | 1.94967     | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| IFI27    | connective tissue cell                                  |      2.29561 |         8640 |       0.0331395   |          0.0106331   |             149479 | 0.476193    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| KRT14    | connective tissue cell                                  |      2.22595 |        12669 |       0.0486246   |          0.0155915   |             149479 | 0.577956    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| LGALS3BP | connective tissue cell                                  |      1.87572 |        23415 |       0.0898007   |          0.0288164   |             149479 | 0.808737    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| RBM42    | connective tissue cell                                  |      1.69745 |        13559 |       0.051944    |          0.0166868   |             149479 | 0.474061    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| SCPEP1   | connective tissue cell                                  |      1.78669 |        20410 |       0.0781983   |          0.0251182   |             149479 | 0.698078    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| COL17A1  | fibroblast                                              |      1.81969 |         1195 |       0.0051064   |          0.00147066  |             135486 | 0.140282    | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| EFNA1    | fibroblast                                              |      1.77218 |         4137 |       0.017678    |          0.00509132  |             135486 | 0.217313    | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| GAMT     | fibroblast                                              |      1.64918 |         6818 |       0.0291132   |          0.00839078  |             135486 | 0.26787     | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| HSPA1A   | fibroblast                                              |      2.63467 |        49302 |       0.210522    |          0.060675    |             135486 | 1.79122     | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| IFI27    | fibroblast                                              |      2.32391 |         7836 |       0.0334602   |          0.00964361  |             135486 | 0.464544    | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| KRT14    | fibroblast                                              |      2.21882 |        11765 |       0.0502735   |          0.0144789   |             135486 | 0.557208    | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| LGALS3BP | fibroblast                                              |      1.88332 |        22223 |       0.0948934   |          0.0273494   |             135486 | 0.78994     | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| RBM42    | fibroblast                                              |      1.69493 |        12395 |       0.0529273   |          0.0152543   |             135486 | 0.447276    | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| SCPEP1   | fibroblast                                              |      1.78663 |        19815 |       0.0846111   |          0.0243859   |             135486 | 0.693552    | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| COL17A1  | neoplastic cell                                         |      1.99107 |          282 |       0.0116274   |          0.000347052 |              21536 | 0.170543    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| EFNA1    | neoplastic cell                                         |      2.08464 |         2218 |       0.0914526   |          0.00272965  |              21536 | 0.35927     | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| GAMT     | neoplastic cell                                         |      1.83142 |          512 |       0.0211108   |          0.000630108 |              21536 | 0.150956    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| HSPA1A   | neoplastic cell                                         |      2.60145 |         6151 |       0.253618    |          0.00756991  |              21536 | 0.823042    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| IFI27    | neoplastic cell                                         |      1.68681 |          698 |       0.0287799   |          0.000859015 |              21536 | 0.131303    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| KRT14    | neoplastic cell                                         |      3.75004 |         3256 |       0.134251    |          0.00400709  |              21536 | 0.857795    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| LGALS3BP | neoplastic cell                                         |      1.99507 |         5211 |       0.21486     |          0.00641307  |              21536 | 0.593418    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| RBM42    | neoplastic cell                                         |      2.14087 |         2220 |       0.0915351   |          0.00273211  |              21536 | 0.373272    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| SCPEP1   | neoplastic cell                                         |      1.70619 |          988 |       0.0407372   |          0.00121591  |              21536 | 0.160891    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| COL17A1  | malignant cell                                          |      1.99107 |          282 |       0.0116274   |          0.000347052 |              21536 | 0.170543    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| EFNA1    | malignant cell                                          |      2.08464 |         2218 |       0.0914526   |          0.00272965  |              21536 | 0.35927     | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| GAMT     | malignant cell                                          |      1.83142 |          512 |       0.0211108   |          0.000630108 |              21536 | 0.150956    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| HSPA1A   | malignant cell                                          |      2.60145 |         6151 |       0.253618    |          0.00756991  |              21536 | 0.823042    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| IFI27    | malignant cell                                          |      1.68681 |          698 |       0.0287799   |          0.000859015 |              21536 | 0.131303    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| KRT14    | malignant cell                                          |      3.75004 |         3256 |       0.134251    |          0.00400709  |              21536 | 0.857795    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| LGALS3BP | malignant cell                                          |      1.99507 |         5211 |       0.21486     |          0.00641307  |              21536 | 0.593418    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| RBM42    | malignant cell                                          |      2.14087 |         2220 |       0.0915351   |          0.00273211  |              21536 | 0.373272    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| SCPEP1   | malignant cell                                          |      1.70619 |          988 |       0.0407372   |          0.00121591  |              21536 | 0.160891    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| COL17A1  | abnormal cell                                           |      1.99107 |          282 |       0.0116274   |          0.000347052 |              21536 | 0.170543    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| EFNA1    | abnormal cell                                           |      2.08464 |         2218 |       0.0914526   |          0.00272965  |              21536 | 0.35927     | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| GAMT     | abnormal cell                                           |      1.83142 |          512 |       0.0211108   |          0.000630108 |              21536 | 0.150956    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| HSPA1A   | abnormal cell                                           |      2.60145 |         6151 |       0.253618    |          0.00756991  |              21536 | 0.823042    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| IFI27    | abnormal cell                                           |      1.68681 |          698 |       0.0287799   |          0.000859015 |              21536 | 0.131303    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| KRT14    | abnormal cell                                           |      3.75004 |         3256 |       0.134251    |          0.00400709  |              21536 | 0.857795    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| LGALS3BP | abnormal cell                                           |      1.99507 |         5211 |       0.21486     |          0.00641307  |              21536 | 0.593418    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| RBM42    | abnormal cell                                           |      2.14087 |         2220 |       0.0915351   |          0.00273211  |              21536 | 0.373272    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| SCPEP1   | abnormal cell                                           |      1.70619 |          988 |       0.0407372   |          0.00121591  |              21536 | 0.160891    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| COL17A1  | hair follicular keratinocyte                            |      1.76391 |          789 |       0.232332    |          0.000971006 |               8122 | 0.457757    | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| EFNA1    | hair follicular keratinocyte                            |      1.67417 |          637 |       0.187574    |          0.000783943 |               8122 | 0.364936    | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| GAMT     | hair follicular keratinocyte                            |      1.55148 |           38 |       0.0111896   |          4.67658e-05 |               8122 | 0.055955    | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| HSPA1A   | hair follicular keratinocyte                            |      3.00312 |         2906 |       0.855713    |          0.00357636  |               8122 | 1.74794     | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| IFI27    | hair follicular keratinocyte                            |      1.94323 |          111 |       0.0326855   |          0.000136605 |               8122 | 0.1863      | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| KRT14    | hair follicular keratinocyte                            |      4.37114 |         3372 |       0.992933    |          0.00414985  |               8122 | 2.30136     | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| LGALS3BP | hair follicular keratinocyte                            |      1.57412 |           51 |       0.0150177   |          6.27647e-05 |               8122 | 0.0675741   | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| RBM42    | hair follicular keratinocyte                            |      1.49171 |           22 |       0.00647821  |          2.7075e-05  |               8122 | 0.033804    | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| SCPEP1   | hair follicular keratinocyte                            |      1.64349 |          196 |       0.057715    |          0.000241213 |               8122 | 0.152129    | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| COL17A1  | melanocyte                                              |      1.80981 |          335 |       0.0277134   |          0.000412278 |              16294 | 0.15134     | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| EFNA1    | melanocyte                                              |      1.86621 |         2059 |       0.170334    |          0.00253397  |              16294 | 0.420083    | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| GAMT     | melanocyte                                              |      1.73909 |          119 |       0.00984447  |          0.000146451 |              16294 | 0.102011    | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| HSPA1A   | melanocyte                                              |      2.75078 |         5364 |       0.443746    |          0.00660137  |              16294 | 1.12628     | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| IFI27    | melanocyte                                              |      2.50522 |         1372 |       0.113501    |          0.00168849  |              16294 | 0.475714    | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| KRT14    | melanocyte                                              |      2.37538 |         3799 |       0.314279    |          0.00467535  |              16294 | 0.802571    | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| LGALS3BP | melanocyte                                              |      1.80847 |         2240 |       0.185308    |          0.00275672  |              16294 | 0.432638    | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| RBM42    | melanocyte                                              |      1.74111 |          862 |       0.0713104   |          0.00106085  |              16294 | 0.21235     | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| SCPEP1   | melanocyte                                              |      1.81156 |          144 |       0.0119126   |          0.000177218 |              16294 | 0.123535    | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| COL17A1  | sebaceous gland cell                                    |      2.37004 |           11 |       0.0366667   |          1.35375e-05 |                213 | 0.294891    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| EFNA1    | sebaceous gland cell                                    |      2.17396 |           62 |       0.206667    |          7.63022e-05 |                213 | 0.502867    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| GAMT     | sebaceous gland cell                                    |      1.8935  |            1 |       0.00740741  |          1.23068e-06 |                213 | 0.133538    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| HSPA1A   | sebaceous gland cell                                    |      2.6795  |           43 |       0.318519    |          5.29192e-05 |                213 | 0.794554    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| IFI27    | sebaceous gland cell                                    |      2.64172 |            3 |       0.0222222   |          3.69204e-06 |                213 | 0.339865    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| KRT14    | sebaceous gland cell                                    |      2.50666 |           65 |       0.216667    |          7.99942e-05 |                213 | 0.599781    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| LGALS3BP | sebaceous gland cell                                    |      1.91798 |            7 |       0.0233333   |          8.61476e-06 |                213 | 0.16359     | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| RBM42    | sebaceous gland cell                                    |      1.96204 |            3 |       0.01        |          3.69204e-06 |                213 | 0.154336    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| SCPEP1   | sebaceous gland cell                                    |      2.14275 |           18 |       0.06        |          2.21522e-05 |                213 | 0.274146    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| COL17A1  | neuron associated cell                                  |      1.875   |          308 |       0.162877    |          0.000379049 |               7763 | 0.369482    | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| EFNA1    | neuron associated cell                                  |      1.83332 |          538 |       0.0857917   |          0.000662106 |               7763 | 0.249062    | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| GAMT     | neuron associated cell                                  |      1.59755 |          230 |       0.0366768   |          0.000283056 |               7763 | 0.110077    | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| HSPA1A   | neuron associated cell                                  |      3.06618 |         3894 |       0.620954    |          0.00479227  |               7763 | 1.43477     | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| IFI27    | neuron associated cell                                  |      2.30548 |          116 |       0.0184978   |          0.000142759 |               7763 | 0.254249    | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| KRT14    | neuron associated cell                                  |      4.53381 |         1178 |       0.187849    |          0.00144974  |               7763 | 1.08165     | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| LGALS3BP | neuron associated cell                                  |      1.67851 |          227 |       0.0361984   |          0.000279364 |               7763 | 0.129205    | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| RBM42    | neuron associated cell                                  |      1.6439  |          606 |       0.0966353   |          0.000745792 |               7763 | 0.220346    | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| SCPEP1   | neuron associated cell                                  |      1.84467 |          666 |       0.106203    |          0.000819633 |               7763 | 0.28551     | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| COL17A1  | neural cell                                             |      1.875   |          308 |       0.162877    |          0.000379049 |               8590 | 0.369482    | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| EFNA1    | neural cell                                             |      1.83405 |          542 |       0.0782785   |          0.000667028 |               8590 | 0.238066    | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| GAMT     | neural cell                                             |      1.59977 |          313 |       0.0383203   |          0.000385203 |               8590 | 0.115059    | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| HSPA1A   | neural cell                                             |      3.02641 |         4395 |       0.538075    |          0.00540884  |               8590 | 1.31257     | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| IFI27    | neural cell                                             |      2.30548 |          116 |       0.0184978   |          0.000142759 |               8590 | 0.254249    | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| KRT14    | neural cell                                             |      4.53381 |         1178 |       0.187849    |          0.00144974  |               8590 | 1.08165     | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| LGALS3BP | neural cell                                             |      1.67689 |          240 |       0.034662    |          0.000295363 |               8590 | 0.12681     | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| RBM42    | neural cell                                             |      1.64096 |          729 |       0.0892507   |          0.000897166 |               8590 | 0.211467    | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| SCPEP1   | neural cell                                             |      1.81465 |          769 |       0.0941479   |          0.000946393 |               8590 | 0.262489    | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| COL17A1  | Schwann cell                                            |      1.94119 |           41 |       0.0545939   |          5.04579e-05 |                623 | 0.216999    | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| EFNA1    | Schwann cell                                            |      1.91843 |          165 |       0.219707    |          0.000203062 |                623 | 0.462013    | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| GAMT     | Schwann cell                                            |      1.74218 |            2 |       0.00266312  |          2.46136e-06 |                623 | 0.0892212   | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| HSPA1A   | Schwann cell                                            |      2.2605  |          103 |       0.13715     |          0.00012676  |                623 | 0.420853    | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| IFI27    | Schwann cell                                            |      2.38399 |          102 |       0.135819    |          0.000125529 |                623 | 0.44921     | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| KRT14    | Schwann cell                                            |      2.09418 |            6 |       0.00798935  |          7.38408e-06 |                623 | 0.183897    | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| LGALS3BP | Schwann cell                                            |      1.78797 |           53 |       0.0705726   |          6.5226e-05  |                623 | 0.20356     | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| RBM42    | Schwann cell                                            |      1.71655 |           25 |       0.0332889   |          3.0767e-05  |                623 | 0.129401    | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| SCPEP1   | Schwann cell                                            |      2.02504 |          126 |       0.167776    |          0.000155066 |                623 | 0.409417    | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| COL17A1  | endothelial cell                                        |      1.82391 |          250 |       0.00571873  |          0.00030767  |              53757 | 0.1198      | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| EFNA1    | endothelial cell                                        |      1.84292 |        12508 |       0.286048    |          0.0153933   |              53757 | 0.836044    | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| GAMT     | endothelial cell                                        |      1.69743 |           38 |       0.000884132 |          4.67658e-05 |              53757 | 0.0763995   | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| HSPA1A   | endothelial cell                                        |      2.40738 |         9797 |       0.224049    |          0.012057    |              53757 | 0.817526    | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| IFI27    | endothelial cell                                        |      2.81868 |        26662 |       0.609738    |          0.0328124   |              53757 | 1.89771     | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| KRT14    | endothelial cell                                        |      2.68658 |         1369 |       0.031852    |          0.0016848   |              53757 | 0.397781    | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| LGALS3BP | endothelial cell                                        |      1.7489  |          125 |       0.00285936  |          0.000153835 |              53757 | 0.094089    | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| RBM42    | endothelial cell                                        |      1.62469 |         2757 |       0.0630503   |          0.00339298  |              53757 | 0.21632     | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| SCPEP1   | endothelial cell                                        |      1.68267 |          251 |       0.0057416   |          0.000308901 |              53757 | 0.0851121   | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| COL17A1  | sebum secreting cell                                    |      1.82275 |            1 |       0.00740741  |          1.23068e-06 |                117 | 0.116135    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| EFNA1    | sebum secreting cell                                    |      2.19523 |           55 |       0.407407    |          6.76874e-05 |                117 | 0.809045    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| GAMT     | sebum secreting cell                                    |      1.8935  |            1 |       0.00740741  |          1.23068e-06 |                117 | 0.133538    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| HSPA1A   | sebum secreting cell                                    |      2.6795  |           43 |       0.318519    |          5.29192e-05 |                117 | 0.794554    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| IFI27    | sebum secreting cell                                    |      2.64172 |            3 |       0.0222222   |          3.69204e-06 |                117 | 0.339865    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| KRT14    | sebum secreting cell                                    |      3.17732 |            2 |       0.0148148   |          2.46136e-06 |                117 | 0.460484    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| LGALS3BP | sebum secreting cell                                    |      1.85507 |            6 |       0.0444444   |          7.38408e-06 |                117 | 0.179759    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| RBM42    | sebum secreting cell                                    |      1.67727 |            1 |       0.00740741  |          1.23068e-06 |                117 | 0.0803468   | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| SCPEP1   | sebum secreting cell                                    |      1.92974 |            5 |       0.037037    |          6.1534e-06  |                117 | 0.186991    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| COL17A1  | exocrine cell                                           |      1.82275 |            1 |       0.00740741  |          1.23068e-06 |                117 | 0.116135    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| EFNA1    | exocrine cell                                           |      2.19523 |           55 |       0.407407    |          6.76874e-05 |                117 | 0.809045    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| GAMT     | exocrine cell                                           |      1.8935  |            1 |       0.00740741  |          1.23068e-06 |                117 | 0.133538    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| HSPA1A   | exocrine cell                                           |      2.6795  |           43 |       0.318519    |          5.29192e-05 |                117 | 0.794554    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| IFI27    | exocrine cell                                           |      2.64172 |            3 |       0.0222222   |          3.69204e-06 |                117 | 0.339865    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| KRT14    | exocrine cell                                           |      3.17732 |            2 |       0.0148148   |          2.46136e-06 |                117 | 0.460484    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| LGALS3BP | exocrine cell                                           |      1.85507 |            6 |       0.0444444   |          7.38408e-06 |                117 | 0.179759    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| RBM42    | exocrine cell                                           |      1.67727 |            1 |       0.00740741  |          1.23068e-06 |                117 | 0.0803468   | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| SCPEP1   | exocrine cell                                           |      1.92974 |            5 |       0.037037    |          6.1534e-06  |                117 | 0.186991    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| COL17A1  | endothelial cell of vascular tree                       |      1.83058 |          235 |       0.00610659  |          0.00028921  |              47624 | 0.121666    | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| EFNA1    | endothelial cell of vascular tree                       |      1.83957 |        11238 |       0.291942    |          0.0138304   |              47624 | 0.813904    | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| GAMT     | endothelial cell of vascular tree                       |      1.66883 |           26 |       0.000688796 |          3.19977e-05 |              47624 | 0.0687861   | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| HSPA1A   | endothelial cell of vascular tree                       |      2.21813 |         7679 |       0.199486    |          0.00945039  |              47624 | 0.683833    | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| IFI27    | endothelial cell of vascular tree                       |      2.84035 |        24309 |       0.631501    |          0.0299166   |              47624 | 1.87981     | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| KRT14    | endothelial cell of vascular tree                       |      2.69891 |         1316 |       0.0348637   |          0.00161957  |              47624 | 0.404074    | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| LGALS3BP | endothelial cell of vascular tree                       |      1.72881 |          107 |       0.00278045  |          0.000131683 |              47624 | 0.0886016   | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| RBM42    | endothelial cell of vascular tree                       |      1.62413 |         2503 |       0.0650231   |          0.00308039  |              47624 | 0.213109    | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| SCPEP1   | endothelial cell of vascular tree                       |      1.68699 |          211 |       0.00548294  |          0.000259673 |              47624 | 0.0848379   | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| COL17A1  | stromal cell                                            |      1.72962 |           46 |       0.0148819   |          5.66113e-05 |               1014 | 0.105505    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| EFNA1    | stromal cell                                            |      1.89409 |          322 |       0.104173    |          0.000396279 |               1014 | 0.286455    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| GAMT     | stromal cell                                            |      2.47744 |           40 |       0.0129408   |          4.92272e-05 |               1014 | 0.286409    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| HSPA1A   | stromal cell                                            |      2.88942 |          417 |       0.134908    |          0.000513194 |               1014 | 0.579656    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| IFI27    | stromal cell                                            |      2.2656  |            9 |       0.00291168  |          1.10761e-05 |               1014 | 0.218519    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| KRT14    | stromal cell                                            |      1.76955 |           10 |       0.0032352   |          1.23068e-05 |               1014 | 0.097001    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| LGALS3BP | stromal cell                                            |      2.12996 |           21 |       0.00679392  |          2.58443e-05 |               1014 | 0.19126     | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| RBM42    | stromal cell                                            |      2.54413 |           76 |       0.0245875   |          9.35317e-05 |               1014 | 0.321141    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| SCPEP1   | stromal cell                                            |      2.57798 |           73 |       0.023617    |          8.98396e-05 |               1014 | 0.327939    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| COL17A1  | endothelial cell of lymphatic vessel                    |      1.69263 |           23 |       0.00365776  |          2.83056e-05 |               5230 | 0.0790219   | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| EFNA1    | endothelial cell of lymphatic vessel                    |      1.92953 |         2155 |       0.342716    |          0.00265212  |               5230 | 0.696513    | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| GAMT     | endothelial cell of lymphatic vessel                    |      1.69216 |           14 |       0.00222646  |          1.72295e-05 |               5230 | 0.0765454   | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| HSPA1A   | endothelial cell of lymphatic vessel                    |      2.61616 |         2071 |       0.329358    |          0.00254874  |               5230 | 0.843387    | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| IFI27    | endothelial cell of lymphatic vessel                    |      1.83856 |          215 |       0.0341921   |          0.000264596 |               5230 | 0.165281    | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| KRT14    | endothelial cell of lymphatic vessel                    |      2.61177 |          255 |       0.0405534   |          0.000313823 |               5230 | 0.365978    | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| LGALS3BP | endothelial cell of lymphatic vessel                    |      1.78494 |           27 |       0.00429389  |          3.32284e-05 |               5230 | 0.102779    | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| RBM42    | endothelial cell of lymphatic vessel                    |      1.66889 |          386 |       0.0613868   |          0.000475042 |               5230 | 0.168397    | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| SCPEP1   | endothelial cell of lymphatic vessel                    |      1.67807 |           84 |       0.0133588   |          0.000103377 |               5230 | 0.0914403   | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| COL17A1  | melanocyte of skin                                      |      1.75148 |          183 |       0.071821    |          0.000225214 |                977 | 0.199545    | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| EFNA1    | melanocyte of skin                                      |      1.78617 |          628 |       0.246468    |          0.000772867 |                977 | 0.480615    | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| GAMT     | melanocyte of skin                                      |      2.60789 |            2 |       0.000784929 |          2.46136e-06 |                977 | 0.299364    | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| HSPA1A   | melanocyte of skin                                      |      2.05244 |           40 |       0.0156986   |          4.92272e-05 |                977 | 0.185999    | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| IFI27    | melanocyte of skin                                      |      1.77584 |           15 |       0.00588697  |          1.84602e-05 |                977 | 0.102645    | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| KRT14    | melanocyte of skin                                      |      2.51294 |           44 |       0.0172684   |          5.41499e-05 |                977 | 0.301729    | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| LGALS3BP | melanocyte of skin                                      |      1.8637  |           51 |       0.0200157   |          6.27647e-05 |                977 | 0.146308    | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| RBM42    | melanocyte of skin                                      |      1.75981 |           10 |       0.00392465  |          1.23068e-05 |                977 | 0.0956396   | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| SCPEP1   | melanocyte of skin                                      |      1.78054 |            4 |       0.00156986  |          4.92272e-06 |                977 | 0.0970663   | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| COL17A1  | supporting cell                                         |      1.75195 |          386 |       0.0215642   |          0.000475042 |               8252 | 0.129094    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| EFNA1    | supporting cell                                         |      1.73515 |         1701 |       0.0950279   |          0.00209339  |               8252 | 0.266382    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| GAMT     | supporting cell                                         |      1.68498 |          131 |       0.00731844  |          0.000161219 |               8252 | 0.0851974   | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| HSPA1A   | supporting cell                                         |      2.289   |         2573 |       0.143743    |          0.00316654  |               8252 | 0.496405    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| IFI27    | supporting cell                                         |      2.02098 |          786 |       0.0439106   |          0.000967314 |               8252 | 0.238293    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| KRT14    | supporting cell                                         |      2.32868 |          889 |       0.0496648   |          0.00109407  |               8252 | 0.325062    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| LGALS3BP | supporting cell                                         |      1.75297 |          688 |       0.0384358   |          0.000846708 |               8252 | 0.161825    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| RBM42    | supporting cell                                         |      1.66225 |          849 |       0.0474302   |          0.00104485  |               8252 | 0.156823    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| SCPEP1   | supporting cell                                         |      1.66625 |          249 |       0.0139106   |          0.000306439 |               8252 | 0.093279    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| COL17A1  | pericyte                                                |      1.75195 |          386 |       0.0215642   |          0.000475042 |               8252 | 0.129094    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| EFNA1    | pericyte                                                |      1.73515 |         1701 |       0.0950279   |          0.00209339  |               8252 | 0.266382    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| GAMT     | pericyte                                                |      1.68498 |          131 |       0.00731844  |          0.000161219 |               8252 | 0.0851974   | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| HSPA1A   | pericyte                                                |      2.289   |         2573 |       0.143743    |          0.00316654  |               8252 | 0.496405    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| IFI27    | pericyte                                                |      2.02098 |          786 |       0.0439106   |          0.000967314 |               8252 | 0.238293    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| KRT14    | pericyte                                                |      2.32868 |          889 |       0.0496648   |          0.00109407  |               8252 | 0.325062    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| LGALS3BP | pericyte                                                |      1.75297 |          688 |       0.0384358   |          0.000846708 |               8252 | 0.161825    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| RBM42    | pericyte                                                |      1.66225 |          849 |       0.0474302   |          0.00104485  |               8252 | 0.156823    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| SCPEP1   | pericyte                                                |      1.66625 |          249 |       0.0139106   |          0.000306439 |               8252 | 0.093279    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| COL17A1  | mural cell                                              |      1.75195 |          386 |       0.0215642   |          0.000475042 |               8252 | 0.129094    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| EFNA1    | mural cell                                              |      1.73515 |         1701 |       0.0950279   |          0.00209339  |               8252 | 0.266382    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| GAMT     | mural cell                                              |      1.68498 |          131 |       0.00731844  |          0.000161219 |               8252 | 0.0851974   | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| HSPA1A   | mural cell                                              |      2.289   |         2573 |       0.143743    |          0.00316654  |               8252 | 0.496405    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| IFI27    | mural cell                                              |      2.02098 |          786 |       0.0439106   |          0.000967314 |               8252 | 0.238293    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| KRT14    | mural cell                                              |      2.32868 |          889 |       0.0496648   |          0.00109407  |               8252 | 0.325062    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| LGALS3BP | mural cell                                              |      1.75297 |          688 |       0.0384358   |          0.000846708 |               8252 | 0.161825    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| RBM42    | mural cell                                              |      1.66225 |          849 |       0.0474302   |          0.00104485  |               8252 | 0.156823    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| SCPEP1   | mural cell                                              |      1.66625 |          249 |       0.0139106   |          0.000306439 |               8252 | 0.093279    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| COL17A1  | hematopoietic cell                                      |      1.95432 |         3367 |       0.0142786   |          0.0041437   |             135695 | 0.238734    | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| EFNA1    | hematopoietic cell                                      |      1.73061 |         8466 |       0.0296338   |          0.0104189   |             135695 | 0.327816    | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| GAMT     | hematopoietic cell                                      |      1.64297 |         3563 |       0.0121022   |          0.00438491  |             135695 | 0.163534    | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| HSPA1A   | hematopoietic cell                                      |      3.26665 |        92787 |       0.302484    |          0.114191    |             135695 | 3.11721     | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| IFI27    | hematopoietic cell                                      |      2.05496 |         1758 |       0.00701402  |          0.00216354  |             135695 | 0.214388    | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| KRT14    | hematopoietic cell                                      |      2.21316 |         4626 |       0.0162531   |          0.00569313  |             135695 | 0.335266    | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| LGALS3BP | hematopoietic cell                                      |      1.68439 |         1722 |       0.00575106  |          0.00211923  |             135695 | 0.12048     | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| RBM42    | hematopoietic cell                                      |      1.67137 |        14976 |       0.0488668   |          0.0184307   |             135695 | 0.496677    | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| SCPEP1   | hematopoietic cell                                      |      1.6628  |         4430 |       0.0144593   |          0.00545191  |             135695 | 0.192534    | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| COL17A1  | leukocyte                                               |      1.95615 |         3337 |       0.0142609   |          0.00410678  |             133052 | 0.238446    | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| EFNA1    | leukocyte                                               |      1.73047 |         8461 |       0.0298044   |          0.0104128   |             133052 | 0.327918    | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| GAMT     | leukocyte                                               |      1.64267 |         3462 |       0.0119037   |          0.00426061  |             133052 | 0.160763    | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| HSPA1A   | leukocyte                                               |      3.26645 |        91218 |       0.300922    |          0.11226     |             133052 | 3.07756     | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| IFI27    | leukocyte                                               |      2.05907 |         1736 |       0.00697878  |          0.00213646  |             133052 | 0.214822    | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| KRT14    | leukocyte                                               |      2.17483 |         3953 |       0.0139753   |          0.00486488  |             133052 | 0.306438    | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| LGALS3BP | leukocyte                                               |      1.68396 |         1713 |       0.00575978  |          0.00210815  |             133052 | 0.120175    | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| RBM42    | leukocyte                                               |      1.67132 |        14848 |       0.048997    |          0.0182731   |             133052 | 0.49382     | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| SCPEP1   | leukocyte                                               |      1.66265 |         4324 |       0.0142678   |          0.00532146  |             133052 | 0.189694    | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| COL17A1  | mononuclear cell                                        |      1.95482 |         3301 |       0.0144308   |          0.00406247  |             131093 | 0.237518    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| EFNA1    | mononuclear cell                                        |      1.7273  |         8293 |       0.0297544   |          0.010206    |             131093 | 0.323073    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| GAMT     | mononuclear cell                                        |      1.6418  |         3395 |       0.0118935   |          0.00417816  |             131093 | 0.158942    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| HSPA1A   | mononuclear cell                                        |      3.27094 |        90001 |       0.302367    |          0.110762    |             131093 | 3.05194     | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| IFI27    | mononuclear cell                                        |      2.06079 |         1714 |       0.00703656  |          0.00210939  |             131093 | 0.214811    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| KRT14    | mononuclear cell                                        |      2.17744 |         3895 |       0.0140108   |          0.0047935   |             131093 | 0.305756    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| LGALS3BP | mononuclear cell                                        |      1.68219 |         1695 |       0.0057937   |          0.002086    |             131093 | 0.119361    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| RBM42    | mononuclear cell                                        |      1.67086 |        14549 |       0.0488934   |          0.0179052   |             131093 | 0.486452    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| SCPEP1   | mononuclear cell                                        |      1.65891 |         4250 |       0.0142812   |          0.00523039  |             131093 | 0.187036    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| COL17A1  | monocyte-derived dendritic cell                         |      1.89074 |            1 |       8.1314e-05  |          1.23068e-06 |               1769 | 0.121871    | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| EFNA1    | monocyte-derived dendritic cell                         |      1.59082 |            1 |       8.1314e-05  |          1.23068e-06 |               1769 | 0.0480913   | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| GAMT     | monocyte-derived dendritic cell                         |      1.4534  |            1 |       8.1314e-05  |          1.23068e-06 |               1769 | 0.0142853   | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| HSPA1A   | monocyte-derived dendritic cell                         |      1.80784 |         1560 |       0.12685     |          0.00191986  |               1769 | 0.328649    | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| IFI27    | monocyte-derived dendritic cell                         |      2.39792 |           54 |       0.00439096  |          6.64567e-05 |               1769 | 0.254356    | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| KRT14    | monocyte-derived dendritic cell                         |      2.34077 |           26 |       0.00211416  |          3.19977e-05 |               1769 | 0.236218    | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| LGALS3BP | monocyte-derived dendritic cell                         |      1.7421  |            2 |       0.000162628 |          2.46136e-06 |               1769 | 0.0854512   | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| RBM42    | monocyte-derived dendritic cell                         |      1.60052 |          112 |       0.00910717  |          0.000137836 |               1769 | 0.0666515   | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| SCPEP1   | monocyte-derived dendritic cell                         |      1.61067 |           12 |       0.000975768 |          1.47682e-05 |               1769 | 0.0545757   | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| COL17A1  | myeloid dendritic cell                                  |      1.89074 |            1 |       8.1314e-05  |          1.23068e-06 |               1778 | 0.121871    | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| EFNA1    | myeloid dendritic cell                                  |      1.59082 |            1 |       8.1314e-05  |          1.23068e-06 |               1778 | 0.0480913   | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| GAMT     | myeloid dendritic cell                                  |      1.4534  |            1 |       8.1314e-05  |          1.23068e-06 |               1778 | 0.0142853   | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| HSPA1A   | myeloid dendritic cell                                  |      1.81085 |         1568 |       0.12639     |          0.00192971  |               1778 | 0.328891    | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| IFI27    | myeloid dendritic cell                                  |      2.39792 |           54 |       0.00439096  |          6.64567e-05 |               1778 | 0.254356    | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| KRT14    | myeloid dendritic cell                                  |      2.34077 |           26 |       0.00211416  |          3.19977e-05 |               1778 | 0.236218    | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| LGALS3BP | myeloid dendritic cell                                  |      1.7421  |            2 |       0.000162628 |          2.46136e-06 |               1778 | 0.0854512   | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| RBM42    | myeloid dendritic cell                                  |      1.60052 |          112 |       0.00910717  |          0.000137836 |               1778 | 0.0666515   | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| SCPEP1   | myeloid dendritic cell                                  |      1.59358 |           13 |       0.00104788  |          1.59988e-05 |               1778 | 0.0505052   | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| COL17A1  | erythroid lineage cell                                  |      1.56496 |            4 |       0.00396825  |          4.92272e-06 |                435 | 0.0476319   | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| EFNA1    | erythroid lineage cell                                  |      2.23455 |            3 |       0.00287356  |          3.69204e-06 |                435 | 0.210681    | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| GAMT     | erythroid lineage cell                                  |      2.04087 |           10 |       0.00933707  |          1.23068e-05 |                435 | 0.172899    | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| HSPA1A   | erythroid lineage cell                                  |      2.8578  |          320 |       0.298786    |          0.000393818 |                435 | 0.815394    | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| IFI27    | erythroid lineage cell                                  |      1.70169 |           12 |       0.0119048   |          1.47682e-05 |                435 | 0.0933603   | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| KRT14    | erythroid lineage cell                                  |      3.04299 |           34 |       0.0337302   |          4.18431e-05 |                435 | 0.456574    | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| LGALS3BP | erythroid lineage cell                                  |      2.00253 |            2 |       0.00193237  |          2.46136e-06 |                435 | 0.15217     | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| RBM42    | erythroid lineage cell                                  |      1.81088 |           38 |       0.0354809   |          4.67658e-05 |                435 | 0.156201    | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| SCPEP1   | erythroid lineage cell                                  |      1.77934 |           12 |       0.0112045   |          1.47682e-05 |                435 | 0.111413    | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| COL17A1  | perivascular cell                                       |      1.75119 |          387 |       0.0141686   |          0.000476273 |              15137 | 0.117838    | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| EFNA1    | perivascular cell                                       |      1.72517 |         1846 |       0.0675794   |          0.00227184  |              15137 | 0.226199    | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| GAMT     | perivascular cell                                       |      1.61781 |          665 |       0.0243465   |          0.000818402 |              15137 | 0.106894    | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| HSPA1A   | perivascular cell                                       |      2.35917 |         6126 |       0.224281    |          0.00753914  |              15137 | 0.718842    | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| IFI27    | perivascular cell                                       |      2.01735 |          792 |       0.0289961   |          0.000974698 |              15137 | 0.21517     | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| KRT14    | perivascular cell                                       |      2.32948 |          890 |       0.032584    |          0.00109531  |              15137 | 0.29966     | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| LGALS3BP | perivascular cell                                       |      1.69274 |         1848 |       0.0676576   |          0.0022743   |              15137 | 0.218386    | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| RBM42    | perivascular cell                                       |      1.64076 |         1926 |       0.0705133   |          0.00237029  |              15137 | 0.211734    | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| SCPEP1   | perivascular cell                                       |      1.62932 |          657 |       0.0240518   |          0.000808557 |              15137 | 0.109094    | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| COL17A1  | vascular associated smooth muscle cell                  |      1.45844 |            1 |       0.000106225 |          1.23068e-06 |               6885 | 0.0155633   | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| EFNA1    | vascular associated smooth muscle cell                  |      1.60818 |          145 |       0.0153993   |          0.000178449 |               6885 | 0.078757    | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| GAMT     | vascular associated smooth muscle cell                  |      1.60133 |          534 |       0.056724    |          0.000657183 |               6885 | 0.148296    | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| HSPA1A   | vascular associated smooth muscle cell                  |      2.40999 |         3553 |       0.377417    |          0.00437261  |               6885 | 0.899949    | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| IFI27    | vascular associated smooth muscle cell                  |      1.54129 |            6 |       0.000637349 |          7.38408e-06 |               6885 | 0.0368588   | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| KRT14    | vascular associated smooth muscle cell                  |      3.03553 |            1 |       0.000106225 |          1.23068e-06 |               6885 | 0.40352     | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| LGALS3BP | vascular associated smooth muscle cell                  |      1.65702 |         1160 |       0.123221    |          0.00142759  |               6885 | 0.276606    | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| RBM42    | vascular associated smooth muscle cell                  |      1.62381 |         1077 |       0.114404    |          0.00132544  |               6885 | 0.253242    | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| SCPEP1   | vascular associated smooth muscle cell                  |      1.60678 |          408 |       0.0433305   |          0.000502117 |               6885 | 0.126554    | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| COL17A1  | contractile cell                                        |      1.75993 |          397 |       0.0141286   |          0.00048858  |              19951 | 0.120166    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| EFNA1    | contractile cell                                        |      1.72989 |         1927 |       0.0574881   |          0.00237152  |              19951 | 0.214145    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| GAMT     | contractile cell                                        |      1.61579 |          813 |       0.0242556   |          0.00100054  |              19951 | 0.109777    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| HSPA1A   | contractile cell                                        |      2.51    |         9671 |       0.287528    |          0.0119019   |              19951 | 0.934997    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| IFI27    | contractile cell                                        |      2.01714 |          830 |       0.0246767   |          0.00102146  |              19951 | 0.209542    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| KRT14    | contractile cell                                        |      2.32184 |          905 |       0.0269065   |          0.00111377  |              19951 | 0.289623    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| LGALS3BP | contractile cell                                        |      1.69473 |         2345 |       0.069719    |          0.00288594  |              19951 | 0.23377     | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| RBM42    | contractile cell                                        |      1.63529 |         2147 |       0.0640551   |          0.00264227  |              19951 | 0.205949    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| SCPEP1   | contractile cell                                        |      1.62969 |          916 |       0.0272319   |          0.0011273   |              19951 | 0.120105    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| COL17A1  | lymphocyte                                              |      1.96358 |         3146 |       0.0246708   |          0.00387172  |              83193 | 0.251352    | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| EFNA1    | lymphocyte                                              |      1.73057 |         7024 |       0.0395747   |          0.0086443   |              83193 | 0.308475    | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| GAMT     | lymphocyte                                              |      1.5942  |          992 |       0.00541254  |          0.00122083  |              83193 | 0.0804518   | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| HSPA1A   | lymphocyte                                              |      3.22827 |        54165 |       0.277291    |          0.0666598   |              83193 | 2.15287     | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| IFI27    | lymphocyte                                              |      1.69849 |          638 |       0.00448084  |          0.000785174 |              83193 | 0.096302    | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| KRT14    | lymphocyte                                              |      1.90213 |         2073 |       0.0115932   |          0.0025512   |              83193 | 0.191141    | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| LGALS3BP | lymphocyte                                              |      1.66466 |         1190 |       0.00621835  |          0.00146451  |              83193 | 0.103695    | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| RBM42    | lymphocyte                                              |      1.66632 |        11566 |       0.0592017   |          0.014234    |              83193 | 0.429964    | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| SCPEP1   | lymphocyte                                              |      1.63094 |         2399 |       0.0122845   |          0.0029524   |              83193 | 0.133207    | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| COL17A1  | blood cell                                              |      2.06895 |           11 |       0.0082769   |          1.35375e-05 |                564 | 0.178239    | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| EFNA1    | blood cell                                              |      1.63199 |            2 |       0.00150489  |          2.46136e-06 |                564 | 0.0603788   | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| GAMT     | blood cell                                              |      1.67718 |           17 |       0.0110104   |          2.09216e-05 |                564 | 0.086108    | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| HSPA1A   | blood cell                                              |      2.68595 |          426 |       0.274485    |          0.00052427  |                564 | 0.739183    | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| IFI27    | blood cell                                              |      1.68766 |           13 |       0.00978179  |          1.59988e-05 |                564 | 0.0867477   | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| KRT14    | blood cell                                              |      3.00716 |           35 |       0.0344488   |          4.30738e-05 |                564 | 0.448862    | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| LGALS3BP | blood cell                                              |      1.39773 |            1 |       0.000992063 |          1.23068e-06 |                564 | 0.00195747  | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| RBM42    | blood cell                                              |      1.65671 |           45 |       0.0289948   |          5.53806e-05 |                564 | 0.108715    | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| SCPEP1   | blood cell                                              |      1.73129 |           14 |       0.00906736  |          1.72295e-05 |                564 | 0.0964341   | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| COL17A1  | T cell                                                  |      1.99123 |         2814 |       0.0294019   |          0.00346313  |              66842 | 0.257367    | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| EFNA1    | T cell                                                  |      1.71713 |         4708 |       0.0341325   |          0.00579404  |              66842 | 0.24201     | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| GAMT     | T cell                                                  |      1.59456 |          687 |       0.00484803  |          0.000845477 |              66842 | 0.0724519   | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| HSPA1A   | T cell                                                  |      3.19788 |        43444 |       0.282945    |          0.0534657   |              66842 | 1.8993      | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| IFI27    | T cell                                                  |      1.71346 |          522 |       0.00479489  |          0.000642415 |              66842 | 0.0977011   | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| KRT14    | T cell                                                  |      1.91405 |         1955 |       0.0132992   |          0.00240598  |              66842 | 0.193831    | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| LGALS3BP | T cell                                                  |      1.65952 |         1015 |       0.00667957  |          0.00124914  |              66842 | 0.0989657   | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| RBM42    | T cell                                                  |      1.67018 |         9967 |       0.0649012   |          0.0122662   |              66842 | 0.401493    | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| SCPEP1   | T cell                                                  |      1.62645 |         1730 |       0.0112673   |          0.00212908  |              66842 | 0.114692    | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| COL17A1  | myeloid cell                                            |      1.81394 |          161 |       0.00179068  |          0.000198139 |              45633 | 0.109342    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| EFNA1    | myeloid cell                                            |      1.75033 |         1069 |       0.0118907   |          0.0013156   |              45633 | 0.130404    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| GAMT     | myeloid cell                                            |      1.66409 |         2424 |       0.0265292   |          0.00298317  |              45633 | 0.163322    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| HSPA1A   | myeloid cell                                            |      3.29825 |        33711 |       0.368149    |          0.0414874   |              45633 | 1.82068     | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| IFI27    | myeloid cell                                            |      2.27487 |         1065 |       0.0118545   |          0.00131067  |              45633 | 0.259288    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| KRT14    | myeloid cell                                            |      2.50277 |         1813 |       0.020245    |          0.00223122  |              45633 | 0.345699    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| LGALS3BP | myeloid cell                                            |      1.72185 |          494 |       0.00551746  |          0.000607956 |              45633 | 0.100184    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| RBM42    | myeloid cell                                            |      1.68895 |         3022 |       0.0330906   |          0.00371911  |              45633 | 0.193482    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| SCPEP1   | myeloid cell                                            |      1.69823 |         1874 |       0.0205031   |          0.00230629  |              45633 | 0.149622    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| COL17A1  | myeloid leukocyte                                       |      1.82266 |          154 |       0.0017331   |          0.000189525 |              44570 | 0.111233    | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| EFNA1    | myeloid leukocyte                                       |      1.74897 |         1066 |       0.0119967   |          0.0013119   |              44570 | 0.130157    | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| GAMT     | myeloid leukocyte                                       |      1.66388 |         2358 |       0.0264807   |          0.00290194  |              44570 | 0.161631    | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| HSPA1A   | myeloid leukocyte                                       |      3.30568 |        32937 |       0.369249    |          0.0405349   |              44570 | 1.80578     | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| IFI27    | myeloid leukocyte                                       |      2.2814  |         1053 |       0.011854    |          0.00129591  |              44570 | 0.260609    | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| KRT14    | myeloid leukocyte                                       |      2.49245 |         1779 |       0.0200915   |          0.00218938  |              44570 | 0.342122    | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| LGALS3BP | myeloid leukocyte                                       |      1.72071 |          492 |       0.00555938  |          0.000605494 |              44570 | 0.0999186   | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| RBM42    | myeloid leukocyte                                       |      1.68817 |         2931 |       0.0329026   |          0.00360712  |              44570 | 0.190846    | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| SCPEP1   | myeloid leukocyte                                       |      1.69969 |         1800 |       0.0201837   |          0.00221522  |              44570 | 0.147744    | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| COL17A1  | glial cell                                              |      1.94119 |           41 |       0.0545939   |          5.04579e-05 |               4616 | 0.216999    | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| EFNA1    | glial cell                                              |      1.89938 |          173 |       0.0337166   |          0.000212908 |               4616 | 0.178532    | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| GAMT     | glial cell                                              |      1.59641 |          228 |       0.0444358   |          0.000280595 |               4616 | 0.121387    | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| HSPA1A   | glial cell                                              |      3.26272 |         3004 |       0.585461    |          0.00369696  |               4616 | 1.40875     | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| IFI27    | glial cell                                              |      2.31001 |          115 |       0.0224128   |          0.000141528 |               4616 | 0.261211    | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| KRT14    | glial cell                                              |      1.70654 |           39 |       0.00760086  |          4.79965e-05 |               4616 | 0.0887382   | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| LGALS3BP | glial cell                                              |      1.67905 |          217 |       0.042292    |          0.000267058 |               4616 | 0.13824     | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| RBM42    | glial cell                                              |      1.64565 |          590 |       0.114987    |          0.000726101 |               4616 | 0.247924    | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| SCPEP1   | glial cell                                              |      1.85858 |          209 |       0.0407328   |          0.000257212 |               4616 | 0.179874    | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| COL17A1  | smooth muscle cell                                      |      1.56376 |            7 |       0.000694307 |          8.61476e-06 |               7125 | 0.042496    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| EFNA1    | smooth muscle cell                                      |      1.70133 |          191 |       0.0189409   |          0.00023506  |               7125 | 0.108077    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| GAMT     | smooth muscle cell                                      |      1.60753 |          538 |       0.0533624   |          0.000662106 |               7125 | 0.144874    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| HSPA1A   | smooth muscle cell                                      |      2.4113  |         3677 |       0.364709    |          0.00452521  |               7125 | 0.884156    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| IFI27    | smooth muscle cell                                      |      2.00899 |           34 |       0.00337235  |          4.18431e-05 |               7125 | 0.156679    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| KRT14    | smooth muscle cell                                      |      2.07262 |           10 |       0.000991867 |          1.23068e-05 |               7125 | 0.168191    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| LGALS3BP | smooth muscle cell                                      |      1.6601  |         1171 |       0.116148    |          0.00144113  |               7125 | 0.267015    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| RBM42    | smooth muscle cell                                      |      1.62437 |         1085 |       0.107618    |          0.00133529  |               7125 | 0.243389    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| SCPEP1   | smooth muscle cell                                      |      1.61323 |          412 |       0.0408568   |          0.00050704  |               7125 | 0.124527    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| COL17A1  | muscle cell                                             |      1.56376 |            7 |       0.000694307 |          8.61476e-06 |               7125 | 0.042496    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| EFNA1    | muscle cell                                             |      1.70133 |          191 |       0.0189409   |          0.00023506  |               7125 | 0.108077    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| GAMT     | muscle cell                                             |      1.60753 |          538 |       0.0533624   |          0.000662106 |               7125 | 0.144874    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| HSPA1A   | muscle cell                                             |      2.4113  |         3677 |       0.364709    |          0.00452521  |               7125 | 0.884156    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| IFI27    | muscle cell                                             |      2.00899 |           34 |       0.00337235  |          4.18431e-05 |               7125 | 0.156679    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| KRT14    | muscle cell                                             |      2.07262 |           10 |       0.000991867 |          1.23068e-05 |               7125 | 0.168191    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| LGALS3BP | muscle cell                                             |      1.6601  |         1171 |       0.116148    |          0.00144113  |               7125 | 0.267015    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| RBM42    | muscle cell                                             |      1.62437 |         1085 |       0.107618    |          0.00133529  |               7125 | 0.243389    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| SCPEP1   | muscle cell                                             |      1.61323 |          412 |       0.0408568   |          0.00050704  |               7125 | 0.124527    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| COL17A1  | macrophage                                              |      1.64287 |           76 |       0.00150337  |          9.35317e-05 |              30751 | 0.0648086   | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| EFNA1    | macrophage                                              |      1.71583 |          646 |       0.0127787   |          0.000795019 |              30751 | 0.113205    | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| GAMT     | macrophage                                              |      1.66559 |         2241 |       0.0443201   |          0.00275795  |              30751 | 0.186032    | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| HSPA1A   | macrophage                                              |      3.34191 |        21909 |       0.433292    |          0.026963    |              30751 | 1.64889     | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| IFI27    | macrophage                                              |      2.29345 |          969 |       0.0191638   |          0.00119253  |              30751 | 0.272543    | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| KRT14    | macrophage                                              |      2.49903 |         1436 |       0.0284058   |          0.00176726  |              30751 | 0.348068    | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| LGALS3BP | macrophage                                              |      1.71491 |          470 |       0.00929717  |          0.00057842  |              30751 | 0.103577    | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| RBM42    | macrophage                                              |      1.70106 |         2033 |       0.0402152   |          0.00250197  |              30751 | 0.183663    | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| SCPEP1   | macrophage                                              |      1.71638 |          971 |       0.0192076   |          0.00119499  |              30751 | 0.130701    | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| COL17A1  | inflammatory macrophage                                 |      1.39582 |            1 |       0.000660066 |          1.23068e-06 |                281 | 0.000990099 | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| EFNA1    | inflammatory macrophage                                 |      1.70227 |            2 |       0.00132013  |          2.46136e-06 |                281 | 0.077389    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| GAMT     | inflammatory macrophage                                 |      2.00453 |            1 |       0.000660066 |          1.23068e-06 |                281 | 0.15073     | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| HSPA1A   | inflammatory macrophage                                 |      2.64375 |          225 |       0.148515    |          0.000276903 |                281 | 0.535075    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| IFI27    | inflammatory macrophage                                 |      2.1218  |            3 |       0.0019802   |          3.69204e-06 |                281 | 0.181604    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| KRT14    | inflammatory macrophage                                 |      1.92635 |           26 |       0.0171617   |          3.19977e-05 |                281 | 0.156844    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| LGALS3BP | inflammatory macrophage                                 |      2.24559 |            1 |       0.000660066 |          1.23068e-06 |                281 | 0.210028    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| RBM42    | inflammatory macrophage                                 |      1.73662 |           20 |       0.0132013   |          2.46136e-05 |                281 | 0.104089    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| SCPEP1   | inflammatory macrophage                                 |      1.73518 |            2 |       0.00132013  |          2.46136e-06 |                281 | 0.0854838   | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| COL17A1  | elicited macrophage                                     |      1.39582 |            1 |       0.000660066 |          1.23068e-06 |                281 | 0.000990099 | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| EFNA1    | elicited macrophage                                     |      1.70227 |            2 |       0.00132013  |          2.46136e-06 |                281 | 0.077389    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| GAMT     | elicited macrophage                                     |      2.00453 |            1 |       0.000660066 |          1.23068e-06 |                281 | 0.15073     | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| HSPA1A   | elicited macrophage                                     |      2.64375 |          225 |       0.148515    |          0.000276903 |                281 | 0.535075    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| IFI27    | elicited macrophage                                     |      2.1218  |            3 |       0.0019802   |          3.69204e-06 |                281 | 0.181604    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| KRT14    | elicited macrophage                                     |      1.92635 |           26 |       0.0171617   |          3.19977e-05 |                281 | 0.156844    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| LGALS3BP | elicited macrophage                                     |      2.24559 |            1 |       0.000660066 |          1.23068e-06 |                281 | 0.210028    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| RBM42    | elicited macrophage                                     |      1.73662 |           20 |       0.0132013   |          2.46136e-05 |                281 | 0.104089    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| SCPEP1   | elicited macrophage                                     |      1.73518 |            2 |       0.00132013  |          2.46136e-06 |                281 | 0.0854838   | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| COL17A1  | mature T cell                                           |      1.99298 |         2792 |       0.0312123   |          0.00343606  |              63688 | 0.259991    | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| EFNA1    | mature T cell                                           |      1.71619 |         4674 |       0.0355178   |          0.0057522   |              63688 | 0.24305     | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| GAMT     | mature T cell                                           |      1.59136 |          646 |       0.00477211  |          0.000795019 |              63688 | 0.0705755   | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| HSPA1A   | mature T cell                                           |      3.22094 |        41573 |       0.282416    |          0.0511631   |              63688 | 1.85975     | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| IFI27    | mature T cell                                           |      1.67383 |          393 |       0.00383004  |          0.000483657 |              63688 | 0.0834413   | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| KRT14    | mature T cell                                           |      1.72142 |         1268 |       0.0090092   |          0.0015605   |              63688 | 0.123696    | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| LGALS3BP | mature T cell                                           |      1.66005 |          992 |       0.00680851  |          0.00122083  |              63688 | 0.0987449   | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| RBM42    | mature T cell                                           |      1.67054 |         9730 |       0.0660934   |          0.0119745   |              63688 | 0.397741    | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| SCPEP1   | mature T cell                                           |      1.61888 |         1620 |       0.0110051   |          0.0019937   |              63688 | 0.109824    | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| COL17A1  | phagocyte                                               |      1.77704 |          155 |       0.0015312   |          0.000190755 |              47900 | 0.0997307   | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| EFNA1    | phagocyte                                               |      1.70921 |         1269 |       0.0125361   |          0.00156173  |              47900 | 0.126006    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| GAMT     | phagocyte                                               |      1.66145 |         2403 |       0.0235189   |          0.00295732  |              47900 | 0.157658    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| HSPA1A   | phagocyte                                               |      3.33544 |        35836 |       0.350238    |          0.0441026   |              47900 | 1.85342     | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| IFI27    | phagocyte                                               |      2.27562 |         1076 |       0.0106323   |          0.00132421  |              47900 | 0.257901    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| KRT14    | phagocyte                                               |      2.49068 |         1822 |       0.0183692   |          0.0022423   |              47900 | 0.340123    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| LGALS3BP | phagocyte                                               |      1.7235  |          505 |       0.00499061  |          0.000621493 |              47900 | 0.100061    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| RBM42    | phagocyte                                               |      1.68847 |         2983 |       0.0291879   |          0.00367112  |              47900 | 0.186581    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| SCPEP1   | phagocyte                                               |      1.69516 |         1851 |       0.0180924   |          0.00227799  |              47900 | 0.144704    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| COL17A1  | mononuclear phagocyte                                   |      1.77704 |          155 |       0.0015312   |          0.000190755 |              47900 | 0.0997307   | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| EFNA1    | mononuclear phagocyte                                   |      1.70921 |         1269 |       0.0125361   |          0.00156173  |              47900 | 0.126006    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| GAMT     | mononuclear phagocyte                                   |      1.66145 |         2403 |       0.0235189   |          0.00295732  |              47900 | 0.157658    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| HSPA1A   | mononuclear phagocyte                                   |      3.33544 |        35836 |       0.350238    |          0.0441026   |              47900 | 1.85342     | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| IFI27    | mononuclear phagocyte                                   |      2.27562 |         1076 |       0.0106323   |          0.00132421  |              47900 | 0.257901    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| KRT14    | mononuclear phagocyte                                   |      2.49068 |         1822 |       0.0183692   |          0.0022423   |              47900 | 0.340123    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| LGALS3BP | mononuclear phagocyte                                   |      1.7235  |          505 |       0.00499061  |          0.000621493 |              47900 | 0.100061    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| RBM42    | mononuclear phagocyte                                   |      1.68847 |         2983 |       0.0291879   |          0.00367112  |              47900 | 0.186581    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| SCPEP1   | mononuclear phagocyte                                   |      1.69516 |         1851 |       0.0180924   |          0.00227799  |              47900 | 0.144704    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| COL17A1  | professional antigen presenting cell                    |      1.78531 |          145 |       0.00156867  |          0.000178449 |              43981 | 0.101586    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| EFNA1    | professional antigen presenting cell                    |      1.70483 |         1036 |       0.0112177   |          0.00127498  |              43981 | 0.117418    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| GAMT     | professional antigen presenting cell                    |      1.66309 |         2354 |       0.024974    |          0.00289702  |              43981 | 0.159082    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| HSPA1A   | professional antigen presenting cell                    |      3.31426 |        33064 |       0.349115    |          0.0406912   |              43981 | 1.7807      | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| IFI27    | professional antigen presenting cell                    |      2.28074 |         1069 |       0.0115784   |          0.0013156   |              43981 | 0.260413    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| KRT14    | professional antigen presenting cell                    |      2.49498 |         1812 |       0.0200633   |          0.00222999  |              43981 | 0.343486    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| LGALS3BP | professional antigen presenting cell                    |      1.72078 |          501 |       0.00537121  |          0.000616571 |              43981 | 0.099867    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| RBM42    | professional antigen presenting cell                    |      1.69602 |         2799 |       0.0295912   |          0.00344467  |              43981 | 0.184675    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| SCPEP1   | professional antigen presenting cell                    |      1.70874 |         1201 |       0.0126891   |          0.00147805  |              43981 | 0.124506    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| COL17A1  | secretory cell                                          |      1.75586 |           69 |       0.0103479   |          8.49169e-05 |               1752 | 0.105705    | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| EFNA1    | secretory cell                                          |      1.82132 |          471 |       0.0706359   |          0.00057965  |               1752 | 0.221785    | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| GAMT     | secretory cell                                          |      1.66955 |           52 |       0.00779844  |          6.39954e-05 |               1752 | 0.0802438   | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| HSPA1A   | secretory cell                                          |      2.68533 |          796 |       0.119376    |          0.000979621 |               1752 | 0.515155    | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| IFI27    | secretory cell                                          |      1.82996 |           21 |       0.00314937  |          2.58443e-05 |               1752 | 0.111995    | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| KRT14    | secretory cell                                          |      1.90006 |           21 |       0.00314937  |          2.58443e-05 |               1752 | 0.129239    | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| LGALS3BP | secretory cell                                          |      1.73906 |           27 |       0.00404919  |          3.32284e-05 |               1752 | 0.0911256   | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| RBM42    | secretory cell                                          |      1.67618 |          248 |       0.0371926   |          0.000305209 |               1752 | 0.130622    | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| SCPEP1   | secretory cell                                          |      1.83239 |           47 |       0.00704859  |          5.7842e-05  |               1752 | 0.11906     | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| COL17A1  | regulatory T cell                                       |      2.05545 |         2153 |       0.0852606   |          0.00264965  |              17196 | 0.341257    | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| EFNA1    | regulatory T cell                                       |      1.72124 |         1322 |       0.0523523   |          0.00162696  |              17196 | 0.189948    | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| GAMT     | regulatory T cell                                       |      1.58923 |          110 |       0.00327138  |          0.000135375 |              17196 | 0.0550732   | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| HSPA1A   | regulatory T cell                                       |      3.02232 |        10030 |       0.29829     |          0.0123437   |              17196 | 1.08569     | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| IFI27    | regulatory T cell                                       |      1.68632 |           85 |       0.00336607  |          0.000104608 |              17196 | 0.078505    | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| KRT14    | regulatory T cell                                       |      1.76705 |          281 |       0.00835688  |          0.000345821 |              17196 | 0.110505    | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| LGALS3BP | regulatory T cell                                       |      1.6566  |          312 |       0.00927881  |          0.000383972 |              17196 | 0.0854548   | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| RBM42    | regulatory T cell                                       |      1.67595 |         2487 |       0.0739628   |          0.0030607   |              17196 | 0.238887    | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| SCPEP1   | regulatory T cell                                       |      1.6268  |          416 |       0.0123717   |          0.000511963 |              17196 | 0.085232    | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| COL17A1  | erythrocyte                                             |      1.56496 |            4 |       0.00396825  |          4.92272e-06 |                381 | 0.0476319   | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| EFNA1    | erythrocyte                                             |      1.745   |            1 |       0.000992063 |          1.23068e-06 |                381 | 0.0873844   | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| GAMT     | erythrocyte                                             |      1.63528 |            5 |       0.00496032  |          6.1534e-06  |                381 | 0.0664418   | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| HSPA1A   | erythrocyte                                             |      2.86837 |          289 |       0.286706    |          0.000355666 |                381 | 0.799137    | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| IFI27    | erythrocyte                                             |      1.70169 |           12 |       0.0119048   |          1.47682e-05 |                381 | 0.0933603   | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| KRT14    | erythrocyte                                             |      3.04299 |           34 |       0.0337302   |          4.18431e-05 |                381 | 0.456574    | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| LGALS3BP | erythrocyte                                             |      1.39773 |            1 |       0.000992063 |          1.23068e-06 |                381 | 0.00195747  | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| RBM42    | erythrocyte                                             |      1.62902 |           27 |       0.0267857   |          3.32284e-05 |                381 | 0.0981624   | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| SCPEP1   | erythrocyte                                             |      1.61771 |            8 |       0.00793651  |          9.84544e-06 |                381 | 0.0666545   | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| COL17A1  | collagen secreting cell                                 |      1.65399 |           42 |       0.0191606   |          5.16886e-05 |                336 | 0.093223    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| EFNA1    | collagen secreting cell                                 |      1.69314 |          252 |       0.114964    |          0.000310131 |                336 | 0.251544    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| GAMT     | collagen secreting cell                                 |      1.48286 |            2 |       0.000912409 |          2.46136e-06 |                336 | 0.0228047   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| HSPA1A   | collagen secreting cell                                 |      1.76498 |           13 |       0.00593066  |          1.59988e-05 |                336 | 0.0999925   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| IFI27    | collagen secreting cell                                 |      1.51336 |            3 |       0.00136861  |          3.69204e-06 |                336 | 0.031014    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| KRT14    | collagen secreting cell                                 |      1.66766 |            9 |       0.00410584  |          1.10761e-05 |                336 | 0.073221    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| LGALS3BP | collagen secreting cell                                 |      1.51402 |            8 |       0.00364964  |          9.84544e-06 |                336 | 0.0347185   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| RBM42    | collagen secreting cell                                 |      1.6201  |            4 |       0.00182482  |          4.92272e-06 |                336 | 0.0579801   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| SCPEP1   | collagen secreting cell                                 |      1.90858 |            3 |       0.00136861  |          3.69204e-06 |                336 | 0.128237    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| COL17A1  | chondrocyte                                             |      1.65399 |           42 |       0.0191606   |          5.16886e-05 |                336 | 0.093223    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| EFNA1    | chondrocyte                                             |      1.69314 |          252 |       0.114964    |          0.000310131 |                336 | 0.251544    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| GAMT     | chondrocyte                                             |      1.48286 |            2 |       0.000912409 |          2.46136e-06 |                336 | 0.0228047   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| HSPA1A   | chondrocyte                                             |      1.76498 |           13 |       0.00593066  |          1.59988e-05 |                336 | 0.0999925   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| IFI27    | chondrocyte                                             |      1.51336 |            3 |       0.00136861  |          3.69204e-06 |                336 | 0.031014    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| KRT14    | chondrocyte                                             |      1.66766 |            9 |       0.00410584  |          1.10761e-05 |                336 | 0.073221    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| LGALS3BP | chondrocyte                                             |      1.51402 |            8 |       0.00364964  |          9.84544e-06 |                336 | 0.0347185   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| RBM42    | chondrocyte                                             |      1.6201  |            4 |       0.00182482  |          4.92272e-06 |                336 | 0.0579801   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| SCPEP1   | chondrocyte                                             |      1.90858 |            3 |       0.00136861  |          3.69204e-06 |                336 | 0.128237    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| COL17A1  | cytotoxic T cell                                        |      2.0734  |          108 |       0.00964803  |          0.000132913 |               4237 | 0.183695    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| EFNA1    | cytotoxic T cell                                        |      1.9245  |          218 |       0.0194747   |          0.000268288 |               4237 | 0.164416    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| GAMT     | cytotoxic T cell                                        |      1.5869  |           60 |       0.00536001  |          7.38408e-05 |               4237 | 0.0564467   | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| HSPA1A   | cytotoxic T cell                                        |      2.73629 |         2203 |       0.196802    |          0.00271119  |               4237 | 0.677239    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| IFI27    | cytotoxic T cell                                        |      1.84428 |           44 |       0.00393068  |          5.41499e-05 |               4237 | 0.117236    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| KRT14    | cytotoxic T cell                                        |      1.72273 |          427 |       0.0381454   |          0.0005255   |               4237 | 0.147752    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| LGALS3BP | cytotoxic T cell                                        |      1.83385 |           77 |       0.00687869  |          9.47623e-05 |               4237 | 0.119876    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| RBM42    | cytotoxic T cell                                        |      1.65433 |          808 |       0.0721815   |          0.000994389 |               4237 | 0.191028    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| SCPEP1   | cytotoxic T cell                                        |      1.61554 |          292 |       0.0260854   |          0.000359359 |               4237 | 0.100089    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| COL17A1  | Langerhans cell                                         |      2.07157 |           33 |       0.00264614  |          4.06124e-05 |               6294 | 0.17096     | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| EFNA1    | Langerhans cell                                         |      1.88806 |           21 |       0.00168391  |          2.58443e-05 |               6294 | 0.12409     | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| GAMT     | Langerhans cell                                         |      1.98046 |            1 |       8.04311e-05 |          1.23068e-06 |               6294 | 0.14394     | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| HSPA1A   | Langerhans cell                                         |      3.55051 |         5538 |       0.44407     |          0.00681551  |               6294 | 1.32763     | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| IFI27    | Langerhans cell                                         |      1.79029 |            7 |       0.000563018 |          8.61476e-06 |               6294 | 0.0980248   | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| KRT14    | Langerhans cell                                         |      2.52419 |          296 |       0.0237351   |          0.000364281 |               6294 | 0.320181    | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| LGALS3BP | Langerhans cell                                         |      2.00397 |            1 |       8.04311e-05 |          1.23068e-06 |               6294 | 0.149724    | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| RBM42    | Langerhans cell                                         |      1.71835 |          311 |       0.0249379   |          0.000382741 |               6294 | 0.12411     | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| SCPEP1   | Langerhans cell                                         |      1.68867 |           86 |       0.006896    |          0.000105838 |               6294 | 0.0844026   | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| COL17A1  | mast cell                                               |      1.91785 |           26 |       0.0059894   |          3.19977e-05 |               1299 | 0.137995    | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| EFNA1    | mast cell                                               |      1.89288 |          164 |       0.0377793   |          0.000201831 |               1299 | 0.182813    | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| GAMT     | mast cell                                               |      1.67259 |           49 |       0.0112877   |          6.03033e-05 |               1299 | 0.0861565   | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| HSPA1A   | mast cell                                               |      2.70184 |          740 |       0.170468    |          0.000910703 |               1299 | 0.594523    | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| IFI27    | mast cell                                               |      1.73093 |           15 |       0.00345543  |          1.84602e-05 |               1299 | 0.087951    | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| KRT14    | mast cell                                               |      1.85376 |           10 |       0.00230362  |          1.23068e-05 |               1299 | 0.11632     | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| LGALS3BP | mast cell                                               |      1.82399 |           13 |       0.0029947   |          1.59988e-05 |               1299 | 0.110105    | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| RBM42    | mast cell                                               |      1.6771  |          243 |       0.0559779   |          0.000299055 |               1299 | 0.158907    | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| SCPEP1   | mast cell                                               |      1.81405 |           39 |       0.00898411  |          4.79965e-05 |               1299 | 0.117262    | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| COL17A1  | conventional dendritic cell                             |      1.96267 |           56 |       0.00174613  |          6.89181e-05 |               8544 | 0.143368    | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| EFNA1    | conventional dendritic cell                             |      1.69358 |          332 |       0.010352    |          0.000408586 |               8544 | 0.0966352   | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| GAMT     | conventional dendritic cell                             |      1.662   |            3 |       9.36534e-05 |          3.69204e-06 |               8544 | 0.0656678   | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| HSPA1A   | conventional dendritic cell                             |      3.15739 |         7174 |       0.22294     |          0.0088289   |               8544 | 0.938075    | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| IFI27    | conventional dendritic cell                             |      2.20118 |           80 |       0.00249742  |          9.84544e-05 |               8544 | 0.203735    | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| KRT14    | conventional dendritic cell                             |      2.4741  |          343 |       0.010695    |          0.000422123 |               8544 | 0.289414    | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| LGALS3BP | conventional dendritic cell                             |      1.69148 |           17 |       0.000530703 |          2.09216e-05 |               8544 | 0.0739076   | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| RBM42    | conventional dendritic cell                             |      1.69122 |          437 |       0.013626    |          0.000537807 |               8544 | 0.10346     | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| SCPEP1   | conventional dendritic cell                             |      1.68653 |          102 |       0.00316977  |          0.000125529 |               8544 | 0.078667    | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| COL17A1  | dendritic cell                                          |      1.93185 |           68 |       0.00162676  |          8.36862e-05 |              12884 | 0.135893    | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| EFNA1    | dendritic cell                                          |      1.68661 |          390 |       0.00932992  |          0.000479965 |              12884 | 0.0947651   | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| GAMT     | dendritic cell                                          |      1.61806 |          108 |       0.0025272   |          0.000132913 |              12884 | 0.061002    | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| HSPA1A   | dendritic cell                                          |      3.25919 |        10882 |       0.253772    |          0.0133923   |              12884 | 1.09741     | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| IFI27    | dendritic cell                                          |      2.15758 |          100 |       0.00239446  |          0.000123068 |              12884 | 0.193331    | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| KRT14    | dendritic cell                                          |      2.4795  |          376 |       0.0094565   |          0.000462736 |              12884 | 0.289669    | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| LGALS3BP | dendritic cell                                          |      1.82685 |           29 |       0.000694395 |          3.56897e-05 |              12884 | 0.107737    | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| RBM42    | dendritic cell                                          |      1.68537 |          736 |       0.0172071   |          0.00090578  |              12884 | 0.114492    | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| SCPEP1   | dendritic cell                                          |      1.67329 |          195 |       0.00454747  |          0.000239983 |              12884 | 0.0796834   | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| COL17A1  | innate lymphoid cell                                    |      1.72381 |          314 |       0.0109618   |          0.000386433 |               7739 | 0.104559    | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| EFNA1    | innate lymphoid cell                                    |      1.75861 |         2281 |       0.0751516   |          0.00280718  |               7739 | 0.256112    | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| GAMT     | innate lymphoid cell                                    |      1.58901 |          234 |       0.00762811  |          0.000287979 |               7739 | 0.0644984   | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| HSPA1A   | innate lymphoid cell                                    |      3.39112 |         3058 |       0.0996871   |          0.00376342  |               7739 | 0.712953    | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| IFI27    | innate lymphoid cell                                    |      1.62581 |          114 |       0.00375593  |          0.000140298 |               7739 | 0.064893    | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| KRT14    | innate lymphoid cell                                    |      1.70616 |          113 |       0.00394484  |          0.000139067 |               7739 | 0.0849189   | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| LGALS3BP | innate lymphoid cell                                    |      1.70464 |          155 |       0.00541107  |          0.000190755 |               7739 | 0.0877412   | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| RBM42    | innate lymphoid cell                                    |      1.65819 |         1107 |       0.0360868   |          0.00136236  |               7739 | 0.144935    | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| SCPEP1   | innate lymphoid cell                                    |      1.64173 |          363 |       0.0118334   |          0.000446737 |               7739 | 0.0868382   | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| COL17A1  | natural killer cell                                     |      1.73014 |          260 |       0.0140222   |          0.000319977 |               5255 | 0.109425    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| EFNA1    | natural killer cell                                     |      1.76345 |         1810 |       0.0976162   |          0.00222753  |               5255 | 0.279816    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| GAMT     | natural killer cell                                     |      1.58108 |          157 |       0.00846726  |          0.000193217 |               5255 | 0.0619788   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| HSPA1A   | natural killer cell                                     |      3.82528 |         2098 |       0.113149    |          0.00258197  |               5255 | 0.817151    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| IFI27    | natural killer cell                                     |      1.63937 |           92 |       0.00496171  |          0.000113223 |               5255 | 0.0695156   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| KRT14    | natural killer cell                                     |      1.6922  |           78 |       0.00420667  |          9.5993e-05  |               5255 | 0.0810473   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| LGALS3BP | natural killer cell                                     |      1.7394  |          111 |       0.00598641  |          0.000136605 |               5255 | 0.0961099   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| RBM42    | natural killer cell                                     |      1.66056 |          555 |       0.029932    |          0.000683027 |               5255 | 0.123178    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| SCPEP1   | natural killer cell                                     |      1.62895 |           94 |       0.00506957  |          0.000115684 |               5255 | 0.0671619   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| COL17A1  | group 1 innate lymphoid cell                            |      1.73014 |          260 |       0.0140222   |          0.000319977 |               5255 | 0.109425    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| EFNA1    | group 1 innate lymphoid cell                            |      1.76345 |         1810 |       0.0976162   |          0.00222753  |               5255 | 0.279816    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| GAMT     | group 1 innate lymphoid cell                            |      1.58108 |          157 |       0.00846726  |          0.000193217 |               5255 | 0.0619788   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| HSPA1A   | group 1 innate lymphoid cell                            |      3.82528 |         2098 |       0.113149    |          0.00258197  |               5255 | 0.817151    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| IFI27    | group 1 innate lymphoid cell                            |      1.63937 |           92 |       0.00496171  |          0.000113223 |               5255 | 0.0695156   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| KRT14    | group 1 innate lymphoid cell                            |      1.6922  |           78 |       0.00420667  |          9.5993e-05  |               5255 | 0.0810473   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| LGALS3BP | group 1 innate lymphoid cell                            |      1.7394  |          111 |       0.00598641  |          0.000136605 |               5255 | 0.0961099   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| RBM42    | group 1 innate lymphoid cell                            |      1.66056 |          555 |       0.029932    |          0.000683027 |               5255 | 0.123178    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| SCPEP1   | group 1 innate lymphoid cell                            |      1.62895 |           94 |       0.00506957  |          0.000115684 |               5255 | 0.0671619   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| COL17A1  | alpha-beta T cell                                       |      1.78396 |          259 |       0.0250145   |          0.000318746 |              35237 | 0.139128    | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| EFNA1    | alpha-beta T cell                                       |      1.72252 |          848 |       0.0169312   |          0.00104362  |              35237 | 0.125877    | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| GAMT     | alpha-beta T cell                                       |      1.59657 |          348 |       0.00646132  |          0.000428277 |              35237 | 0.0673152   | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| HSPA1A   | alpha-beta T cell                                       |      3.40341 |        26371 |       0.401422    |          0.0324543   |              35237 | 1.72216     | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| IFI27    | alpha-beta T cell                                       |      1.68894 |           90 |       0.00382783  |          0.000110761 |              35237 | 0.0799609   | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| KRT14    | alpha-beta T cell                                       |      1.75644 |          170 |       0.0028739   |          0.000209216 |              35237 | 0.0970345   | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| LGALS3BP | alpha-beta T cell                                       |      1.63986 |          474 |       0.00739377  |          0.000583342 |              35237 | 0.082356    | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| RBM42    | alpha-beta T cell                                       |      1.6877  |         6110 |       0.0929914   |          0.00751945  |              35237 | 0.356351    | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| SCPEP1   | alpha-beta T cell                                       |      1.62284 |          567 |       0.00863093  |          0.000697795 |              35237 | 0.0822333   | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| COL17A1  | mature alpha-beta T cell                                |      1.78396 |          259 |       0.0250145   |          0.000318746 |              35188 | 0.139128    | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| EFNA1    | mature alpha-beta T cell                                |      1.72232 |          845 |       0.0168986   |          0.00103992  |              35188 | 0.125706    | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| GAMT     | mature alpha-beta T cell                                |      1.59701 |          346 |       0.00643386  |          0.000425815 |              35188 | 0.0673348   | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| HSPA1A   | mature alpha-beta T cell                                |      3.40246 |        26333 |       0.401338    |          0.0324075   |              35188 | 1.7209      | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| IFI27    | mature alpha-beta T cell                                |      1.68894 |           90 |       0.00382783  |          0.000110761 |              35188 | 0.0799609   | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| KRT14    | mature alpha-beta T cell                                |      1.75644 |          170 |       0.0028739   |          0.000209216 |              35188 | 0.0970345   | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| LGALS3BP | mature alpha-beta T cell                                |      1.63986 |          474 |       0.00739377  |          0.000583342 |              35188 | 0.082356    | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| RBM42    | mature alpha-beta T cell                                |      1.68769 |         6105 |       0.09303     |          0.0075133   |              35188 | 0.356288    | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| SCPEP1   | mature alpha-beta T cell                                |      1.62304 |          566 |       0.00862634  |          0.000696565 |              35188 | 0.0822502   | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| COL17A1  | CD8-positive, alpha-beta T cell                         |      1.78396 |          259 |       0.0250145   |          0.000318746 |              23663 | 0.139128    | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| EFNA1    | CD8-positive, alpha-beta T cell                         |      1.72573 |          813 |       0.0209915   |          0.00100054  |              23663 | 0.131925    | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| GAMT     | CD8-positive, alpha-beta T cell                         |      1.61122 |          244 |       0.00665939  |          0.000300286 |              23663 | 0.0687472   | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| HSPA1A   | CD8-positive, alpha-beta T cell                         |      3.39246 |        16625 |       0.34296     |          0.0204601   |              23663 | 1.40035     | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| IFI27    | CD8-positive, alpha-beta T cell                         |      1.67716 |           87 |       0.007109    |          0.000107069 |              23663 | 0.0819151   | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| KRT14    | CD8-positive, alpha-beta T cell                         |      1.75969 |          142 |       0.00301467  |          0.000174757 |              23663 | 0.0973793   | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| LGALS3BP | CD8-positive, alpha-beta T cell                         |      1.65067 |          351 |       0.00746904  |          0.000431969 |              23663 | 0.0822067   | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| RBM42    | CD8-positive, alpha-beta T cell                         |      1.70278 |         4705 |       0.0970603   |          0.00579035  |              23663 | 0.332802    | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| SCPEP1   | CD8-positive, alpha-beta T cell                         |      1.625   |          437 |       0.00901496  |          0.000537807 |              23663 | 0.0802534   | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| COL17A1  | memory T cell                                           |      1.78396 |          259 |       0.0250145   |          0.000318746 |              10052 | 0.139128    | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| EFNA1    | memory T cell                                           |      1.73028 |          772 |       0.0279568   |          0.000950085 |              10052 | 0.142518    | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| GAMT     | memory T cell                                           |      1.59918 |           90 |       0.00521437  |          0.000110761 |              10052 | 0.0599617   | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| HSPA1A   | memory T cell                                           |      3.36238 |         6338 |       0.229521    |          0.00780005  |              10052 | 0.978522    | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| IFI27    | memory T cell                                           |      1.6689  |           83 |       0.00801623  |          0.000102146 |              10052 | 0.0811473   | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| KRT14    | memory T cell                                           |      1.73576 |           85 |       0.00307815  |          0.000104608 |              10052 | 0.0902343   | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| LGALS3BP | memory T cell                                           |      1.6645  |          197 |       0.00713122  |          0.000242444 |              10052 | 0.0814455   | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| RBM42    | memory T cell                                           |      1.68107 |         2051 |       0.0742443   |          0.00252412  |              10052 | 0.230216    | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| SCPEP1   | memory T cell                                           |      1.59302 |          177 |       0.00640979  |          0.00021783  |              10052 | 0.0623033   | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| COL17A1  | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.78396 |          259 |       0.0250145   |          0.000318746 |              10050 | 0.139128    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| EFNA1    | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.73028 |          772 |       0.0279568   |          0.000950085 |              10050 | 0.142518    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| GAMT     | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.59918 |           90 |       0.00521437  |          0.000110761 |              10050 | 0.0599617   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| HSPA1A   | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      3.36238 |         6338 |       0.229521    |          0.00780005  |              10050 | 0.978522    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| IFI27    | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.6689  |           83 |       0.00801623  |          0.000102146 |              10050 | 0.0811473   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| KRT14    | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.73576 |           85 |       0.00307815  |          0.000104608 |              10050 | 0.0902343   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| LGALS3BP | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.66102 |          196 |       0.00709785  |          0.000241213 |              10050 | 0.0805143   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| RBM42    | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.68038 |         2050 |       0.0742377   |          0.00252289  |              10050 | 0.230011    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| SCPEP1   | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.59302 |          177 |       0.00640979  |          0.00021783  |              10050 | 0.0623033   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| COL17A1  | CD8-positive, alpha-beta memory T cell                  |      1.78396 |          259 |       0.0250145   |          0.000318746 |              10050 | 0.139128    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| EFNA1    | CD8-positive, alpha-beta memory T cell                  |      1.73028 |          772 |       0.0279568   |          0.000950085 |              10050 | 0.142518    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| GAMT     | CD8-positive, alpha-beta memory T cell                  |      1.59918 |           90 |       0.00521437  |          0.000110761 |              10050 | 0.0599617   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| HSPA1A   | CD8-positive, alpha-beta memory T cell                  |      3.36238 |         6338 |       0.229521    |          0.00780005  |              10050 | 0.978522    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| IFI27    | CD8-positive, alpha-beta memory T cell                  |      1.6689  |           83 |       0.00801623  |          0.000102146 |              10050 | 0.0811473   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| KRT14    | CD8-positive, alpha-beta memory T cell                  |      1.73576 |           85 |       0.00307815  |          0.000104608 |              10050 | 0.0902343   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| LGALS3BP | CD8-positive, alpha-beta memory T cell                  |      1.66102 |          196 |       0.00709785  |          0.000241213 |              10050 | 0.0805143   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| RBM42    | CD8-positive, alpha-beta memory T cell                  |      1.68038 |         2050 |       0.0742377   |          0.00252289  |              10050 | 0.230011    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| SCPEP1   | CD8-positive, alpha-beta memory T cell                  |      1.59302 |          177 |       0.00640979  |          0.00021783  |              10050 | 0.0623033   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| COL17A1  | helper T cell                                           |      1.6656  |          272 |       0.00637719  |          0.000334745 |              10399 | 0.0823658   | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| EFNA1    | helper T cell                                           |      1.69137 |         2286 |       0.0535965   |          0.00281333  |              10399 | 0.207356    | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| GAMT     | helper T cell                                           |      1.57104 |          156 |       0.00365751  |          0.000191986 |              10399 | 0.0522696   | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| HSPA1A   | helper T cell                                           |      2.96916 |         5231 |       0.122644    |          0.00643769  |              10399 | 0.695188    | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| IFI27    | helper T cell                                           |      1.6168  |          174 |       0.00407953  |          0.000214138 |              10399 | 0.0645876   | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| KRT14    | helper T cell                                           |      1.67825 |          396 |       0.00928444  |          0.000487349 |              10399 | 0.0927815   | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| LGALS3BP | helper T cell                                           |      1.63574 |          140 |       0.00328238  |          0.000172295 |              10399 | 0.0672426   | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| RBM42    | helper T cell                                           |      1.61835 |         1322 |       0.030995    |          0.00162696  |              10399 | 0.132602    | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| SCPEP1   | helper T cell                                           |      1.61014 |          422 |       0.00989403  |          0.000519347 |              10399 | 0.0775598   | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| COL17A1  | monocyte                                                |      1.74695 |           11 |       0.00123958  |          1.35375e-05 |               4265 | 0.0884729   | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| EFNA1    | monocyte                                                |      1.72868 |          233 |       0.0262565   |          0.000286748 |               4265 | 0.126775    | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| GAMT     | monocyte                                                |      1.57631 |           54 |       0.00608519  |          6.64567e-05 |               4265 | 0.0547857   | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| HSPA1A   | monocyte                                                |      3.56141 |         3045 |       0.343137    |          0.00374742  |               4265 | 1.11971     | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| IFI27    | monocyte                                                |      1.49393 |            7 |       0.000788821 |          8.61476e-06 |               4265 | 0.0254613   | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| KRT14    | monocyte                                                |      1.71089 |           10 |       0.00112689  |          1.23068e-05 |               4265 | 0.0794101   | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| LGALS3BP | monocyte                                                |      1.89668 |            6 |       0.000676133 |          7.38408e-06 |               4265 | 0.124342    | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| RBM42    | monocyte                                                |      1.57943 |          214 |       0.0241154   |          0.000263365 |               4265 | 0.0863973   | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| SCPEP1   | monocyte                                                |      1.6713  |          685 |       0.0771918   |          0.000843016 |               4265 | 0.199797    | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| COL17A1  | lymphocyte of B lineage                                 |      1.82389 |           18 |       0.00568541  |          2.21522e-05 |               8612 | 0.114235    | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| EFNA1    | lymphocyte of B lineage                                 |      1.71068 |           35 |       0.00380352  |          4.30738e-05 |               8612 | 0.0839665   | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| GAMT     | lymphocyte of B lineage                                 |      1.6078  |           71 |       0.00651675  |          8.73783e-05 |               8612 | 0.0635832   | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| HSPA1A   | lymphocyte of B lineage                                 |      3.33558 |         7663 |       0.689243    |          0.0094307   |               8612 | 1.69297     | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| IFI27    | lymphocyte of B lineage                                 |      1.93412 |            2 |       0.000631712 |          2.46136e-06 |               8612 | 0.133391    | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| KRT14    | lymphocyte of B lineage                                 |      1.66902 |            5 |       0.00157928  |          6.1534e-06  |               8612 | 0.0696697   | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| LGALS3BP | lymphocyte of B lineage                                 |      1.61585 |           20 |       0.00185736  |          2.46136e-05 |               8612 | 0.0573639   | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| RBM42    | lymphocyte of B lineage                                 |      1.60648 |          492 |       0.0442526   |          0.000605494 |               8612 | 0.12986     | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| SCPEP1   | lymphocyte of B lineage                                 |      1.64351 |          306 |       0.0276448   |          0.000376588 |               8612 | 0.109641    | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| COL17A1  | plasma cell                                             |      1.82389 |           18 |       0.00568541  |          2.21522e-05 |               1692 | 0.114235    | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| EFNA1    | plasma cell                                             |      1.73082 |           29 |       0.00915982  |          3.56897e-05 |               1692 | 0.0968128   | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| GAMT     | plasma cell                                             |      1.57039 |           16 |       0.0050537   |          1.96909e-05 |               1692 | 0.0508791   | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| HSPA1A   | plasma cell                                             |      3.91254 |         1562 |       0.493367    |          0.00192232  |               1692 | 1.39622     | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| IFI27    | plasma cell                                             |      1.93412 |            2 |       0.000631712 |          2.46136e-06 |               1692 | 0.133391    | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| KRT14    | plasma cell                                             |      1.66902 |            5 |       0.00157928  |          6.1534e-06  |               1692 | 0.0696697   | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| LGALS3BP | plasma cell                                             |      1.69397 |            8 |       0.00252685  |          9.84544e-06 |               1692 | 0.0772989   | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| RBM42    | plasma cell                                             |      1.59037 |           25 |       0.0078964   |          3.0767e-05  |               1692 | 0.0602733   | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| SCPEP1   | plasma cell                                             |      1.65661 |           27 |       0.00852811  |          3.32284e-05 |               1692 | 0.077563    | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| COL17A1  | granulocyte                                             |      2.35694 |            7 |       0.0218069   |          8.61476e-06 |                183 | 0.269284    | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| EFNA1    | granulocyte                                             |      1.51899 |            1 |       0.00311526  |          1.23068e-06 |                183 | 0.034972    | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| GAMT     | granulocyte                                             |      1.69463 |           12 |       0.0223881   |          1.47682e-05 |                183 | 0.10735     | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| HSPA1A   | granulocyte                                             |      2.30113 |          137 |       0.251838    |          0.000168603 |                183 | 0.603689    | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| IFI27    | granulocyte                                             |      1.51927 |            1 |       0.00311526  |          1.23068e-06 |                183 | 0.0350405   | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| KRT14    | granulocyte                                             |      1.78896 |            1 |       0.125       |          1.23068e-06 |                183 | 0.284212    | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| RBM42    | granulocyte                                             |      1.69825 |           18 |       0.0330882   |          2.21522e-05 |                183 | 0.124432    | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| SCPEP1   | granulocyte                                             |      1.88274 |            6 |       0.011194    |          7.38408e-06 |                183 | 0.136689    | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| COL17A1  | plasmacytoid dendritic cell                             |      1.71081 |            7 |       0.00343137  |          8.61476e-06 |                941 | 0.082776    | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| EFNA1    | plasmacytoid dendritic cell                             |      1.62931 |           34 |       0.0166667   |          4.18431e-05 |                941 | 0.0832205   | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| GAMT     | plasmacytoid dendritic cell                             |      1.71207 |           29 |       0.00962815  |          3.56897e-05 |                941 | 0.0929024   | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| HSPA1A   | plasmacytoid dendritic cell                             |      3.48787 |          691 |       0.229416    |          0.0008504   |                941 | 0.875142    | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| IFI27    | plasmacytoid dendritic cell                             |      2.26346 |            2 |       0.000980392 |          2.46136e-06 |                941 | 0.21493     | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| LGALS3BP | plasmacytoid dendritic cell                             |      2.19024 |            5 |       0.00245098  |          6.1534e-06  |                941 | 0.199194    | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| RBM42    | plasmacytoid dendritic cell                             |      1.69907 |          150 |       0.0498008   |          0.000184602 |                941 | 0.152838    | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| SCPEP1   | plasmacytoid dendritic cell                             |      1.69106 |           23 |       0.00763612  |          2.83056e-05 |                941 | 0.084605    | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| EFNA1    | activated CD8-positive, alpha-beta T cell               |      1.64463 |           21 |       0.00402685  |          2.58443e-05 |               5157 | 0.0677206   | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| GAMT     | activated CD8-positive, alpha-beta T cell               |      1.68873 |           59 |       0.010502    |          7.26101e-05 |               5157 | 0.0891836   | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| HSPA1A   | activated CD8-positive, alpha-beta T cell               |      4.06944 |         4378 |       0.779281    |          0.00538792  |               5157 | 1.93055     | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| IFI27    | activated CD8-positive, alpha-beta T cell               |      2.0178  |            2 |       0.00496278  |          2.46136e-06 |               5157 | 0.160471    | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| KRT14    | activated CD8-positive, alpha-beta T cell               |      1.78231 |            7 |       0.00134228  |          8.61476e-06 |               5157 | 0.0972302   | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| LGALS3BP | activated CD8-positive, alpha-beta T cell               |      1.6727  |           61 |       0.010858    |          7.50715e-05 |               5157 | 0.0858224   | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| RBM42    | activated CD8-positive, alpha-beta T cell               |      1.80002 |          557 |       0.0991456   |          0.000685489 |               5157 | 0.261351    | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| SCPEP1   | activated CD8-positive, alpha-beta T cell               |      1.7063  |           72 |       0.0128159   |          8.8609e-05  |               5157 | 0.0972858   | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| EFNA1    | naive T cell                                            |      1.62391 |           44 |       0.00430234  |          5.41499e-05 |               5149 | 0.0635847   | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| GAMT     | naive T cell                                            |      1.57514 |          110 |       0.0107558   |          0.000135375 |               5149 | 0.0628349   | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| HSPA1A   | naive T cell                                            |      2.93081 |         3724 |       0.364134    |          0.00458305  |               5149 | 1.01221     | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| IFI27    | naive T cell                                            |      2.19368 |            1 |       0.000301841 |          1.23068e-06 |               5149 | 0.196721    | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| KRT14    | naive T cell                                            |      1.73381 |           43 |       0.00621927  |          5.29192e-05 |               5149 | 0.093471    | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| LGALS3BP | naive T cell                                            |      1.62849 |          102 |       0.0099736   |          0.000125529 |               5149 | 0.0745954   | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| RBM42    | naive T cell                                            |      1.69566 |         1033 |       0.101007    |          0.00127129  |               5149 | 0.249775    | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| SCPEP1   | naive T cell                                            |      1.58941 |           92 |       0.0089958   |          0.000113223 |               5149 | 0.0632771   | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| EFNA1    | myofibroblast cell                                      |      1.63002 |           35 |       0.00632225  |          4.30738e-05 |               4558 | 0.0679039   | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| GAMT     | myofibroblast cell                                      |      1.58373 |          144 |       0.0260116   |          0.000177218 |               4558 | 0.0886375   | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| HSPA1A   | myofibroblast cell                                      |      2.78215 |         3419 |       0.617594    |          0.00420769  |               4558 | 1.34858     | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| IFI27    | myofibroblast cell                                      |      1.66971 |            9 |       0.00162572  |          1.10761e-05 |               4558 | 0.0700042   | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| KRT14    | myofibroblast cell                                      |      1.65642 |            5 |       0.000903179 |          6.1534e-06  |               4558 | 0.0655568   | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| LGALS3BP | myofibroblast cell                                      |      1.69038 |          482 |       0.0870665   |          0.000593188 |               4558 | 0.214482    | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| RBM42    | myofibroblast cell                                      |      1.58343 |          213 |       0.0384754   |          0.000262135 |               4558 | 0.108899    | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| SCPEP1   | myofibroblast cell                                      |      1.60999 |          251 |       0.0453396   |          0.000308901 |               4558 | 0.12663     | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| EFNA1    | CD4-positive, alpha-beta T cell                         |      1.63555 |           32 |       0.00283839  |          3.93818e-05 |              10660 | 0.0639652   | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| GAMT     | CD4-positive, alpha-beta T cell                         |      1.56443 |          100 |       0.00621775  |          0.000123068 |              10660 | 0.0531553   | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| HSPA1A   | CD4-positive, alpha-beta T cell                         |      3.43401 |         8958 |       0.556986    |          0.0110244   |              10660 | 1.54955     | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| IFI27    | CD4-positive, alpha-beta T cell                         |      2.03039 |            3 |       0.000266099 |          3.69204e-06 |              10660 | 0.156548    | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| KRT14    | CD4-positive, alpha-beta T cell                         |      1.73375 |           23 |       0.00208522  |          2.83056e-05 |              10660 | 0.0867798   | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| LGALS3BP | CD4-positive, alpha-beta T cell                         |      1.61059 |          121 |       0.00751833  |          0.000148912 |              10660 | 0.0669588   | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| RBM42    | CD4-positive, alpha-beta T cell                         |      1.6418  |         1312 |       0.0815211   |          0.00161465  |              10660 | 0.213922    | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| SCPEP1   | CD4-positive, alpha-beta T cell                         |      1.61102 |          111 |       0.0069017   |          0.000136605 |              10660 | 0.065903    | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| COL17A1  | granular cell of epidermis                              |      1.60525 |            3 |       0.0141509   |          3.69204e-06 |                496 | 0.0727938   | 0.672067  | 0.638418  | 1.05271  | -1.42142   |  0.404666  |   0.401355  | 0.40439   |       0.0598422 |       7 |      3.45841 |
| EFNA1    | granular cell of epidermis                              |      1.65991 |           45 |       0.212264    |          5.53806e-05 |                496 | 0.384406    | 0.672067  | 0.638418  | 1.05271  | -1.42142   |  0.404666  |   0.401355  | 0.40439   |       0.0598422 |       7 |      3.45841 |
| HSPA1A   | granular cell of epidermis                              |      2.86162 |          186 |       0.877358    |          0.000228906 |                496 | 1.68101     | 0.672067  | 0.638418  | 1.05271  | -1.42142   |  0.404666  |   0.401355  | 0.40439   |       0.0598422 |       7 |      3.45841 |
| IFI27    | granular cell of epidermis                              |      2.22769 |          104 |       0.490566    |          0.000127991 |                496 | 0.94293     | 0.672067  | 0.638418  | 1.05271  | -1.42142   |  0.404666  |   0.401355  | 0.40439   |       0.0598422 |       7 |      3.45841 |
| KRT14    | granular cell of epidermis                              |      2.20923 |          155 |       0.731132    |          0.000190755 |                496 | 1.30045     | 0.672067  | 0.638418  | 1.05271  | -1.42142   |  0.404666  |   0.401355  | 0.40439   |       0.0598422 |       7 |      3.45841 |
| RBM42    | granular cell of epidermis                              |      1.48347 |            2 |       0.00943396  |          2.46136e-06 |                496 | 0.0357361   | 0.672067  | 0.638418  | 1.05271  | -1.42142   |  0.404666  |   0.401355  | 0.40439   |       0.0598422 |       7 |      3.45841 |
| SCPEP1   | granular cell of epidermis                              |      1.57681 |            1 |       0.00471698  |          1.23068e-06 |                496 | 0.0515989   | 0.672067  | 0.638418  | 1.05271  | -1.42142   |  0.404666  |   0.401355  | 0.40439   |       0.0598422 |       7 |      3.45841 |
| COL17A1  | blood vessel endothelial cell                           |      1.77903 |            4 |       0.00543478  |          4.92272e-06 |                585 | 0.102492    | 0.239063  | 0.261568  | 0.913962 | -0.168862  |  0.113291  |   0.126522  | 0.313953  |       0.173634  |       7 |      2.29934 |
| EFNA1    | blood vessel endothelial cell                           |      1.79002 |          217 |       0.290495    |          0.000267058 |                585 | 0.537843    | 0.239063  | 0.261568  | 0.913962 | -0.168862  |  0.113291  |   0.126522  | 0.313953  |       0.173634  |       7 |      2.29934 |
| HSPA1A   | blood vessel endothelial cell                           |      1.72235 |           30 |       0.0401606   |          3.69204e-05 |                585 | 0.141254    | 0.239063  | 0.261568  | 0.913962 | -0.168862  |  0.113291  |   0.126522  | 0.313953  |       0.173634  |       7 |      2.29934 |
| IFI27    | blood vessel endothelial cell                           |      1.81992 |          280 |       0.374833    |          0.00034459  |                585 | 0.673199    | 0.239063  | 0.261568  | 0.913962 | -0.168862  |  0.113291  |   0.126522  | 0.313953  |       0.173634  |       7 |      2.29934 |
| LGALS3BP | blood vessel endothelial cell                           |      1.72467 |            7 |       0.00951087  |          8.61476e-06 |                585 | 0.0953035   | 0.239063  | 0.261568  | 0.913962 | -0.168862  |  0.113291  |   0.126522  | 0.313953  |       0.173634  |       7 |      2.29934 |
| RBM42    | blood vessel endothelial cell                           |      1.82397 |           24 |       0.0321285   |          2.95363e-05 |                585 | 0.154062    | 0.239063  | 0.261568  | 0.913962 | -0.168862  |  0.113291  |   0.126522  | 0.313953  |       0.173634  |       7 |      2.29934 |
| SCPEP1   | blood vessel endothelial cell                           |      1.71868 |           23 |       0.03125     |          2.83056e-05 |                585 | 0.12682     | 0.239063  | 0.261568  | 0.913962 | -0.168862  |  0.113291  |   0.126522  | 0.313953  |       0.173634  |       7 |      2.29934 |
| COL17A1  | suprabasal keratinocyte                                 |      1.88917 |           12 |       0.00657174  |          1.47682e-05 |                417 | 0.131481    | 0.123703  | 0.170117  | 0.727162 |  2.39987   |  0.0356631 |   0.0598275 | 0.192192  |       0.406996  |       7 |      1.94876 |
| EFNA1    | suprabasal keratinocyte                                 |      1.81992 |          370 |       0.202629    |          0.000455352 |                417 | 0.417031    | 0.123703  | 0.170117  | 0.727162 |  2.39987   |  0.0356631 |   0.0598275 | 0.192192  |       0.406996  |       7 |      1.94876 |
| HSPA1A   | suprabasal keratinocyte                                 |      2.42656 |            1 |       0.000547645 |          1.23068e-06 |                417 | 0.254379    | 0.123703  | 0.170117  | 0.727162 |  2.39987   |  0.0356631 |   0.0598275 | 0.192192  |       0.406996  |       7 |      1.94876 |
| IFI27    | suprabasal keratinocyte                                 |      1.68748 |            3 |       0.00164294  |          3.69204e-06 |                417 | 0.0742586   | 0.123703  | 0.170117  | 0.727162 |  2.39987   |  0.0356631 |   0.0598275 | 0.192192  |       0.406996  |       7 |      1.94876 |
| KRT14    | suprabasal keratinocyte                                 |      1.75237 |           10 |       0.00547645  |          1.23068e-05 |                417 | 0.0961366   | 0.123703  | 0.170117  | 0.727162 |  2.39987   |  0.0356631 |   0.0598275 | 0.192192  |       0.406996  |       7 |      1.94876 |
| LGALS3BP | suprabasal keratinocyte                                 |      1.74968 |           19 |       0.0104053   |          2.33829e-05 |                417 | 0.103083    | 0.123703  | 0.170117  | 0.727162 |  2.39987   |  0.0356631 |   0.0598275 | 0.192192  |       0.406996  |       7 |      1.94876 |
| SCPEP1   | suprabasal keratinocyte                                 |      1.8543  |            2 |       0.00109529  |          2.46136e-06 |                417 | 0.114449    | 0.123703  | 0.170117  | 0.727162 |  2.39987   |  0.0356631 |   0.0598275 | 0.192192  |       0.406996  |       7 |      1.94876 |
| COL17A1  | neutrophil                                              |      2.35694 |            7 |       0.0218069   |          8.61476e-06 |                105 | 0.269284    | 0.190538  | 0.204111  | 0.933504 |  3.17725   |  0.0806377 |   0.0846187 | 0.326691  |       0.477619  |       7 |      1.89517 |
| EFNA1    | neutrophil                                              |      1.51899 |            1 |       0.00311526  |          1.23068e-06 |                105 | 0.034972    | 0.190538  | 0.204111  | 0.933504 |  3.17725   |  0.0806377 |   0.0846187 | 0.326691  |       0.477619  |       7 |      1.89517 |
| GAMT     | neutrophil                                              |      1.89927 |            2 |       0.00623053  |          2.46136e-06 |                105 | 0.133217    | 0.190538  | 0.204111  | 0.933504 |  3.17725   |  0.0806377 |   0.0846187 | 0.326691  |       0.477619  |       7 |      1.89517 |
| HSPA1A   | neutrophil                                              |      2.15891 |           86 |       0.267913    |          0.000105838 |                105 | 0.591603    | 0.190538  | 0.204111  | 0.933504 |  3.17725   |  0.0806377 |   0.0846187 | 0.326691  |       0.477619  |       7 |      1.89517 |
| IFI27    | neutrophil                                              |      1.51927 |            1 |       0.00311526  |          1.23068e-06 |                105 | 0.0350405   | 0.190538  | 0.204111  | 0.933504 |  3.17725   |  0.0806377 |   0.0846187 | 0.326691  |       0.477619  |       7 |      1.89517 |
| RBM42    | neutrophil                                              |      2.08825 |            5 |       0.0155763   |          6.1534e-06  |                105 | 0.193793    | 0.190538  | 0.204111  | 0.933504 |  3.17725   |  0.0806377 |   0.0846187 | 0.326691  |       0.477619  |       7 |      1.89517 |
| SCPEP1   | neutrophil                                              |      2.03322 |            3 |       0.00934579  |          3.69204e-06 |                105 | 0.170864    | 0.190538  | 0.204111  | 0.933504 |  3.17725   |  0.0806377 |   0.0846187 | 0.326691  |       0.477619  |       7 |      1.89517 |
| COL17A1  | hematopoietic precursor cell                            |      1.69847 |            3 |       0.0681818   |          3.69204e-06 |                651 | 0.176771    | 0.282295  | 0.241034  | 1.17118  |  6.27904   |  0.142383  |   0.111547  | 0.481616  |       0.759408  |       7 |      1.67819 |
| GAMT     | hematopoietic precursor cell                            |      1.66424 |           59 |       0.0420527   |          7.26101e-05 |                651 | 0.130487    | 0.282295  | 0.241034  | 1.17118  |  6.27904   |  0.142383  |   0.111547  | 0.481616  |       0.759408  |       7 |      1.67819 |
| HSPA1A   | hematopoietic precursor cell                            |      3.00193 |          449 |       0.309442    |          0.000552575 |                651 | 0.869895    | 0.282295  | 0.241034  | 1.17118  |  6.27904   |  0.142383  |   0.111547  | 0.481616  |       0.759408  |       7 |      1.67819 |
| IFI27    | hematopoietic precursor cell                            |      1.43986 |            1 |       0.00833333  |          1.23068e-06 |                651 | 0.0233331   | 0.282295  | 0.241034  | 1.17118  |  6.27904   |  0.142383  |   0.111547  | 0.481616  |       0.759408  |       7 |      1.67819 |
| LGALS3BP | hematopoietic precursor cell                            |      1.98995 |            4 |       0.0160643   |          4.92272e-06 |                651 | 0.17032     | 0.282295  | 0.241034  | 1.17118  |  6.27904   |  0.142383  |   0.111547  | 0.481616  |       0.759408  |       7 |      1.67819 |
| RBM42    | hematopoietic precursor cell                            |      1.68981 |           62 |       0.0493631   |          7.63022e-05 |                651 | 0.147812    | 0.282295  | 0.241034  | 1.17118  |  6.27904   |  0.142383  |   0.111547  | 0.481616  |       0.759408  |       7 |      1.67819 |
| SCPEP1   | hematopoietic precursor cell                            |      1.68626 |           73 |       0.0636443   |          8.98396e-05 |                651 | 0.168623    | 0.282295  | 0.241034  | 1.17118  |  6.27904   |  0.142383  |   0.111547  | 0.481616  |       0.759408  |       7 |      1.67819 |
| EFNA1    | cell of skeletal muscle                                 |      1.75498 |           26 |       0.00440902  |          3.19977e-05 |               3085 | 0.0955577   | 0.316959  | 0.23415   | 1.35366  |  6.02534   |  0.165709  |   0.106526  | 0.600559  |       0.736361  |       7 |      1.57014 |
| GAMT     | cell of skeletal muscle                                 |      1.74092 |          317 |       0.0537561   |          0.000390126 |               3085 | 0.173031    | 0.316959  | 0.23415   | 1.35366  |  6.02534   |  0.165709  |   0.106526  | 0.600559  |       0.736361  |       7 |      1.57014 |
| HSPA1A   | cell of skeletal muscle                                 |      3.11061 |         1853 |       0.314228    |          0.00228045  |               3085 | 0.937148    | 0.316959  | 0.23415   | 1.35366  |  6.02534   |  0.165709  |   0.106526  | 0.600559  |       0.736361  |       7 |      1.57014 |
| IFI27    | cell of skeletal muscle                                 |      1.58793 |           17 |       0.0029473   |          2.09216e-05 |               3085 | 0.0520584   | 0.316959  | 0.23415   | 1.35366  |  6.02534   |  0.165709  |   0.106526  | 0.600559  |       0.736361  |       7 |      1.57014 |
| LGALS3BP | cell of skeletal muscle                                 |      1.58343 |          105 |       0.0182039   |          0.000129221 |               3085 | 0.0759272   | 0.316959  | 0.23415   | 1.35366  |  6.02534   |  0.165709  |   0.106526  | 0.600559  |       0.736361  |       7 |      1.57014 |
| RBM42    | cell of skeletal muscle                                 |      1.61349 |          659 |       0.111752    |          0.000811018 |               3085 | 0.236797    | 0.316959  | 0.23415   | 1.35366  |  6.02534   |  0.165709  |   0.106526  | 0.600559  |       0.736361  |       7 |      1.57014 |
| SCPEP1   | cell of skeletal muscle                                 |      1.5499  |          108 |       0.018724    |          0.000132913 |               3085 | 0.0685289   | 0.316959  | 0.23415   | 1.35366  |  6.02534   |  0.165709  |   0.106526  | 0.600559  |       0.736361  |       7 |      1.57014 |
| EFNA1    | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.57504 |           13 |       0.00294118  |          1.59988e-05 |               2873 | 0.048783    | 0.370943  | 0.24265   | 1.52872  |  5.35978   |  0.202035  |   0.112725  | 0.714664  |       0.675896  |       7 |      1.53754 |
| GAMT     | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.597   |           48 |       0.0108597   |          5.90726e-05 |               2873 | 0.0668945   | 0.370943  | 0.24265   | 1.52872  |  5.35978   |  0.202035  |   0.112725  | 0.714664  |       0.675896  |       7 |      1.53754 |
| HSPA1A   | naive thymus-derived CD8-positive, alpha-beta T cell    |      2.68139 |         2029 |       0.45905     |          0.00249705  |               2873 | 1.05297     | 0.370943  | 0.24265   | 1.52872  |  5.35978   |  0.202035  |   0.112725  | 0.714664  |       0.675896  |       7 |      1.53754 |
| KRT14    | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.71923 |            7 |       0.00158371  |          8.61476e-06 |               2873 | 0.0820749   | 0.370943  | 0.24265   | 1.52872  |  5.35978   |  0.202035  |   0.112725  | 0.714664  |       0.675896  |       7 |      1.53754 |
| LGALS3BP | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.54715 |           30 |       0.00678733  |          3.69204e-05 |               2873 | 0.048096    | 0.370943  | 0.24265   | 1.52872  |  5.35978   |  0.202035  |   0.112725  | 0.714664  |       0.675896  |       7 |      1.53754 |
| RBM42    | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.71362 |          690 |       0.156109    |          0.000849169 |               2873 | 0.3287      | 0.370943  | 0.24265   | 1.52872  |  5.35978   |  0.202035  |   0.112725  | 0.714664  |       0.675896  |       7 |      1.53754 |
| SCPEP1   | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.60199 |           56 |       0.0126697   |          6.89181e-05 |               2873 | 0.0710285   | 0.370943  | 0.24265   | 1.52872  |  5.35978   |  0.202035  |   0.112725  | 0.714664  |       0.675896  |       7 |      1.53754 |
| EFNA1    | activated CD8-positive, alpha-beta T cell, human        |      1.64463 |           21 |       0.00402685  |          2.58443e-05 |               5010 | 0.0677206   | 0.72904   | 0.370736  | 1.96647  |  6.88629   |  0.443004  |   0.206137  | 1         |       0.814576  |       7 |      1.52976 |
| GAMT     | activated CD8-positive, alpha-beta T cell, human        |      1.57597 |           49 |       0.00939597  |          6.03033e-05 |               5010 | 0.0595505   | 0.72904   | 0.370736  | 1.96647  |  6.88629   |  0.443004  |   0.206137  | 1         |       0.814576  |       7 |      1.52976 |
| HSPA1A   | activated CD8-positive, alpha-beta T cell, human        |      4.07181 |         4373 |       0.838543    |          0.00538176  |               5010 | 2.01991     | 0.72904   | 0.370736  | 1.96647  |  6.88629   |  0.443004  |   0.206137  | 1         |       0.814576  |       7 |      1.52976 |
| KRT14    | activated CD8-positive, alpha-beta T cell, human        |      1.78231 |            7 |       0.00134228  |          8.61476e-06 |               5010 | 0.0972302   | 0.72904   | 0.370736  | 1.96647  |  6.88629   |  0.443004  |   0.206137  | 1         |       0.814576  |       7 |      1.52976 |
| LGALS3BP | activated CD8-positive, alpha-beta T cell, human        |      1.64492 |           57 |       0.01093     |          7.01488e-05 |               5010 | 0.0790028   | 0.72904   | 0.370736  | 1.96647  |  6.88629   |  0.443004  |   0.206137  | 1         |       0.814576  |       7 |      1.52976 |
| RBM42    | activated CD8-positive, alpha-beta T cell, human        |      1.67584 |          446 |       0.0855225   |          0.000548883 |               5010 | 0.207735    | 0.72904   | 0.370736  | 1.96647  |  6.88629   |  0.443004  |   0.206137  | 1         |       0.814576  |       7 |      1.52976 |
| SCPEP1   | activated CD8-positive, alpha-beta T cell, human        |      1.58395 |           57 |       0.01093     |          7.01488e-05 |               5010 | 0.0640028   | 0.72904   | 0.370736  | 1.96647  |  6.88629   |  0.443004  |   0.206137  | 1         |       0.814576  |       7 |      1.52976 |
| EFNA1    | skeletal muscle satellite cell                          |      1.58024 |           15 |       0.00260055  |          1.84602e-05 |               2981 | 0.0495996   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| GAMT     | skeletal muscle satellite cell                          |      1.60194 |          242 |       0.0419556   |          0.000297825 |               2981 | 0.119361    | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| HSPA1A   | skeletal muscle satellite cell                          |      3.11306 |         1839 |       0.318828    |          0.00226322  |               2981 | 0.944318    | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| IFI27    | skeletal muscle satellite cell                          |      1.58793 |           17 |       0.0029473   |          2.09216e-05 |               2981 | 0.0520584   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| LGALS3BP | skeletal muscle satellite cell                          |      1.58343 |          105 |       0.0182039   |          0.000129221 |               2981 | 0.0759272   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| RBM42    | skeletal muscle satellite cell                          |      1.61323 |          655 |       0.113558    |          0.000806095 |               2981 | 0.239347    | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| SCPEP1   | skeletal muscle satellite cell                          |      1.5499  |          108 |       0.018724    |          0.000132913 |               2981 | 0.0685289   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| EFNA1    | muscle precursor cell                                   |      1.58024 |           15 |       0.00260055  |          1.84602e-05 |               2981 | 0.0495996   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| GAMT     | muscle precursor cell                                   |      1.60194 |          242 |       0.0419556   |          0.000297825 |               2981 | 0.119361    | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| HSPA1A   | muscle precursor cell                                   |      3.11306 |         1839 |       0.318828    |          0.00226322  |               2981 | 0.944318    | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| IFI27    | muscle precursor cell                                   |      1.58793 |           17 |       0.0029473   |          2.09216e-05 |               2981 | 0.0520584   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| LGALS3BP | muscle precursor cell                                   |      1.58343 |          105 |       0.0182039   |          0.000129221 |               2981 | 0.0759272   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| RBM42    | muscle precursor cell                                   |      1.61323 |          655 |       0.113558    |          0.000806095 |               2981 | 0.239347    | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| SCPEP1   | muscle precursor cell                                   |      1.5499  |          108 |       0.018724    |          0.000132913 |               2981 | 0.0685289   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| EFNA1    | central memory CD8-positive, alpha-beta T cell          |      1.64074 |            5 |       0.000289687 |          6.1534e-06  |               8656 | 0.0607775   | 0.418332  | 0.25391   | 1.64756  |  6.06088   |  0.233924  |   0.120937  | 0.792132  |       0.73959   |       7 |      1.45664 |
| GAMT     | central memory CD8-positive, alpha-beta T cell          |      1.59918 |           90 |       0.00521437  |          0.000110761 |               8656 | 0.0599617   | 0.418332  | 0.25391   | 1.64756  |  6.06088   |  0.233924  |   0.120937  | 0.792132  |       0.73959   |       7 |      1.45664 |
| HSPA1A   | central memory CD8-positive, alpha-beta T cell          |      3.37442 |         6293 |       0.3646      |          0.00774467  |               8656 | 1.18303     | 0.418332  | 0.25391   | 1.64756  |  6.06088   |  0.233924  |   0.120937  | 0.792132  |       0.73959   |       7 |      1.45664 |
| KRT14    | central memory CD8-positive, alpha-beta T cell          |      1.61031 |           30 |       0.00173812  |          3.69204e-05 |               8656 | 0.05606     | 0.418332  | 0.25391   | 1.64756  |  6.06088   |  0.233924  |   0.120937  | 0.792132  |       0.73959   |       7 |      1.45664 |
| LGALS3BP | central memory CD8-positive, alpha-beta T cell          |      1.61294 |           93 |       0.00538818  |          0.000114453 |               8656 | 0.0636775   | 0.418332  | 0.25391   | 1.64756  |  6.06088   |  0.233924  |   0.120937  | 0.792132  |       0.73959   |       7 |      1.45664 |
| RBM42    | central memory CD8-positive, alpha-beta T cell          |      1.67662 |         1978 |       0.1146      |          0.00243428  |               8656 | 0.28792     | 0.418332  | 0.25391   | 1.64756  |  6.06088   |  0.233924  |   0.120937  | 0.792132  |       0.73959   |       7 |      1.45664 |
| SCPEP1   | central memory CD8-positive, alpha-beta T cell          |      1.58884 |          167 |       0.00967555  |          0.000205524 |               8656 | 0.0659361   | 0.418332  | 0.25391   | 1.64756  |  6.06088   |  0.233924  |   0.120937  | 0.792132  |       0.73959   |       7 |      1.45664 |
| EFNA1    | naive thymus-derived CD4-positive, alpha-beta T cell    |      1.65535 |           28 |       0.00845155  |          3.4459e-05  |               1162 | 0.0771621   | 0.352109  | 0.223457  | 1.57573  |  6.54719   |  0.189361  |   0.0987278 | 0.745311  |       0.78377   |       7 |      1.38086 |
| GAMT     | naive thymus-derived CD4-positive, alpha-beta T cell    |      1.55486 |           44 |       0.013281    |          5.41499e-05 |               1162 | 0.0600654   | 0.352109  | 0.223457  | 1.57573  |  6.54719   |  0.189361  |   0.0987278 | 0.745311  |       0.78377   |       7 |      1.38086 |
| HSPA1A   | naive thymus-derived CD4-positive, alpha-beta T cell    |      3.64869 |          965 |       0.291277    |          0.00118761  |               1162 | 1.014       | 0.352109  | 0.223457  | 1.57573  |  6.54719   |  0.189361  |   0.0987278 | 0.745311  |       0.78377   |       7 |      1.38086 |
| IFI27    | naive thymus-derived CD4-positive, alpha-beta T cell    |      2.19368 |            1 |       0.000301841 |          1.23068e-06 |               1162 | 0.196721    | 0.352109  | 0.223457  | 1.57573  |  6.54719   |  0.189361  |   0.0987278 | 0.745311  |       0.78377   |       7 |      1.38086 |
| LGALS3BP | naive thymus-derived CD4-positive, alpha-beta T cell    |      1.6614  |           35 |       0.0105644   |          4.30738e-05 |               1162 | 0.0819843   | 0.352109  | 0.223457  | 1.57573  |  6.54719   |  0.189361  |   0.0987278 | 0.745311  |       0.78377   |       7 |      1.38086 |
| RBM42    | naive thymus-derived CD4-positive, alpha-beta T cell    |      1.61045 |           82 |       0.024751    |          0.000100916 |               1162 | 0.0918489   | 0.352109  | 0.223457  | 1.57573  |  6.54719   |  0.189361  |   0.0987278 | 0.745311  |       0.78377   |       7 |      1.38086 |
| SCPEP1   | naive thymus-derived CD4-positive, alpha-beta T cell    |      1.55478 |            7 |       0.00211289  |          8.61476e-06 |               1162 | 0.0424164   | 0.352109  | 0.223457  | 1.57573  |  6.54719   |  0.189361  |   0.0987278 | 0.745311  |       0.78377   |       7 |      1.38086 |
| GAMT     | muscle fibroblast                                       |      1.9998  |            3 |       0.0177515   |          3.69204e-06 |                105 | 0.175252    | 0.225436  | 0.332608  | 0.677784 |  2.05863   |  0.104121  |   0.178331  | 0.160007  |       0.375996  |       6 |      2.16492 |
| HSPA1A   | muscle fibroblast                                       |      2.05923 |            1 |       0.00591716  |          1.23068e-06 |                105 | 0.172071    | 0.225436  | 0.332608  | 0.677784 |  2.05863   |  0.104121  |   0.178331  | 0.160007  |       0.375996  |       6 |      2.16492 |
| IFI27    | muscle fibroblast                                       |      2.09803 |            6 |       0.035503    |          7.38408e-06 |                105 | 0.226113    | 0.225436  | 0.332608  | 0.677784 |  2.05863   |  0.104121  |   0.178331  | 0.160007  |       0.375996  |       6 |      2.16492 |
| LGALS3BP | muscle fibroblast                                       |      2.29223 |           25 |       0.147929    |          3.0767e-05  |                105 | 0.442974    | 0.225436  | 0.332608  | 0.677784 |  2.05863   |  0.104121  |   0.178331  | 0.160007  |       0.375996  |       6 |      2.16492 |
| RBM42    | muscle fibroblast                                       |      2.07413 |           65 |       0.384615    |          7.99942e-05 |                105 | 0.745303    | 0.225436  | 0.332608  | 0.677784 |  2.05863   |  0.104121  |   0.178331  | 0.160007  |       0.375996  |       6 |      2.16492 |
| SCPEP1   | muscle fibroblast                                       |      2.16601 |            5 |       0.0295858   |          6.1534e-06  |                105 | 0.233936    | 0.225436  | 0.332608  | 0.677784 |  2.05863   |  0.104121  |   0.178331  | 0.160007  |       0.375996  |       6 |      2.16492 |
| COL17A1  | myeloid lineage restricted progenitor cell              |      1.69847 |            3 |       0.0681818   |          3.69204e-06 |                358 | 0.176771    | 0.302208  | 0.321603  | 0.939692 |  4.57677   |  0.155782  |   0.170305  | 0.330724  |       0.604762  |       6 |      1.86052 |
| GAMT     | myeloid lineage restricted progenitor cell              |      1.77653 |           19 |       0.0286576   |          2.33829e-05 |                358 | 0.137066    | 0.302208  | 0.321603  | 0.939692 |  4.57677   |  0.155782  |   0.170305  | 0.330724  |       0.604762  |       6 |      1.86052 |
| HSPA1A   | myeloid lineage restricted progenitor cell              |      2.9989  |          244 |       0.34512     |          0.000300286 |                358 | 0.9178      | 0.302208  | 0.321603  | 0.939692 |  4.57677   |  0.155782  |   0.170305  | 0.330724  |       0.604762  |       6 |      1.86052 |
| LGALS3BP | myeloid lineage restricted progenitor cell              |      2.60733 |            1 |       0.037037    |          1.23068e-06 |                358 | 0.353581    | 0.302208  | 0.321603  | 0.939692 |  4.57677   |  0.155782  |   0.170305  | 0.330724  |       0.604762  |       6 |      1.86052 |
| RBM42    | myeloid lineage restricted progenitor cell              |      1.73506 |           34 |       0.0480905   |          4.18431e-05 |                358 | 0.15637     | 0.302208  | 0.321603  | 0.939692 |  4.57677   |  0.155782  |   0.170305  | 0.330724  |       0.604762  |       6 |      1.86052 |
| SCPEP1   | myeloid lineage restricted progenitor cell              |      1.66317 |           57 |       0.0806223   |          7.01488e-05 |                358 | 0.18803     | 0.302208  | 0.321603  | 0.939692 |  4.57677   |  0.155782  |   0.170305  | 0.330724  |       0.604762  |       6 |      1.86052 |
| EFNA1    | precursor B cell                                        |      1.77939 |            3 |       0.029703    |          3.69204e-06 |                215 | 0.138958    | 0.274598  | 0.266515  | 1.03033  |  5.15209   |  0.137203  |   0.13013   | 0.389802  |       0.657029  |       6 |      1.63612 |
| GAMT     | precursor B cell                                        |      1.65079 |           24 |       0.0533333   |          2.95363e-05 |                215 | 0.143267    | 0.274598  | 0.266515  | 1.03033  |  5.15209   |  0.137203  |   0.13013   | 0.389802  |       0.657029  |       6 |      1.63612 |
| HSPA1A   | precursor B cell                                        |      3.34609 |          100 |       0.222222    |          0.000123068 |                215 | 0.815439    | 0.274598  | 0.266515  | 1.03033  |  5.15209   |  0.137203  |   0.13013   | 0.389802  |       0.657029  |       6 |      1.63612 |
| LGALS3BP | precursor B cell                                        |      1.58173 |            7 |       0.0216718   |          8.61476e-06 |                215 | 0.0783844   | 0.274598  | 0.266515  | 1.03033  |  5.15209   |  0.137203  |   0.13013   | 0.389802  |       0.657029  |       6 |      1.63612 |
| RBM42    | precursor B cell                                        |      1.69821 |           52 |       0.115556    |          6.39954e-05 |                215 | 0.248931    | 0.274598  | 0.266515  | 1.03033  |  5.15209   |  0.137203  |   0.13013   | 0.389802  |       0.657029  |       6 |      1.63612 |
| SCPEP1   | precursor B cell                                        |      1.70794 |           29 |       0.0644444   |          3.56897e-05 |                215 | 0.174111    | 0.274598  | 0.266515  | 1.03033  |  5.15209   |  0.137203  |   0.13013   | 0.389802  |       0.657029  |       6 |      1.63612 |
| GAMT     | T follicular helper cell                                |      1.54709 |           10 |       0.00325839  |          1.23068e-05 |               2906 | 0.0423135   | 0.719337  | 0.403852  | 1.78119  |  5.55983   |  0.436475  |   0.230289  | 0.879231  |       0.694071  |       6 |      1.56135 |
| HSPA1A   | T follicular helper cell                                |      4.06852 |         2341 |       0.762789    |          0.00288102  |               2906 | 1.85722     | 0.719337  | 0.403852  | 1.78119  |  5.55983   |  0.436475  |   0.230289  | 0.879231  |       0.694071  |       6 |      1.56135 |
| KRT14    | T follicular helper cell                                |      1.78463 |           17 |       0.00553926  |          2.09216e-05 |               2906 | 0.104335    | 0.719337  | 0.403852  | 1.78119  |  5.55983   |  0.436475  |   0.230289  | 0.879231  |       0.694071  |       6 |      1.56135 |
| LGALS3BP | T follicular helper cell                                |      1.57704 |           28 |       0.00912349  |          3.4459e-05  |               2906 | 0.0589054   | 0.719337  | 0.403852  | 1.78119  |  5.55983   |  0.436475  |   0.230289  | 0.879231  |       0.694071  |       6 |      1.56135 |
| RBM42    | T follicular helper cell                                |      1.66144 |          484 |       0.157706    |          0.000595649 |               2906 | 0.31337     | 0.719337  | 0.403852  | 1.78119  |  5.55983   |  0.436475  |   0.230289  | 0.879231  |       0.694071  |       6 |      1.56135 |
| SCPEP1   | T follicular helper cell                                |      1.5327  |           26 |       0.00847181  |          3.19977e-05 |               2906 | 0.0469733   | 0.719337  | 0.403852  | 1.78119  |  5.55983   |  0.436475  |   0.230289  | 0.879231  |       0.694071  |       6 |      1.56135 |
| GAMT     | CD8-positive, alpha-beta regulatory T cell              |      1.54218 |           44 |       0.00525499  |          5.41499e-05 |               4446 | 0.0449076   | 0.390988  | 0.279145  | 1.40066  |  4.47838   |  0.215523  |   0.13934   | 0.631195  |       0.595823  |       6 |      1.5197  |
| HSPA1A   | CD8-positive, alpha-beta regulatory T cell              |      3.22247 |         2954 |       0.352801    |          0.00363543  |               4446 | 1.04867     | 0.390988  | 0.279145  | 1.40066  |  4.47838   |  0.215523  |   0.13934   | 0.631195  |       0.595823  |       6 |      1.5197  |
| KRT14    | CD8-positive, alpha-beta regulatory T cell              |      1.78774 |           42 |       0.00501612  |          5.16886e-05 |               4446 | 0.104907    | 0.390988  | 0.279145  | 1.40066  |  4.47838   |  0.215523  |   0.13934   | 0.631195  |       0.595823  |       6 |      1.5197  |
| LGALS3BP | CD8-positive, alpha-beta regulatory T cell              |      1.64866 |           48 |       0.00573271  |          5.90726e-05 |               4446 | 0.0719132   | 0.390988  | 0.279145  | 1.40066  |  4.47838   |  0.215523  |   0.13934   | 0.631195  |       0.595823  |       6 |      1.5197  |
| RBM42    | CD8-positive, alpha-beta regulatory T cell              |      1.70029 |         1253 |       0.149648    |          0.00154204  |               4446 | 0.329099    | 0.390988  | 0.279145  | 1.40066  |  4.47838   |  0.215523  |   0.13934   | 0.631195  |       0.595823  |       6 |      1.5197  |
| SCPEP1   | CD8-positive, alpha-beta regulatory T cell              |      1.61572 |          105 |       0.0125403   |          0.000129221 |               4446 | 0.0753732   | 0.390988  | 0.279145  | 1.40066  |  4.47838   |  0.215523  |   0.13934   | 0.631195  |       0.595823  |       6 |      1.5197  |
| EFNA1    | neuron                                                  |      1.9318  |            4 |       0.00612557  |          4.92272e-06 |                827 | 0.141107    | 0.247321  | 0.230306  | 1.07388  |  5.83406   |  0.118848  |   0.103723  | 0.418189  |       0.718984  |       6 |      1.48202 |
| GAMT     | neuron                                                  |      1.60593 |           83 |       0.0437533   |          0.000102146 |                827 | 0.119262    | 0.247321  | 0.230306  | 1.07388  |  5.83406   |  0.118848  |   0.103723  | 0.418189  |       0.718984  |       6 |      1.48202 |
| HSPA1A   | neuron                                                  |      2.71733 |          501 |       0.264101    |          0.000616571 |                827 | 0.733108    | 0.247321  | 0.230306  | 1.07388  |  5.83406   |  0.118848  |   0.103723  | 0.418189  |       0.718984  |       6 |      1.48202 |
| LGALS3BP | neuron                                                  |      1.64852 |           13 |       0.0199081   |          1.59988e-05 |                827 | 0.0923101   | 0.247321  | 0.230306  | 1.07388  |  5.83406   |  0.118848  |   0.103723  | 0.418189  |       0.718984  |       6 |      1.48202 |
| RBM42    | neuron                                                  |      1.62649 |          123 |       0.0648392   |          0.000151374 |                827 | 0.156899    | 0.247321  | 0.230306  | 1.07388  |  5.83406   |  0.118848  |   0.103723  | 0.418189  |       0.718984  |       6 |      1.48202 |
| SCPEP1   | neuron                                                  |      1.62056 |          103 |       0.0542963   |          0.00012676  |                827 | 0.139152    | 0.247321  | 0.230306  | 1.07388  |  5.83406   |  0.118848  |   0.103723  | 0.418189  |       0.718984  |       6 |      1.48202 |
| GAMT     | CD4-positive helper T cell                              |      1.51964 |           14 |       0.00291121  |          1.72295e-05 |               3987 | 0.0351355   | 0.625804  | 0.359663  | 1.73997  |  5.52127   |  0.373535  |   0.198062  | 0.852365  |       0.690568  |       6 |      1.48198 |
| HSPA1A   | CD4-positive helper T cell                              |      3.60764 |         3214 |       0.66833     |          0.00395541  |               3987 | 1.62289     | 0.625804  | 0.359663  | 1.73997  |  5.52127   |  0.373535  |   0.198062  | 0.852365  |       0.690568  |       6 |      1.48198 |
| KRT14    | CD4-positive helper T cell                              |      1.78463 |           17 |       0.00553926  |          2.09216e-05 |               3987 | 0.104335    | 0.625804  | 0.359663  | 1.73997  |  5.52127   |  0.373535  |   0.198062  | 0.852365  |       0.690568  |       6 |      1.48198 |
| LGALS3BP | CD4-positive helper T cell                              |      1.58012 |           38 |       0.00790185  |          4.67658e-05 |               3987 | 0.0580694   | 0.625804  | 0.359663  | 1.73997  |  5.52127   |  0.373535  |   0.198062  | 0.852365  |       0.690568  |       6 |      1.48198 |
| RBM42    | CD4-positive helper T cell                              |      1.65354 |          667 |       0.138698    |          0.000820863 |               3987 | 0.287261    | 0.625804  | 0.359663  | 1.73997  |  5.52127   |  0.373535  |   0.198062  | 0.852365  |       0.690568  |       6 |      1.48198 |
| SCPEP1   | CD4-positive helper T cell                              |      1.54988 |           37 |       0.00769391  |          4.55352e-05 |               3987 | 0.050294    | 0.625804  | 0.359663  | 1.73997  |  5.52127   |  0.373535  |   0.198062  | 0.852365  |       0.690568  |       6 |      1.48198 |
| EFNA1    | B cell                                                  |      1.61334 |            6 |       0.000994036 |          7.38408e-06 |               6920 | 0.055118    | 0.674496  | 0.362289  | 1.86176  |  5.92111   |  0.406301  |   0.199977  | 0.931752  |       0.726891  |       6 |      1.40094 |
| GAMT     | B cell                                                  |      1.61868 |           55 |       0.00711606  |          6.76874e-05 |               6920 | 0.0667797   | 0.674496  | 0.362289  | 1.86176  |  5.92111   |  0.406301  |   0.199977  | 0.931752  |       0.726891  |       6 |      1.40094 |
| HSPA1A   | B cell                                                  |      3.18786 |         6101 |       0.767228    |          0.00750838  |               6920 | 1.73652     | 0.674496  | 0.362289  | 1.86176  |  5.92111   |  0.406301  |   0.199977  | 0.931752  |       0.726891  |       6 |      1.40094 |
| LGALS3BP | B cell                                                  |      1.56378 |           12 |       0.00157853  |          1.47682e-05 |               6920 | 0.0439453   | 0.674496  | 0.362289  | 1.86176  |  5.92111   |  0.406301  |   0.199977  | 0.931752  |       0.726891  |       6 |      1.40094 |
| RBM42    | B cell                                                  |      1.60735 |          467 |       0.0587274   |          0.000574727 |               6920 | 0.151191    | 0.674496  | 0.362289  | 1.86176  |  5.92111   |  0.406301  |   0.199977  | 0.931752  |       0.726891  |       6 |      1.40094 |
| SCPEP1   | B cell                                                  |      1.64225 |          279 |       0.035303    |          0.00034336  |               6920 | 0.120175    | 0.674496  | 0.362289  | 1.86176  |  5.92111   |  0.406301  |   0.199977  | 0.931752  |       0.726891  |       6 |      1.40094 |
| GAMT     | mature NK T cell                                        |      1.59269 |            1 |       0.000980392 |          1.23068e-06 |                849 | 0.0498998   | 0.60169   | 0.333448  | 1.80445  |  5.86318   |  0.357308  |   0.178944  | 0.894391  |       0.721628  |       6 |      1.36757 |
| HSPA1A   | mature NK T cell                                        |      3.25101 |          737 |       0.722549    |          0.000907011 |                849 | 1.55767     | 0.60169   | 0.333448  | 1.80445  |  5.86318   |  0.357308  |   0.178944  | 0.894391  |       0.721628  |       6 |      1.36757 |
| KRT14    | mature NK T cell                                        |      1.7686  |            5 |       0.00490196  |          6.1534e-06  |                849 | 0.0991508   | 0.60169   | 0.333448  | 1.80445  |  5.86318   |  0.357308  |   0.178944  | 0.894391  |       0.721628  |       6 |      1.36757 |
| LGALS3BP | mature NK T cell                                        |      1.5144  |            2 |       0.00196078  |          2.46136e-06 |                849 | 0.0321351   | 0.60169   | 0.333448  | 1.80445  |  5.86318   |  0.357308  |   0.178944  | 0.894391  |       0.721628  |       6 |      1.36757 |
| RBM42    | mature NK T cell                                        |      1.56531 |           87 |       0.0852941   |          0.000107069 |                849 | 0.171677    | 0.60169   | 0.333448  | 1.80445  |  5.86318   |  0.357308  |   0.178944  | 0.894391  |       0.721628  |       6 |      1.36757 |
| SCPEP1   | mature NK T cell                                        |      1.65917 |           17 |       0.0166667   |          2.09216e-05 |                849 | 0.0901616   | 0.60169   | 0.333448  | 1.80445  |  5.86318   |  0.357308  |   0.178944  | 0.894391  |       0.721628  |       6 |      1.36757 |
| EFNA1    | group 3 innate lymphoid cell                            |      1.4744  |            2 |       0.00117165  |          2.46136e-06 |                348 | 0.0211112   | 0.247195  | 0.159634  | 1.54852  |  5.6878    |  0.118763  |   0.0521823 | 0.727569  |       0.705696  |       6 |      1.08795 |
| GAMT     | group 3 innate lymphoid cell                            |      1.59299 |           39 |       0.0228471   |          4.79965e-05 |                348 | 0.0836757   | 0.247195  | 0.159634  | 1.54852  |  5.6878    |  0.118763  |   0.0521823 | 0.727569  |       0.705696  |       6 |      1.08795 |
| HSPA1A   | group 3 innate lymphoid cell                            |      3.24405 |          228 |       0.133568    |          0.000280595 |                348 | 0.660397    | 0.247195  | 0.159634  | 1.54852  |  5.6878    |  0.118763  |   0.0521823 | 0.727569  |       0.705696  |       6 |      1.08795 |
| IFI27    | group 3 innate lymphoid cell                            |      1.50221 |            1 |       0.000585823 |          1.23068e-06 |                348 | 0.0270511   | 0.247195  | 0.159634  | 1.54852  |  5.6878    |  0.118763  |   0.0521823 | 0.727569  |       0.705696  |       6 |      1.08795 |
| RBM42    | group 3 innate lymphoid cell                            |      1.57458 |           59 |       0.0345636   |          7.26101e-05 |                348 | 0.0971958   | 0.247195  | 0.159634  | 1.54852  |  5.6878    |  0.118763  |   0.0521823 | 0.727569  |       0.705696  |       6 |      1.08795 |
| SCPEP1   | group 3 innate lymphoid cell                            |      1.60415 |           19 |       0.0111306   |          2.33829e-05 |                348 | 0.0683717   | 0.247195  | 0.159634  | 1.54852  |  5.6878    |  0.118763  |   0.0521823 | 0.727569  |       0.705696  |       6 |      1.08795 |
| EFNA1    | mesenchymal stem cell                                   |      2.8717  |            5 |       0.0174216   |          6.1534e-06  |                159 | 0.389286    | 0.209277  | 0.484093  | 0.432308 |  4.80638   |  0.0932477 |   0.288807  | 0         |       0.625622  |       5 |      2.15139 |
| GAMT     | mesenchymal stem cell                                   |      2.59288 |           12 |       0.0418118   |          1.47682e-05 |                159 | 0.35745     | 0.209277  | 0.484093  | 0.432308 |  4.80638   |  0.0932477 |   0.288807  | 0         |       0.625622  |       5 |      2.15139 |
| HSPA1A   | mesenchymal stem cell                                   |      2.80772 |           97 |       0.337979    |          0.000119376 |                159 | 0.856568    | 0.209277  | 0.484093  | 0.432308 |  4.80638   |  0.0932477 |   0.288807  | 0         |       0.625622  |       5 |      2.15139 |
| RBM42    | mesenchymal stem cell                                   |      2.55348 |           23 |       0.0801394   |          2.83056e-05 |                159 | 0.405509    | 0.209277  | 0.484093  | 0.432308 |  4.80638   |  0.0932477 |   0.288807  | 0         |       0.625622  |       5 |      2.15139 |
| SCPEP1   | mesenchymal stem cell                                   |      2.59979 |           22 |       0.0766551   |          2.7075e-05  |                159 | 0.411651    | 0.209277  | 0.484093  | 0.432308 |  4.80638   |  0.0932477 |   0.288807  | 0         |       0.625622  |       5 |      2.15139 |
| GAMT     | immature B cell                                         |      1.65071 |           14 |       0.0363636   |          1.72295e-05 |                310 | 0.117556    | 0.601996  | 0.408742  | 1.4728   |  4.71236   |  0.357514  |   0.233854  | 0.678219  |       0.61708   |       5 |      1.49208 |
| HSPA1A   | immature B cell                                         |      3.80141 |          226 |       0.587013    |          0.000278134 |                310 | 1.47762     | 0.601996  | 0.408742  | 1.4728   |  4.71236   |  0.357514  |   0.233854  | 0.678219  |       0.61708   |       5 |      1.49208 |
| LGALS3BP | immature B cell                                         |      1.52038 |            1 |       0.0025974   |          1.23068e-06 |                310 | 0.0345383   | 0.601996  | 0.408742  | 1.4728   |  4.71236   |  0.357514  |   0.233854  | 0.678219  |       0.61708   |       5 |      1.49208 |
| RBM42    | immature B cell                                         |      1.62108 |           44 |       0.114286    |          5.41499e-05 |                310 | 0.227861    | 0.601996  | 0.408742  | 1.4728   |  4.71236   |  0.357514  |   0.233854  | 0.678219  |       0.61708   |       5 |      1.49208 |
| SCPEP1   | immature B cell                                         |      1.75419 |           25 |       0.0649351   |          3.0767e-05  |                310 | 0.186131    | 0.601996  | 0.408742  | 1.4728   |  4.71236   |  0.357514  |   0.233854  | 0.678219  |       0.61708   |       5 |      1.49208 |
| GAMT     | small pre-B-II cell                                     |      1.63987 |            7 |       0.0315315   |          8.61476e-06 |                105 | 0.107474    | 0.306932  | 0.295212  | 1.0397   |  3.92762   |  0.158961  |   0.151058  | 0.395912  |       0.545789  |       5 |      1.47837 |
| HSPA1A   | small pre-B-II cell                                     |      3.41401 |           49 |       0.220721    |          6.03033e-05 |                105 | 0.828683    | 0.306932  | 0.295212  | 1.0397   |  3.92762   |  0.158961  |   0.151058  | 0.395912  |       0.545789  |       5 |      1.47837 |
| LGALS3BP | small pre-B-II cell                                     |      1.66686 |            1 |       0.0045045   |          1.23068e-06 |                105 | 0.0734321   | 0.306932  | 0.295212  | 1.0397   |  3.92762   |  0.158961  |   0.151058  | 0.395912  |       0.545789  |       5 |      1.47837 |
| RBM42    | small pre-B-II cell                                     |      1.65058 |           24 |       0.108108    |          2.95363e-05 |                105 | 0.225378    | 0.306932  | 0.295212  | 1.0397   |  3.92762   |  0.158961  |   0.151058  | 0.395912  |       0.545789  |       5 |      1.47837 |
| SCPEP1   | small pre-B-II cell                                     |      1.71446 |           24 |       0.108108    |          2.95363e-05 |                105 | 0.241093    | 0.306932  | 0.295212  | 1.0397   |  3.92762   |  0.158961  |   0.151058  | 0.395912  |       0.545789  |       5 |      1.47837 |
| GAMT     | pro-B cell                                              |      1.64269 |           24 |       0.0866426   |          2.95363e-05 |                101 | 0.191238    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| HSPA1A   | pro-B cell                                              |      2.77697 |           51 |       0.184116    |          6.27647e-05 |                101 | 0.617115    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| LGALS3BP | pro-B cell                                              |      1.51671 |            2 |       0.0196078   |          2.46136e-06 |                101 | 0.0591739   | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| RBM42    | pro-B cell                                              |      1.61114 |           22 |       0.0794224   |          2.7075e-05  |                101 | 0.172601    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| SCPEP1   | pro-B cell                                              |      2.17846 |            2 |       0.0114286   |          2.46136e-06 |                101 | 0.209691    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| GAMT     | lymphoid lineage restricted progenitor cell             |      1.64269 |           24 |       0.0866426   |          2.95363e-05 |                101 | 0.191238    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| HSPA1A   | lymphoid lineage restricted progenitor cell             |      2.77697 |           51 |       0.184116    |          6.27647e-05 |                101 | 0.617115    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| LGALS3BP | lymphoid lineage restricted progenitor cell             |      1.51671 |            2 |       0.0196078   |          2.46136e-06 |                101 | 0.0591739   | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| RBM42    | lymphoid lineage restricted progenitor cell             |      1.61114 |           22 |       0.0794224   |          2.7075e-05  |                101 | 0.172601    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| SCPEP1   | lymphoid lineage restricted progenitor cell             |      2.17846 |            2 |       0.0114286   |          2.46136e-06 |                101 | 0.209691    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| GAMT     | effector CD8-positive, alpha-beta T cell                |      1.68828 |            3 |       0.00309598  |          3.69204e-06 |                826 | 0.0766347   | 0.565649  | 0.357209  | 1.58352  |  4.62182   |  0.333056  |   0.196272  | 0.750388  |       0.608855  |       5 |      1.3408  |
| HSPA1A   | effector CD8-positive, alpha-beta T cell                |      2.53425 |          687 |       0.708978    |          0.000845477 |                826 | 1.35981     | 0.565649  | 0.357209  | 1.58352  |  4.62182   |  0.333056  |   0.196272  | 0.750388  |       0.608855  |       5 |      1.3408  |
| LGALS3BP | effector CD8-positive, alpha-beta T cell                |      1.64008 |           16 |       0.0165119   |          1.96909e-05 |                826 | 0.0852116   | 0.565649  | 0.357209  | 1.58352  |  4.62182   |  0.333056  |   0.196272  | 0.750388  |       0.608855  |       5 |      1.3408  |
| RBM42    | effector CD8-positive, alpha-beta T cell                |      1.631   |          112 |       0.115583    |          0.000137836 |                826 | 0.233865    | 0.565649  | 0.357209  | 1.58352  |  4.62182   |  0.333056  |   0.196272  | 0.750388  |       0.608855  |       5 |      1.3408  |
| SCPEP1   | effector CD8-positive, alpha-beta T cell                |      1.46891 |            8 |       0.00825593  |          9.84544e-06 |                826 | 0.0305309   | 0.565649  | 0.357209  | 1.58352  |  4.62182   |  0.333056  |   0.196272  | 0.750388  |       0.608855  |       5 |      1.3408  |
| GAMT     | T-helper 17 cell                                        |      1.45102 |            4 |       0.00229885  |          4.92272e-06 |               1081 | 0.0170991   | 0.421236  | 0.272943  | 1.54331  |  4.27586   |  0.235878  |   0.134817  | 0.724177  |       0.577425  |       5 |      1.20312 |
| HSPA1A   | T-helper 17 cell                                        |      2.37176 |          873 |       0.501724    |          0.00107438  |               1081 | 1.01337     | 0.421236  | 0.272943  | 1.54331  |  4.27586   |  0.235878  |   0.134817  | 0.724177  |       0.577425  |       5 |      1.20312 |
| LGALS3BP | T-helper 17 cell                                        |      1.58876 |           10 |       0.00574713  |          1.23068e-05 |               1081 | 0.0562977   | 0.421236  | 0.272943  | 1.54331  |  4.27586   |  0.235878  |   0.134817  | 0.724177  |       0.577425  |       5 |      1.20312 |
| RBM42    | T-helper 17 cell                                        |      1.63265 |          183 |       0.105172    |          0.000225214 |               1081 | 0.22034     | 0.421236  | 0.272943  | 1.54331  |  4.27586   |  0.235878  |   0.134817  | 0.724177  |       0.577425  |       5 |      1.20312 |
| SCPEP1   | T-helper 17 cell                                        |      1.59049 |           11 |       0.00632184  |          1.35375e-05 |               1081 | 0.0576079   | 0.421236  | 0.272943  | 1.54331  |  4.27586   |  0.235878  |   0.134817  | 0.724177  |       0.577425  |       5 |      1.20312 |
| GAMT     | B-2 B cell                                              |      1.51697 |            5 |       0.00521376  |          6.1534e-06  |                293 | 0.0377189   | 0.348671  | 0.2183    | 1.59721  |  4.9199    |  0.187048  |   0.0949673 | 0.759307  |       0.635935  |       5 |      1.04124 |
| HSPA1A   | B-2 B cell                                              |      3.3033  |          234 |       0.244004    |          0.000287979 |                293 | 0.840769    | 0.348671  | 0.2183    | 1.59721  |  4.9199    |  0.187048  |   0.0949673 | 0.759307  |       0.635935  |       5 |      1.04124 |
| LGALS3BP | B-2 B cell                                              |      1.56125 |            2 |       0.00208551  |          2.46136e-06 |                293 | 0.0438482   | 0.348671  | 0.2183    | 1.59721  |  4.9199    |  0.187048  |   0.0949673 | 0.759307  |       0.635935  |       5 |      1.04124 |
| RBM42    | B-2 B cell                                              |      1.59611 |           24 |       0.0250261   |          2.95363e-05 |                293 | 0.0873568   | 0.348671  | 0.2183    | 1.59721  |  4.9199    |  0.187048  |   0.0949673 | 0.759307  |       0.635935  |       5 |      1.04124 |
| SCPEP1   | B-2 B cell                                              |      1.54775 |           28 |       0.0291971   |          3.4459e-05  |                293 | 0.08181     | 0.348671  | 0.2183    | 1.59721  |  4.9199    |  0.187048  |   0.0949673 | 0.759307  |       0.635935  |       5 |      1.04124 |
| GAMT     | mature B cell                                           |      1.51697 |            5 |       0.00521376  |          6.1534e-06  |                334 | 0.0377189   | 0.33603   | 0.211555  | 1.58838  |  4.9227    |  0.178541  |   0.0900479 | 0.753555  |       0.636189  |       5 |      1.03103 |
| HSPA1A   | mature B cell                                           |      3.28633 |          268 |       0.226734    |          0.000329822 |                334 | 0.811497    | 0.33603   | 0.211555  | 1.58838  |  4.9227    |  0.178541  |   0.0900479 | 0.753555  |       0.636189  |       5 |      1.03103 |
| LGALS3BP | mature B cell                                           |      1.56125 |            2 |       0.00208551  |          2.46136e-06 |                334 | 0.0438482   | 0.33603   | 0.211555  | 1.58838  |  4.9227    |  0.178541  |   0.0900479 | 0.753555  |       0.636189  |       5 |      1.03103 |
| RBM42    | mature B cell                                           |      1.5841  |           29 |       0.0245347   |          3.56897e-05 |                334 | 0.0837835   | 0.33603   | 0.211555  | 1.58838  |  4.9227    |  0.178541  |   0.0900479 | 0.753555  |       0.636189  |       5 |      1.03103 |
| SCPEP1   | mature B cell                                           |      1.56054 |           30 |       0.0264784   |          3.69204e-05 |                334 | 0.0809272   | 0.33603   | 0.211555  | 1.58838  |  4.9227    |  0.178541  |   0.0900479 | 0.753555  |       0.636189  |       5 |      1.03103 |
| GAMT     | plasmacytoid dendritic cell, human                      |      1.91788 |            4 |       0.00411523  |          4.92272e-06 |                679 | 0.134668    | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| HSPA1A   | plasmacytoid dendritic cell, human                      |      3.61397 |          566 |       0.582305    |          0.000696565 |                679 | 1.43253     | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| RBM42    | plasmacytoid dendritic cell, human                      |      1.67242 |          101 |       0.103909    |          0.000124299 |                679 | 0.22628     | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| SCPEP1   | plasmacytoid dendritic cell, human                      |      1.69969 |            8 |       0.00823045  |          9.84544e-06 |                679 | 0.0872629   | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| GAMT     | dendritic cell, human                                   |      1.91788 |            4 |       0.00411523  |          4.92272e-06 |                679 | 0.134668    | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| HSPA1A   | dendritic cell, human                                   |      3.61397 |          566 |       0.582305    |          0.000696565 |                679 | 1.43253     | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| RBM42    | dendritic cell, human                                   |      1.67242 |          101 |       0.103909    |          0.000124299 |                679 | 0.22628     | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| SCPEP1   | dendritic cell, human                                   |      1.69969 |            8 |       0.00823045  |          9.84544e-06 |                679 | 0.0872629   | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| GAMT     | megakaryocyte                                           |      1.61952 |           34 |       0.0923913   |          4.18431e-05 |                182 | 0.194399    | 0.393567  | 0.340853  | 1.15465  |  3.76622   |  0.217259  |   0.184344  | 0.47084   |       0.531126  |       4 |      1.22847 |
| HSPA1A   | megakaryocyte                                           |      3.10014 |          124 |       0.336957    |          0.000152604 |                182 | 0.927608    | 0.393567  | 0.340853  | 1.15465  |  3.76622   |  0.217259  |   0.184344  | 0.47084   |       0.531126  |       4 |      1.22847 |
| RBM42    | megakaryocyte                                           |      1.64921 |           22 |       0.0597826   |          2.7075e-05  |                182 | 0.152506    | 0.393567  | 0.340853  | 1.15465  |  3.76622   |  0.217259  |   0.184344  | 0.47084   |       0.531126  |       4 |      1.22847 |
| SCPEP1   | megakaryocyte                                           |      1.72398 |            2 |       0.00543478  |          2.46136e-06 |                182 | 0.0889006   | 0.393567  | 0.340853  | 1.15465  |  3.76622   |  0.217259  |   0.184344  | 0.47084   |       0.531126  |       4 |      1.22847 |
| GAMT     | eurydendroid cell                                       |      1.5887  |           72 |       0.0578778   |          8.8609e-05  |                614 | 0.135951    | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| HSPA1A   | eurydendroid cell                                       |      2.91159 |          379 |       0.304662    |          0.000466428 |                614 | 0.83884     | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| RBM42    | eurydendroid cell                                       |      1.58079 |           62 |       0.0498392   |          7.63022e-05 |                614 | 0.121709    | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| SCPEP1   | eurydendroid cell                                       |      1.61242 |          101 |       0.0811897   |          0.000124299 |                614 | 0.177442    | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| GAMT     | efferent neuron                                         |      1.5887  |           72 |       0.0578778   |          8.8609e-05  |                614 | 0.135951    | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| HSPA1A   | efferent neuron                                         |      2.91159 |          379 |       0.304662    |          0.000466428 |                614 | 0.83884     | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| RBM42    | efferent neuron                                         |      1.58079 |           62 |       0.0498392   |          7.63022e-05 |                614 | 0.121709    | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| SCPEP1   | efferent neuron                                         |      1.61242 |          101 |       0.0811897   |          0.000124299 |                614 | 0.177442    | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| GAMT     | promonocyte                                             |      1.58959 |           11 |       0.0189003   |          1.35375e-05 |                281 | 0.0762541   | 0.398074  | 0.329009  | 1.20992  |  3.69284   |  0.220292  |   0.175706  | 0.506863  |       0.52446   |       4 |      1.19133 |
| HSPA1A   | promonocyte                                             |      3.09104 |          194 |       0.333333    |          0.000238752 |                281 | 0.921598    | 0.398074  | 0.329009  | 1.20992  |  3.69284   |  0.220292  |   0.175706  | 0.506863  |       0.52446   |       4 |      1.19133 |
| RBM42    | promonocyte                                             |      1.64234 |           24 |       0.0412371   |          2.95363e-05 |                281 | 0.123044    | 0.398074  | 0.329009  | 1.20992  |  3.69284   |  0.220292  |   0.175706  | 0.506863  |       0.52446   |       4 |      1.19133 |
| SCPEP1   | promonocyte                                             |      1.63936 |           52 |       0.0893471   |          6.39954e-05 |                281 | 0.195142    | 0.398074  | 0.329009  | 1.20992  |  3.69284   |  0.220292  |   0.175706  | 0.506863  |       0.52446   |       4 |      1.19133 |
| GAMT     | hematopoietic oligopotent progenitor cell               |      1.60085 |            6 |       0.0237154   |          7.38408e-06 |                113 | 0.0861279   | 0.448245  | 0.338345  | 1.32482  |  3.95364   |  0.254053  |   0.182515  | 0.581758  |       0.548152  |       4 |      1.15746 |
| HSPA1A   | hematopoietic oligopotent progenitor cell               |      3.12997 |           98 |       0.387352    |          0.000120607 |                113 | 1.00992     | 0.448245  | 0.338345  | 1.32482  |  3.95364   |  0.254053  |   0.182515  | 0.581758  |       0.548152  |       4 |      1.15746 |
| RBM42    | hematopoietic oligopotent progenitor cell               |      1.7339  |            3 |       0.0234375   |          3.69204e-06 |                113 | 0.11837     | 0.448245  | 0.338345  | 1.32482  |  3.95364   |  0.254053  |   0.182515  | 0.581758  |       0.548152  |       4 |      1.15746 |
| SCPEP1   | hematopoietic oligopotent progenitor cell               |      1.59799 |            6 |       0.0594059   |          7.38408e-06 |                113 | 0.138961    | 0.448245  | 0.338345  | 1.32482  |  3.95364   |  0.254053  |   0.182515  | 0.581758  |       0.548152  |       4 |      1.15746 |
| GAMT     | group 2 innate lymphoid cell                            |      1.59235 |            4 |       0.0123457   |          4.92272e-06 |                175 | 0.0669355   | 0.622311  | 0.378747  | 1.64308  |  3.94017   |  0.371185  |   0.211979  | 0.789209  |       0.546929  |       4 |      1.11061 |
| HSPA1A   | group 2 innate lymphoid cell                            |      3.77357 |          156 |       0.481481    |          0.000191986 |                175 | 1.31082     | 0.622311  | 0.378747  | 1.64308  |  3.94017   |  0.371185  |   0.211979  | 0.789209  |       0.546929  |       4 |      1.11061 |
| RBM42    | group 2 innate lymphoid cell                            |      1.59835 |           13 |       0.0401235   |          1.59988e-05 |                175 | 0.110291    | 0.622311  | 0.378747  | 1.64308  |  3.94017   |  0.371185  |   0.211979  | 0.789209  |       0.546929  |       4 |      1.11061 |
| SCPEP1   | group 2 innate lymphoid cell                            |      1.46761 |            2 |       0.00617284  |          2.46136e-06 |                175 | 0.0269434   | 0.622311  | 0.378747  | 1.64308  |  3.94017   |  0.371185  |   0.211979  | 0.789209  |       0.546929  |       4 |      1.11061 |

In [24]:
def export_dataframe(df):
    unique_cells = df['cell'].unique()[:20]
    filtered_df = df[df['cell'].isin(unique_cells)]

    final_df = (
        filtered_df.groupby('cell', as_index=False)
        .agg({'gene': lambda x: ", ".join(map(str, set(x))), 
              'cell score': lambda x: ", ".join(map(str, set(x)))})
    ).sort_values(by=['cell score'], ascending=[False])

    return final_df

In [25]:
final_df = export_dataframe(results[0])
display_dataframe_as_markdown(final_df)

| cell                                      | gene                                                                |   cell score |
|:------------------------------------------|:--------------------------------------------------------------------|-------------:|
| epithelial cell                           | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      7.67564 |
| epidermal cell                            | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      6.94493 |
| squamous epithelial cell                  | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      6.94387 |
| stratified squamous epithelial cell       | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      6.93894 |
| stratified epithelial cell                | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      6.93894 |
| keratinocyte                              | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      6.93894 |
| basal cell of epidermis                   | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      5.20789 |
| stem cell of epidermis                    | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      5.20348 |
| stem cell                                 | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      5.05776 |
| epithelial fate stem cell                 | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      5.03504 |
| basal cell                                | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      5.03504 |
| progenitor cell                           | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      4.9887  |
| precursor cell                            | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      4.91531 |
| prickle cell                              | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      4.3003  |
| skin fibroblast                           | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      4.22101 |
| columnar/cuboidal epithelial cell         | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      4.10285 |
| Merkel cell                               | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      4.10111 |
| neuron associated cell (sensu Vertebrata) | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      4.10111 |
| connective tissue cell                    | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      4.05615 |
| fibroblast                                | IFI27, KRT14, GAMT, EFNA1, RBM42, LGALS3BP, SCPEP1, COL17A1, HSPA1A |      3.98718 |

In [26]:
def get_top3_cells(df):
    return df['cell'].unique()[:3]

In [27]:
def export_top3_cells(results):

    top3cells = pd.DataFrame()
    for cluster in results:
        top3cells[f'{cluster}'] = get_top3_cells(results[cluster])

    top3cellsT = top3cells.T.reset_index()
    top3cellsT.columns = ['Cluster'] + [f'Cell {i+1}' for i in range(top3cellsT.shape[1]-1)]

    return top3cellsT

In [28]:
display_dataframe_as_markdown(export_top3_cells(results))

|   Cluster | Cell 1                 | Cell 2                              | Cell 3                              |
|----------:|:-----------------------|:------------------------------------|:------------------------------------|
|         0 | epithelial cell        | epidermal cell                      | squamous epithelial cell            |
|         1 | hematopoietic cell     | leukocyte                           | mononuclear cell                    |
|         2 | epithelial cell        | epidermal cell                      | squamous epithelial cell            |
|         3 | epidermal cell         | stratified squamous epithelial cell | stratified epithelial cell          |
|         4 | connective tissue cell | fibroblast                          | skin fibroblast                     |
|         5 | epithelial cell        | squamous epithelial cell            | epidermal cell                      |
|         6 | connective tissue cell | fibroblast                          | epithelial cell                     |
|         7 | epidermal cell         | stratified squamous epithelial cell | stratified epithelial cell          |
|         8 | epidermal cell         | stratified squamous epithelial cell | stratified epithelial cell          |
|         9 | epithelial cell        | squamous epithelial cell            | stratified squamous epithelial cell |
|        10 | connective tissue cell | fibroblast                          | skin fibroblast                     |
|        11 | connective tissue cell | fibroblast                          | skin fibroblast                     |
|        12 | connective tissue cell | fibroblast                          | skin fibroblast                     |
|        13 | connective tissue cell | fibroblast                          | skin fibroblast                     |
|        14 | epithelial cell        | connective tissue cell              | squamous epithelial cell            |
|        15 | connective tissue cell | fibroblast                          | skin fibroblast                     |
|        16 | epithelial cell        | stratified squamous epithelial cell | stratified epithelial cell          |
|        17 | connective tissue cell | fibroblast                          | skin fibroblast                     |
|        18 | sebaceous gland cell   | connective tissue cell              | lymphocyte                          |
|        19 | hematopoietic cell     | leukocyte                           | mononuclear cell                    |
|        20 | epithelial cell        | epidermal cell                      | stratified squamous epithelial cell |
|        21 | hematopoietic cell     | leukocyte                           | mononuclear cell                    |
|        22 | connective tissue cell | fibroblast                          | skin fibroblast                     |
|        23 | hematopoietic cell     | squamous epithelial cell            | stratified squamous epithelial cell |
|        24 | epithelial cell        | epidermal cell                      | stratified squamous epithelial cell |
|        25 | connective tissue cell | fibroblast                          | skin fibroblast                     |
|        26 | connective tissue cell | fibroblast                          | skin fibroblast                     |

# Validation with Tavily and Gemini Connected to a Database

In [30]:
import os
import psycopg2
import google.generativeai as genai
from tavily import TavilyClient
from dotenv import load_dotenv

In [31]:
# Load environment variables
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Initialize Tavily and Gemini clients
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)
genai.configure(api_key=GEMINI_API_KEY)

In [32]:
# Connect to PostgreSQL
def get_db_connection():
    return psycopg2.connect(DATABASE_URL)

In [33]:
# Step 2: Query Database for Existing Relation
def query_database(tissue_name, cell_name, gene_name):
    """Check if a tissue-cell-gene relation exists in the database."""
    conn = get_db_connection()
    cur = conn.cursor()
    query = """
        SELECT sources FROM cell_gene_relations
        WHERE tissue_name = %s AND cell_name = %s AND gene_name = %s;
    """
    cur.execute(query, (tissue_name, cell_name, gene_name))
    result = cur.fetchone()
    cur.close()
    conn.close()
    return result[0] if result else None  # Return sources if found

In [34]:
# Step 3: Query Tavily
def query_tavily(tissue_name, cell_name, gene_name):
    """Search Tavily for cell-gene relations in a specific tissue."""
    query = f"Is {cell_name} related to {gene_name} in {tissue_name}?"
    try:
        response = tavily_client.search(query, search_depth="advanced")
        return response.get("results", [])
    except Exception as e:
        print(f"Tavily Error: {e}")
        return []

In [59]:
import json
import google.generativeai as genai

def summarize_with_gemini(tissue_name, cell_name, gene_name, sources):
    """Use Gemini to summarize if a relation is valid, returning structured JSON."""
    prompt = f"""
    Given the following sources, determine if {cell_name} expresses {gene_name} in {tissue_name}.
    never respond with markdown.
    Respond ONLY with a string of {{ "valid": true }} or {{ "valid": false }}.
    
    Sources: {sources}
    """
    try:
        model = genai.GenerativeModel("gemini-2.0-flash")
        response = model.generate_content(prompt)
        output = response.text.strip()

        # Ensure output is valid JSON
        return json.loads(output)
    except json.JSONDecodeError:
        print(f"Gemini Error: Invalid JSON response: {output}")
        return {"valid": False}
    except Exception as e:
        print(f"Gemini Error: {e}")
        return {"valid": False}


In [53]:
# Step 5: Insert into Database
def insert_into_database(tissue_name, cell_name, gene_name, sources):
    """Insert a valid tissue-cell-gene relation into the database."""
    conn = get_db_connection()
    cur = conn.cursor()
    sources_str = ";".join(source["url"] for source in sources)
    query = """
        INSERT INTO cell_gene_relations (tissue_name, cell_name, gene_name, sources)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (tissue_name, cell_name, gene_name) DO NOTHING;
    """
    cur.execute(query, (tissue_name, cell_name, gene_name, sources_str))
    conn.commit()
    cur.close()
    conn.close()

In [43]:
# Full Workflow Execution
def validate_cell_gene_relation(tissue_name, cell_name, gene_name):
    """Run the full workflow to validate a cell-gene relation."""
    
    # Step 2: Check database first
    existing_sources = query_database(tissue_name, cell_name, gene_name)
    if existing_sources:
        print(f"Relation found in DB: {existing_sources}")
        return True  # Stop if already in DB

    # Step 3: Query Tavily
    sources = query_tavily(tissue_name, cell_name, gene_name)
    if not sources:
        print("No relevant sources found in Tavily.")
        return False  # Stop if no sources found

    # Step 4: Summarize with Gemini
    summary = summarize_with_gemini(tissue_name, cell_name, gene_name, sources)
    if not summary.get("valid", False):
        print("Gemini determined relation is invalid.")
        return False  # Stop if invalid

    # Step 5: Insert into Database
    insert_into_database(tissue_name, cell_name, gene_name, sources)
    print(f"Inserted into DB: {tissue_name} - {cell_name} - {gene_name}")
    return True

In [63]:
validate_cell_gene_relation("skin of body", "keratinocyte", "KRT14")

Relation found in DB: https://medlineplus.gov/genetics/gene/krt14/;https://pmc.ncbi.nlm.nih.gov/articles/PMC6713913/;https://www.sciencedirect.com/science/article/pii/S1525001616308383;https://elifesciences.org/articles/53165;https://pmc.ncbi.nlm.nih.gov/articles/PMC7250575/


True

# Analysis using OpenAI and Tavily

In [ ]:
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain_core.runnables import RunnableConfig, chain

In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import chain
from tavily import TavilyClient
import ast
import asyncio

In [6]:
from langchain_openai import ChatOpenAI

In [27]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini", 
    temperature=0
)

In [77]:
tavily = TavilyClient()

In [34]:
from langchain_community.tools import TavilySearchResults
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

In [55]:
tool = TavilySearchResults(
    max_results=10,
    search_depth="basic",
    include_answer=True,
    include_raw_content=False,
    include_images=False,
)

In [57]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    temperature=0.0
)

In [58]:
res = tool.invoke('is epithelial cell of skin of body tissue related to these genes: [COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1]?')

In [59]:
llm.invoke(f"answer with yes or no. based on this search results,\n{res}\n, is epithelial cell of skin of body tissue related to these genes: [COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1]?")

AIMessage(content='Yes', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-af3eb75a-090a-4dae-ad81-e85eecf4ed66-0', usage_metadata={'input_tokens': 1987, 'output_tokens': 2, 'total_tokens': 1989, 'input_token_details': {'cache_read': 0}})

In [83]:
tissue = tl.tl_uberon(TARGET_UBERON_ID)
tissue

'skin of body'

In [82]:
cell = final_df['cell'][0]
cell

'epithelial cell'

In [86]:
genes = [final_df['gene'][0]]
genes

['COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1']

In [ ]:
query = f"is {cell} of {tissue} tissue related to these genes: {genes}?"

In [ ]:
prompt = ChatPromptTemplate(query)

In [50]:
prompt = ChatPromptTemplate(
    [
        ("system", "You are an expert in cellular biology. answer with a boolean array"),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

In [70]:
final_df = export_dataframe(results[0])
display_dataframe_as_markdown(final_df)

| cells                                     | genes                                                               |   cell score |
|:------------------------------------------|:--------------------------------------------------------------------|-------------:|
| epithelial cell                           | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      7.67564 |
| epidermal cell                            | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      6.94493 |
| squamous epithelial cell                  | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      6.94387 |
| stratified squamous epithelial cell       | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      6.93894 |
| stratified epithelial cell                | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      6.93894 |
| keratinocyte                              | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      6.93894 |
| basal cell of epidermis                   | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      5.20789 |
| stem cell of epidermis                    | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      5.20348 |
| stem cell                                 | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      5.05776 |
| epithelial fate stem cell                 | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      5.03504 |
| basal cell                                | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      5.03504 |
| progenitor cell                           | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.9887  |
| precursor cell                            | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.91531 |
| prickle cell                              | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.3003  |
| skin fibroblast                           | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.22101 |
| columnar/cuboidal epithelial cell         | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.10285 |
| neuron associated cell (sensu Vertebrata) | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.10111 |
| Merkel cell                               | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.10111 |
| connective tissue cell                    | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.05615 |
| fibroblast                                | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      3.98718 |

In [74]:
final_df.head(1)

,cells,genes,cell score
0,epithelial cell,"COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LG...",7.675640368487279


In [97]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

# initialize the agent
agent = create_conversational_retrieval_agent(
    llm,
    tools=[tavily_tool],
)

In [81]:
def generate_queries(cell_name, gene_names: list):
    return f"answer with yes or no. Is {cell_name} related to this genes: {gene_names}?"

In [88]:
cell_names = results[0]['cell'].unique()[:20].tolist()

In [90]:
gene_names = results[0][results[0]['cell'] == cell_names[1]]['gene'].tolist()

In [91]:
gene_names

['COL17A1',
 'EFNA1',
 'GAMT',
 'HSPA1A',
 'IFI27',
 'KRT14',
 'LGALS3BP',
 'RBM42',
 'SCPEP1']

In [92]:
cell_names

['epithelial cell',
 'epidermal cell',
 'squamous epithelial cell',
 'stratified squamous epithelial cell',
 'stratified epithelial cell',
 'keratinocyte',
 'basal cell of epidermis',
 'stem cell of epidermis',
 'stem cell',
 'epithelial fate stem cell',
 'basal cell',
 'progenitor cell',
 'precursor cell',
 'prickle cell',
 'skin fibroblast',
 'columnar/cuboidal epithelial cell',
 'neuron associated cell (sensu Vertebrata)',
 'Merkel cell',
 'connective tissue cell',
 'fibroblast']

In [94]:
test = generate_queries(cell_names[1], gene_names)
test

"answer with yes or no. Is epidermal cell related to this genes: ['COL17A1', 'EFNA1', 'GAMT', 'HSPA1A', 'IFI27', 'KRT14', 'LGALS3BP', 'RBM42', 'SCPEP1']?"

In [100]:
# run the agent
resp = agent.invoke(test)

In [104]:
resp

{'input': "answer with yes or no. Is epidermal cell related to this genes: ['COL17A1', 'EFNA1', 'GAMT', 'HSPA1A', 'IFI27', 'KRT14', 'LGALS3BP', 'RBM42', 'SCPEP1']?",
 'chat_history': [HumanMessage(content="answer with yes or no. Is epidermal cell related to this genes: ['COL17A1', 'EFNA1', 'GAMT', 'HSPA1A', 'IFI27', 'KRT14', 'LGALS3BP', 'RBM42', 'SCPEP1']?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="answer with yes or no. Is epidermal cell related to this genes: ['COL17A1', 'EFNA1', 'GAMT', 'HSPA1A', 'IFI27', 'KRT14', 'LGALS3BP', 'RBM42', 'SCPEP1']?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes.', additional_kwargs={}, response_metadata={})],
 'output': 'Yes.',
 'intermediate_steps': []}

In [66]:
resp

{'input': "answer with yes or no. Is squamous epithelial cell related to this genes: ['ARG1', 'FLG2', 'KPRP', 'LCE1A', 'LCE1B', 'LCE1E', 'LCE2B', 'LCE2C', 'WFDC12']?",
 'chat_history': [HumanMessage(content="answer with yes or no. Is hematopoietic cell related to this genes: ['ARG1', 'FLG2', 'KPRP', 'LCE1A', 'LCE1B', 'LCE1E', 'LCE2B', 'LCE2C', 'WFDC12']?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='No.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="answer with yes or no. Is squamous epithelial cell related to this genes: ['ARG1', 'FLG2', 'KPRP', 'LCE1A', 'LCE1B', 'LCE1E', 'LCE2B', 'LCE2C', 'WFDC12']?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="answer with yes or no. Is squamous epithelial cell related to this genes: ['ARG1', 'FLG2', 'KPRP', 'LCE1A', 'LCE1B', 'LCE1E', 'LCE2B', 'LCE2C', 'WFDC12']?", additional_kwargs={}, response_metadata={}),

# Analysis using LangChain

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import JsonOutputParser

In [31]:
from dotenv import load_dotenv
load_dotenv()

True

In [45]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

load_dotenv(override=True)

client = MongoClient(os.getenv("MONGO_URI"), server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [30]:
llm = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
#JSON Output formatter  
class CellAnnotation(BaseModel):
    class cell(BaseModel):
        cell_type : str = Field(description="name of the chosen cell.")
        reason : str = Field(description="explain the reasoning for choosing the cell. give comments about parameters from the data that support the choice.")
        
    explanation : list[str] = Field(description="explanation of your reasoning step by step. focus on the parameters and number of genes expressed that support the choice of the cell.")
    cells : list[cell]
  

In [33]:
initial_df = df

In [ ]:
prompt = ChatPromptTemplate.from_template("""
                                          
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 5 most similar cell types to a given, unannotated cell cluster, based on a limited dataset.

The dataset is structured as follows:

Each row represents a known cell type expressing top genes from a cluster. The columns are: "Gene" (top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).

Given this dataset, outline the steps to create and apply a scoring system, then finally return a table of top 5 possible cell type identities for the cluster, and its calculated value.

Ensure that:
1. Describe each row of the dataset, and what it represents.
2. What metric is being used, and what value do they indicate.
3. The steps for scoring each possible cell type.
4. How to obtain the top five possible candidates.                                      
""")

chain = prompt | llm | StrOutputParser()

In [ ]:
parser = JsonOutputParser(pydantic_object=CellAnnotation)

prompt = PromptTemplate(
    template="""You are an expert in single-cell RNA sequencing.

        I have a cluster of cells with its genes as follow

        {initial_df}

        with the gene expression of each cell

        {expression_data}

        Based on the given data, which cell is the most valid to be annotated for this cluster of cells according to you?
        
        \n{format_instructions}""",
    input_variables=["initial_df", "expression_data"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

In [103]:
response = chain.invoke({'initial_df': initial_df, 'expression_data': results[0].to_csv(sep='\t', index=False)})

In [104]:
response

{'cell': 'stem cell of epidermis, stem cell, basal cell of epidermis, precursor cell, progenitor cell, epithelial fate stem cell, basal cell, columnar/cuboidal epithelial cell, neuron associated cell (sensu Vertebrata), Merkel cell',
 'explanation': 'The provided data strongly suggests that the cluster of cells is predominantly composed of epidermal cells.  The genes KRT14, which is highly expressed in keratinocytes (epidermal cells), is the most highly expressed gene in the cluster.  The cells listed above are all epidermal cell types or closely related stem/progenitor cells.  The high expression of KRT14, coupled with the presence of other epidermal-related cell types in the list, makes this the most likely annotation for the cluster.  The high cell counts and percentages for these cells further support this conclusion.  While other cell types might express some of these genes, the overall pattern points to a significant epidermal cell population.'}

# Analysis using OpenAI

In [ ]:
# MODEL = "o3-mini"

In [28]:
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [37]:
initial_df = df

In [279]:
instructions = """
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 5 most similar cell types to a given, unannotated cell cluster, based on a limited dataset. You will do this by creating and applying a scoring system.

The dataset is structured as follows:

Each row represents the cells that strongly express a top genes from the cluster. The columns are: "Gene" (a top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).

Follow these steps precisely to prepare the information, which will be automatically formatted into JSON:

**Step 1: Data Understanding**

*   Explain what each row in the dataset represents. Specifically, explain that each row shows a potential connection between the top gene from the cluster and one known cell type, and the metrics for THAT cell type, not for the cluster itself.
*   Explain the meaning of each column: "Gene", "Cell Name", "Expression Value", "Cell Percentage", "Cell Count", and "Tissue Composition." For each, explain what a higher or lower value might indicate *in the context of identifying similarity to the unannotated cluster*.

**Step 2: Develop a Scoring Function (CellTypeScore)**

*   Explain that the goal is to assign each known "Cell Name" (cell type) a score that reflects its similarity to the unannotated cluster.
*   Propose a formula for calculating a `GeneScore` for each *Gene* within each *Cell Name*. This formula should incorporate both "Cell Percentage" and "Expression Value". Justify why you chose to include these two metrics and how they are combined.
*   Propose a formula for combining the `GeneScore` values across all the `Gene` entries for a given "Cell Name" to calculate a final overall `CellTypeScore` for that cell type.

**Step 3: Calculate CellTypeScore for All Cell Names**

*   Explain that the goal is to now apply your scoring system to the known "Cell Name" values and find out which ones are the highest, with relation to what each Gene expresses.
*   Describe in precise steps what it should do: go through each unique "Cell Name" in the dataset, calculate a `GeneScore` and `CellTypeScore` based on Gene, Expression Value, and the percentage that are expressed.

**Step 4: Identify Top 5 Candidates**

*   Explain how to identify the top 5 candidates from the `CellTypeScore` values, now that they have been calculated in Step 3.
"""

In [73]:
instructions ="""
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 3 most similar cell types to a given, unannotated cell cluster, based on a limited dataset.

The dataset is structured as follows:

Each row represents a known cell type expressing top genes from a cluster. The columns are: "Gene" (top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).

Given this dataset, analyze the dataset to determine similarity, then finally return a list of top possible cell type identities for the cluster.

Ensure that:
1. Never make any assumption. If a gene is not expressed, it should be treated as such and not used for consideration.
2. Understand each row of the data and what it represents.
3. Prioritize cell types that's expressing more genes from the cluster, then consider the cell count, cell percentage, and expression value.
4. The steps for determining each possible cell type.
5. How to obtain the top three possible candidates.
6. give the explanation as if writing a report
"""

In [269]:
instructions ="""
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 3 most similar cell types to a given, unannotated cell cluster, based on a limited dataset.

The dataset is structured as follows:
Each row represents a known cell type expressing top genes from a cluster. The columns are: "Gene" (top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).
Given this dataset, analyze the dataset to determine similarity, then finally return a list of top possible cell type identities for the cluster. The dataset does not contain any representation of the unannotated cell cluster.

Cell similiarity is determined by the number of genes expressed in the cell then the cell count, cell percentage, and expression value. The more genes expressed in a cell, the higher the cell count, cell percentage, and expression value, the more similar the cell is to the unannotated cell cluster. the number of genes expressed in the cell is the most important factor in determining the similarity of the cell to the unannotated cell cluster.

Follow these steps precisely to prepare the information
Step 1 : Read the whole dataset
Step 2 : Explain to me what do you think about each cell's similiarity to the unannotated cell cluster based on the given data.
Step 3 : Based on you analysis, what are the top 3 possible cell type identities for the cluster?

Ensure that:
1. Never make any assumption. If a gene is not expressed, it should be treated as such and not used for consideration.
2. Do not consider cells that expressing lesser amount of genes from the cluster.
3. Do not consider cells that express a gene with parameters that deviate too much from other genes expressed by that cell.
4. Explanations must be written like a report.
"""

In [103]:
#JSON Output formatter  
class CellAnnotation(BaseModel):
    class top_cell(BaseModel):
        class gene(BaseModel):
            gene_name : str = Field(description="Name of the gene expressed by the cell type")
            metrics : str = Field(description="cell count, cell percentage, and expression value of the gene")
        
        cell_type : str = Field(description="name of the chosen cell.")
        gene_info : list[gene]
        reason : str = Field(description="explain the reasoning for choosing the cell. give comments about parameters from the data that support the choice.")
    
    explanation : list[str] = Field(description="explanation of your reasoning step by step. focus on the parameters and number of genes expressed that support the choice of the cell.")
    cells : list[top_cell]

In [270]:
print(results[0].to_csv(sep='\t', index=False))

cell	gene	expression	cell count	cell percentage	tissue composition	score	std	kurtosis	mean	CV
CD8-positive, alpha-beta T cell	RBM42	1.950844832292692	254	0.0186545240893067	0.0003493843819679144	9.243588807299892	3.6248595516837985	4.631287732326113	2.821587956603296	1.2846877742019789
CD8-positive, alpha-beta T cell	KRT14	1.8208990097045898	56	0.0047317279256442754	7.702962752048506e-05	0.48249592686384396	3.6248595516837985	4.631287732326113	2.821587956603296	1.2846877742019789
CD8-positive, alpha-beta T cell	GAMT	1.7516368865966796	40	0.022459292532285232	5.502116251463219e-05	1.5736210098566463	3.6248595516837985	4.631287732326113	2.821587956603296	1.2846877742019789
CD8-positive, alpha-beta T cell	LGALS3BP	1.713193620954241	112	0.0092295014421096	0.00015405925504097012	1.770935375463535	3.6248595516837985	4.631287732326113	2.821587956603296	1.2846877742019789
CD8-positive, alpha-beta T cell	IFI27	1.6771648889300468	87	0.0071090047393364926	0.00011967102846932501	1.0372986635325645

In [280]:
content = f"expression data : {results[23].drop(columns=['tissue composition']).to_csv(sep='\t', index=False)}"

In [281]:
#normal
completion = client.beta.chat.completions.parse(
    model = 'gpt-4o-mini-2024-07-18',
    messages=[
        {"role": "developer", 
         "content": instructions},
        {"role": "user", 
         "content": content}
    ],
    temperature=0.0,
    response_format=CellAnnotation,
)

In [ ]:
#Reasoning Model
response = client.chat.completions.create(
    model = "o3-mini",
    reasoning_effort="medium",
    messages=[
        {"role": "developer", 
         "content": """
            - You are an expert in single-cell RNA sequencing. 
            - You will be provided with cell clustering data and gene expression data for each cell. 
            - Based on the given data, you need to identify the 4 most valid cell to be annotated for the cluster. 
            - Provide a list of cells in the order of validity and explain your reasoning step by step. only give out the name of the cells from the given data.
            - Analyze only the data provided. Do not reference or infer from any external sources
            - the answer is in the format of 
                {
                cells : [cell1, cell2, cell3, cell4],
                explanation : <explanation>
                }
            """},
        {"role": "user", 
         "content": content}
    ],
)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `o3-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [282]:
print(completion.choices[0].message.parsed.model_dump_json(indent=4))

{
    "explanation": [
        "Each row in the dataset represents a known cell type and its expression of a specific gene, along with various metrics that describe the expression level and prevalence of that gene in the cell type.",
        "The 'Gene' column indicates the specific gene being analyzed, which is crucial for understanding the biological functions associated with the cell type.",
        "The 'Cell Name' column identifies the known cell type that expresses the gene, allowing for comparison with the unannotated cluster.",
        "The 'Expression Value' column shows the level of gene expression; higher values suggest stronger expression, which may indicate a more significant role of that gene in the cell type's function.",
        "The 'Cell Percentage' column indicates the proportion of cells expressing the gene; a higher percentage suggests that the gene is more commonly expressed within that cell type, which may correlate with its relevance to the unannotated cluster."

In [39]:
import json

output_dict = json.loads(completion.choices[0].message.parsed.model_dump_json(indent=4))

In [40]:
output_dict['cells']

[{'rank': 1,
  'cell_type': 'T cell',
  'reason': 'The T cell type expresses 5 genes from the cluster: KRT14 (1.929), IFI27 (1.713), LGALS3BP (1.698), RBM42 (1.662), and GAMT (1.604). It has a high cell count of 3174, a cell percentage of 0.0336, and a tissue composition of 0.0044, indicating a strong presence in the tissue.'},
 {'rank': 2,
  'cell_type': 'CD8-positive, alpha-beta T cell',
  'reason': 'This cell type expresses 5 genes: RBM42 (1.950), KRT14 (1.820), GAMT (1.751), LGALS3BP (1.713), and IFI27 (1.677). It has a cell count of 254, a cell percentage of 0.0187, and a tissue composition of 0.0003, showing a significant expression of relevant genes.'},
 {'rank': 3,
  'cell_type': 'CD4-positive, alpha-beta T cell',
  'reason': 'This cell type expresses 4 genes: IFI27 (2.030), LGALS3BP (1.680), RBM42 (1.637), and GAMT (1.554). It has a cell count of 91, a cell percentage of 0.0247, and a tissue composition of 0.0001, indicating a relevant but slightly lower expression compared to

# Color Test

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Sample DataFrame
data = {'Values': [5, 3, 8, 2, 9, 1, 7, 4, 6, 5, 3, 2]}
df = pd.DataFrame(data)

# Define normalization (scale from min=1 to max=9)
norm = mcolors.Normalize(vmin=1, vmax=9)
# Choose a colormap (reversed for light→dark)
cmap = plt.get_cmap('YlGnBu_r')  # Or 'Blues_r', 'Greys_r', etc.

# Map values to hex color codes
df['Color'] = df['Values'].apply(
    lambda x: mcolors.to_hex(cmap(norm(x)))  # Convert to hex
)

# Display DataFrame with hex color codes
print(df)

    Values    Color
0        5  #42b6c4
1        3  #225ea8
2        8  #edf8b2
3        2  #253494
4        9  #ffffd9
5        1  #081d58
6        7  #c8e9b4
7        4  #1d91c0
8        6  #80cebb
9        5  #42b6c4
10       3  #225ea8
11       2  #253494
